In [1]:
###IMPORT MODULES AS APPROPRIATE###

In [2]:
import nltk; nltk.download('stopwords')

C:\Users\brady\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\brady\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

True

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\brady\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\Users\brady\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
C:\Users\brady\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:31: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/relea

In [4]:
###INSERT STOP WORDS AS NEEDED###

In [5]:
#NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
###IMPORT DATA - THIS IS CURRENTLY UGLY BUT WHATEVER###

###try this later:
#import csv
#reviews = [row for row in csv.reader(open('reviews.csv'))]
#print reviews

In [7]:
df = [
"MicroRNAs (mi-RNAs) are critical regulators of gene expression. Amplification and overexpression of individual 'oncomiRs' or genetic loss of tumour suppressor mi-RNAs are associated with human cancer and are sufficient to drive tumorigenesis in mouse models. Furthermore, global miRNA depletion caused by genetic and epigenetic alterations in components of the miRNA biogenesis machinery is oncogenic. This, together with the recent identification of novel miRNA regulatory factors and pathways, highlights the importance of miRNA dysregulation in cancer.", 
"PD-L1 and PD-1 (PD) pathway blockade is a highly promising therapy and has elicited durable antitumor responses and long-term remissions in a subset of patients with a broad spectrum of cancers. How to improve, widen, and predict the clinical response to anti-PD therapy is a central theme in the field of cancer immunology and immunotherapy. Oncologic, immunologic, genetic, and biological studies focused on the human cancer microenvironment have yielded substantial insight into this issue. Here, we focus on tumor microenvironment and evaluate several potential therapeutic response markers including the PD-L1 and PD-1 expression pattern, genetic mutations within cancer cells and neoantigens, cancer epigenetics and effector T cell landscape, and microbiota. We further clarify the mechanisms of action of these markers and their roles in shaping, being shaped, and/or predicting therapeutic responses. We also discuss a variety of combinations with PD pathway blockade and their scientific rationales for cancer treatment.", 
"We performed an extensive immunogenomic analysis of more than 10,000 tumors comprising 33 diverse cancer types by utilizing data compiled by TCGA. Across cancer types, we identified six immune subtypes-wound healing, IFN-gamma dominant, inflammatory, lymphocyte depleted, immunologically quiet, and TGF-beta dominant-characterized by differences in macrophage or lymphocyte signatures, Th1:Th2 cell ratio, extent of intratumoral heterogeneity, aneuploidy, extent of neoantigen load, overall cell proliferation, expression of immunomodulatory genes, and prognosis. Specific driver mutations correlated with lower (CTNNB1, NRAS, or IDH1) or higher (BRAF, TP53, or CASP8) leukocyte levels across all cancers. Multiple control modalities of the intracellular and extracellular networks (transcription, microRNAs, copy number, and epigenetic processes) were involved in tumor-immune cell interactions, both across and within immune subtypes. Our immunogenomics pipeline to characterize these heterogeneous tumors and the resulting data are intended to serve as a resource for future targeted studies to further advance the field.", 
"The definition and classification of chronic kidney disease (CKD) have evolved over time, but current international guidelines define this condition as decreased kidney function shown by glomerular filtration rate (GFR) of less than 60 mL/min per 1.73 m(2), or markers of kidney damage, or both, of at least 3 months duration, regardless of the underlying cause. Diabetes and hypertension are the main causes of CKD in all high-income and middle-income countries, and also in many low-income countries. Incidence, prevalence, and progression of CKD also vary within countries by ethnicity and social determinants of health, possibly through epigenetic influence. Many people are asymptomatic or have nonspecific symptoms such as lethargy, itch, or loss of appetite. Diagnosis is commonly made after chance findings from screening tests (urinary dipstick or blood tests), or when symptoms become severe. The best available indicator of overall kidney function is GFR, which is measured either via exogenous markers (eg, DTPA, iohexol), or estimated using equations. Presence of proteinuria is associated with increased risk of progression of CKD and death. Kidney biopsy samples can show definitive evidence of CKD, through common changes such as glomerular sclerosis, tubular atrophy, and interstitial fibrosis. Complications include anaemia due to reduced production of erythropoietin by the kidney; reduced red blood cell survival and iron deficiency; and mineral bone disease caused by disturbed vitamin D, calcium, and phosphate metabolism. People with CKD are five to ten times more likely to die prematurely than they are to progress to end stage kidney disease. This increased risk of death rises exponentially as kidney function worsens and is largely attributable to death from cardiovascular disease, although cancer incidence and mortality are also increased. Health-related quality of life is substantially lower for people with CKD than for the general population, and falls as GFR declines. Interventions targeting specific symptoms, or aimed at supporting educational or lifestyle considerations, make a positive difference to people living with CKD. Inequity in access to services for this disease disproportionally affects disadvantaged populations, and health service provision to incentivise early intervention over provision of care only for advanced CKD is still evolving in many countries.", 
"Pancreatic cancer is a major cause of cancer-associated mortality, with a dismal overall prognosis that has remained virtually unchanged for many decades. Currently, prevention or early diagnosis at a curable stage is exceedingly difficult; patients rarely exhibit symptoms and tumours do not display sensitive and specific markers to aid detection. Pancreatic cancers also have few prevalent genetic mutations; the most commonly mutated genes are KRAS, CDKN2A (encoding p16), TP53 and SMAD4 - none of which are currently druggable. Indeed, therapeutic options are limited and progress in drug development is impeded because most pancreatic cancers are complex at the genomic, epigenetic and metabolic levels, with multiple activated pathways and crosstalk evident. Furthermore, the multilayered interplay between neoplastic and stromal cells in the tumour microenvironment challenges medical treatment. Fewer than 20% of patients have surgically resectable disease; however, neoadjuvant therapies might shift tumours towards resectability. Although newer drug combinations and multimodal regimens in this setting, as well as the adjuvant setting, appreciably extend survival, similar to 80% of patients will relapse after surgery and ultimately die of their disease. Thus, consideration of quality of life and overall survival is important. In this Primer, we summarize the current understanding of the salient pathophysiological, molecular, translational and clinical aspects of this disease. In addition, we present an outline of potential future directions for pancreatic cancer research and patient management.", 
"An increasingly recognized resistance mechanism to androgen receptor (AR)-directed therapy in prostate cancer involves epithelial plasticity, in which tumor cells demonstrate low to absent AR expression and often have neuroendocrine features. The etiology and molecular basis for this 'alternative' treatment-resistant cell state remain incompletely understood. Here, by analyzing whole-exome sequencing data of metastatic biopsies from patients, we observed substantial genomic overlap between castration-resistant tumors that were histologically characterized as prostate adenocarcinomas (CRPC-Adeno) and neuroendocrine prostate cancer (CRPC-NE); analysis of biopsy samples from the same individuals over time points to a model most consistent with divergent clonal evolution. Genome-wide DNA methylation analysis revealed marked epigenetic differences between CRPC-NE tumors and CRPC-Adeno, and also designated samples of CRPC-Adeno with clinical features of AR independence as CRPC-NE, suggesting that epigenetic modifiers may play a role in the induction and/or maintenance of this treatment-resistant state. This study supports the emergence of an alternative, 'AR-indifferent' cell state through divergent clonal evolution as a mechanism of treatment resistance in advanced prostate cancer.", 
"Renal cell carcinoma (RCC) denotes cancer originated from the renal epithelium and accounts for >90% of cancers in the kidney. The disease encompasses >10 histological and molecular subtypes, of which clear cell RCC (ccRCC) is most common and accounts for most cancer-related deaths. Although somatic VHL mutations have been described for some time, more-recent cancer genomic studies have identified mutations in epigenetic regulatory genes and demonstrated marked intra-tumour heterogeneity, which could have prognostic, predictive and therapeutic relevance. Localized RCC can be successfully managed with surgery, whereas metastatic RCC is refractory to conventional chemotherapy. However, over the past decade, marked advances in the treatment of metastatic RCC have been made, with targeted agents including sorafenib, sunitinib, bevacizumab, pazopanib and axitinib, which inhibit vascular endothelial growth factor (VEGF) and its receptor (VEGFR), and everolimus and temsirolimus, which inhibit mechanistic target of rapamycin complex1 (mTORC1), being approved. Since 2015, agents with additional targets aside from VEGFR have been approved, such as cabozantinib and lenvatinib; immunotherapies, such as nivolumab, have also been added to the armamentarium for metastatic RCC. Here, we provide an overview of the biology of RCC, with a focus on ccRCC, as well as updates to complement the current clinical guidelines and an outline of potential future directions for RCC research and therapy.", 
"Suicide is a complex public health problem of global importance. Suicidal behaviour differs between sexes, age groups, geographic regions, and sociopolitical settings, and variably associates with different risk factors, suggesting aetiological heterogeneity. Although there is no effective algorithm to predict suicide in clinical practice, improved recognition and understanding of clinical, psychological, sociological, and biological factors might help the detection of high-risk individuals and assist in treatment selection. Psychotherapeutic, pharmacological, or neuromodulatory treatments of mental disorders can often prevent suicidal behaviour; additionally, regular follow-up of people who attempt suicide by mental health services is key to prevent future suicidal behaviour.", 
"Recent studies have suggested that the intestinal microbiome plays an important role in modulating risk of several chronic diseases, including inflammatory bowel disease, obesity, type 2 diabetes, cardiovascular disease, and cancer. At the same time, it is now understood that diet plays a significant role in shaping the microbiome, with experiments showing that dietary alterations can induce large, temporary microbial shifts within 24 h. Given this association, there may be significant therapeutic utility in altering microbial composition through diet. This review systematically evaluates current data regarding the effects of several common dietary components on intestinal microbiota. We show that consumption of particular types of food produces predictable shifts in existing host bacterial genera. Furthermore, the identity of these bacteria affects host immune and metabolic parameters, with broad implications for human health. Familiarity with these associations will be of tremendous use to the practitioner as well as the patient.", 
"BACKGROUND Diffuse large B-cell lymphomas (DLBCLs) are phenotypically and genetically heterogeneous. Gene-expression profiling has identified subgroups of DLBCL (activated B-cell-like [ABC], germinal-center B-cell-like [GCB], and unclassified) according to cell of origin that are associated with a differential response to chemotherapy and targeted agents. We sought to extend these findings by identifying genetic subtypes of DLBCL based on shared genomic abnormalities and to uncover therapeutic vulnerabilities based on tumor genetics. METHODS We studied 574 DLBCL biopsy samples using exome and transcriptome sequencing, array-based DNA copy-number analysis, and targeted amplicon resequencing of 372 genes to identify genes with recurrent aberrations. We developed and implemented an algorithm to discover genetic subtypes based on the co-occurrence of genetic alterations. RESULTS We identified four prominent genetic subtypes in DLBCL, termed MCD (based on the co-occurrence of MYD88(L265P) and CD79B mutations), BN2 (based on BCL6 fusions and NOTCH2 mutations), N1 (based on NOTCH1 mutations), and EZB (based on EZH2 mutations and BCL2 translocations). Genetic aberrations in multiple genes distinguished each genetic subtype from other DLBCLs. These subtypes differed phenotypically, as judged by differences in gene-expression signatures and responses to immunochemotherapy, with favorable survival in the BN2 and EZB subtypes and inferior outcomes in the MCD and N1 subtypes. Analysis of genetic pathways suggested that MCD and BN2 DLBCLs rely on chronic active B-cell receptor signaling that is amenable to therapeutic inhibition. CONCLUSIONS We uncovered genetic subtypes of DLBCL with distinct genotypic, epigenetic, and clinical characteristics, providing a potential nosology for precision-medicine strategies in DLBCL.", 
"The MYC oncogene encodes a transcription factor, MYC, whose broad effects make its precise oncogenic role enigmatically elusive. The evidence to date suggests that MYC triggers selective gene expression amplification to promote cell growth and proliferation. Through its targets, MYC coordinates nutrient acquisition to produce ATP and key cellular building blocks that increase cell mass and trigger DNA replication and cell division. In cancer, genetic and epigenetic derangements silence checkpoints and unleash MYC's cell growth-and proliferation-promoting metabolic activities. Unbridled growth in response to deregulated MYC expression creates dependence on MYC-driven metabolic pathways, such that reliance on specific metabolic enzymes provides novel targets for cancer therapy. Significance: MYC's expression and activity are tightly regulated in normal cells by multiple mechanisms, including a dependence upon growth factor stimulation and replete nutrient status. In cancer, genetic deregulation of MYC expression and loss of checkpoint components, such as TP53, permit MYC to drive malignant transformation. However, because of the reliance of MYC-driven cancers on specific metabolic pathways, synthetic lethal interactions between MYC overexpression and specific enzyme inhibitors provide novel cancer therapeutic opportunities. (C) 2015 AACR.", 
"Cancer growth and progression are associated with immune suppression. Cancer cells have the ability to activate different immune checkpoint pathways that harbor immunosuppressive functions. Monoclonal antibodies that target immune checkpoints provided an immense breakthrough in cancer therapeutics. Among the immune checkpoint inhibitors, PD-1/PD-L1 and CTLA-4 inhibitors showed promising therapeutic outcomes, and some have been approved for certain cancer treatments, while others are under clinical trials. Recent reports have shown that patients with various malignancies benefit from immune checkpoint inhibitor treatment. However, mainstream initiation of immune checkpoint therapy to treat cancers is obstructed by the low response rate and immune-related adverse events in some cancer patients. This has given rise to the need for developing sets of biomarkers that predict the response to immune checkpoint blockade and immune-related adverse events. In this review, we discuss different predictive biomarkers for anti-PD-1/PD-L1 and anti-CTLA-4 inhibitors, including immune cells, PD-L1 overexpression, neoantigens, and genetic and epigenetic signatures. Potential approaches for further developing highly reliable predictive biomarkers should facilitate patient selection for and decision-making related to immune checkpoint inhibitor-based therapies.", 
"The hippocampus provided the gateway into much of what we have learned about stress and brain structural and functional plasticity, and this initial focus has expanded to other interconnected brain regions, such as the amygdala and prefrontal cortex. Starting with the discovery of adrenal steroid, and later, estrogen receptors in the hippocampal formation, and subsequent discovery of dendritic and spine synapse remodeling and neurogenesis in the dentate gyrus, mechanistic studies have revealed both genomic and rapid non-genomic actions of circulating steroid hormones in the brain. Many of these actions occur epigenetically and result in ever-changing patterns of gene expression, in which there are important sex differences that need further exploration. Moreover, glucocorticoid and estrogen actions occur synergistically with an increasing number of cellular mediators that help determine the qualitative nature of the response. The hippocampus has also been a gateway to understanding lasting epigenetic effects of early-life experiences. These findings in animal models have resulted in translation to the human brain and have helped change thinking about the nature of brain malfunction in psychiatric disorders and during aging, as well as the mechanisms of the effects of early-life adversity on the brain and the body.", 
"EMT and MET comprise the processes by which cells transit between epithelial and mesenchymal states, and they play integral roles in both normal development and cancer metastasis. This article reviews these processes and the molecular pathways that contribute to them. First, we compare embryogenesis and development with cancer metastasis. We then discuss the signaling pathways and the differential expression and down-regulation of receptors in both tumor cells and stromal cells, which play a role in EMT and metastasis. We further delve into the clinical implications of EMT and MET in several types of tumors, and lastly, we discuss the role of epigenetic events that regulate EMT/MET processes. We hypothesize that reversible epigenetic events regulate both EMT and MET, and thus, also regulate the development of different types of metastatic cancers.", 
"This review provides general information to serve as a primer for those embarking on understanding food allergy and also details advances and updates in epidemiology, pathogenesis, diagnosis, and treatment that have occurred over the 4 years since our last comprehensive review. Although firm prevalence data are lacking, there is a strong impression that food allergy has increased, and rates as high as approximately 10% have been documented. Genetic, epigenetic, and environmental risk factors are being elucidated increasingly, creating potential for improved prevention and treatment strategies targeted to those at risk. Insights on pathophysiology reveal a complex interplay of the epithelial barrier, mucosal and systemic immune response, route of exposure, and microbiome among other influences resulting in allergy or tolerance. The diagnosis of food allergy is largely reliant on medical history, tests for sensitization, and oral food challenges, but emerging use of component-resolved diagnostics is improving diagnostic accuracy. Additional novel diagnostics, such as basophil activation tests, determination of epitope binding, DNA methylation signatures, and bioinformatics approaches, will further change the landscape. A number of prevention strategies are under investigation, but early introduction of peanut has been advised as a public health measure based on existing data. Management remains largely based on allergen avoidance, but a panoply of promising treatment strategies are in phase 2 and 3 studies, providing immense hope that better treatment will be imminently and widely available, whereas numerous additional promising treatments are in the preclinical and clinical pipeline.", 
"Macrophages are myeloid immune cells that are strategically positioned throughout the body tissues, where they ingest and degrade dead cells, debris, and foreign material and orchestrate inflammatory processes. Here we review two major recent paradigm shifts in our understanding of tissue macrophage biology. The first is the realization that most tissue-resident macrophages are established prenatally and maintained through adulthood by longevity and self-renewal. Their generation and maintenance are thus independent from ongoing hematopoiesis, although the cells can be complemented by adult monocyte-derived macrophages. Second, aside from being immune sentinels, tissue macrophages form integral components of their host tissue. This entails their specialization in response to local environmental cues to contribute to the development and specific function of their tissue of residence. Factors that govern tissue macrophage specialization are emerging. Moreover, tissue specialization is reflected in discrete gene expression profiles of macrophages, as well as epigenetic signatures reporting actual and potential enhancer usage.", 
"The discovery of long non-coding RNA (lncRNA) has dramatically altered our understanding of cancer. Here, we describe a comprehensive analysis of lncRNA alterations at transcriptional, genomic, and epigenetic levels in 5,037 human tumor specimens across 13 cancer types from The Cancer Genome Atlas. Our results suggest that the expression and dysregulation of lncRNAs are highly cancer type specific compared with protein- coding genes. Using the integrative data generated by this analysis, we present a clinically guided small interfering RNA screening strategy and a co-expression analysis approach to identify cancer driver lncRNAs and predict their functions. This provides a resource for investigating lncRNAs in cancer and lays the groundwork for the development of new diagnostics and treatments.", 
"Multidrug resistance (MDR) is a serious phenomenon employed by cancer cells which hampers the success of cancer pharmacotherapy. One of the common mechanisms of MDR is the overexpression of ATP-binding cassette (ABC) efflux transporters in cancer cells such as P-glycoprotein (P-gp/ABCB1), multidrug resistance-associated protein 2 (MRP2/ABCC2), and breast cancer resistance protein (BCRP/ABCG2) that limits the prolonged and effective use of chemotherapeutic drugs. Researchers have found that developing inhibitors of ABC efflux transporters as chemosensitizers could overcome MDR. But the clinical trials have shown that most of these chemosensitizers are merely toxic and only show limited or no benefits to cancer patients, thus new inhibitors are being explored. Recent findings also suggest that efflux pumps of the ABC transporter family are subject to epigenetic gene regulation. In this review, we summarize recent findings of the role of ABC efflux transporters in MDR. (C) 2015 Elsevier Ireland Ltd. All rights reserved.", 
"The mechanisms underlying human natural killer (NK) cell phenotypic and functional heterogeneity are unknown. Here, we describe the emergence of diverse subsets of human NK cells selectively lacking expression of signaling proteins after human cytomegalovirus (HCMV) infection. The absence of Band myeloid cell-related signaling protein expression in these NK cell subsets correlated with promoter DNA hypermethylation. Genome-wide DNA methylation patterns were strikingly similar between HCMV-associated adaptive NK cells and cytotoxic effector T cells but differed from those of canonical NK cells. Functional interrogation demonstrated altered cytokine responsiveness in adaptive NK cells that was linked to reduced expression of the transcription factor PLZF. Furthermore, subsets of adaptive NK cells demonstrated significantly reduced functional responses to activated autologous T cells. The present results uncover a spectrum of epigenetically unique adaptive NK cell subsets that diversify in response to viral infection and have distinct functional capabilities compared to canonical NK cell subsets.", 
"Cancer tissues are composed of cancer cells and the surrounding stromal cells (e.g., fibroblasts, vascular endothelial cells, and immune cells), in addition to the extracellular matrix. Most studies investigating carcinogenesis and the progression, invasion, metastasis, and angiogenesis of cancer have focused on alterations in cancer cells, including genetic and epigenetic changes. Recently, interactions between cancer cells and the stroma have attracted considerable attention, and increasing evidence has accumulated on this. Several researchers have gradually clarified the origins, features, and roles of cancer-associated fibroblasts (CAFs), a major component of the cancer stroma. CAFs function in a similar manner to myofibroblasts during wound healing. We previously reported the relationship between CAFs and angiogenesis. Interleukin-6 (IL-6), a multifunctional cytokine, plays a central role in regulating inflammatory and immune responses, and important roles in the progression, including proliferation, migration, and angiogenesis, of several cancers. We showed that CAFs are an important IL-6 source and that anti-IL-6 receptor antibody suppressed angiogenesis and inhibited tumor-stroma interactions. Furthermore, CAFs contribute to drug-resistance acquisition in cancer cells. The interaction between cancer cells and the stroma could be a potential target for anti-cancer therapy.", 
"The 'hallmarks of cancer' are generally accepted as a set of genetic and epigenetic alterations that a normal cell must accrue to transform into a fully malignant cancer. It follows that therapies designed to counter these alterations might be effective as anti-cancer strategies. Over the past 30 years, research on the BCL-2-regulated apoptotic pathway has led to the development of small-molecule compounds, known as 'BH3-mimetics', that bind to pro-survival BCL-2 proteins to directly activate apoptosis of malignant cells. This Timeline article focuses on the discovery and study of BCL-2, the wider BCL-2 protein family and, specifically, its roles in cancer development and therapy.", 
"The extent of tumor heterogeneity is an emerging theme that researchers are only beginning to understand. How genetic and epigenetic heterogeneity affects tumor evolution and clinical progression is unknown. The precise nature of the environmental factors that influence this heterogeneity is also yet to be characterized. Nature Medicine, Nature Biotechnology and the Volkswagen Foundation organized a meeting focused on identifying the obstacles that need to be overcome to advance translational research in and tumor heterogeneity. Once these key questions were established, the attendees devised potential solutions. Their ideas are presented here.", 
"Humans show strong sex differences in immunity to infection and autoimmunity, suggesting sex hormones modulate immune responses. Indeed, receptors for estrogens (ERs) regulate cells and pathways in the innate and adaptive immune system, as well as immune cell development. ERs are ligand-dependent transcription factors that mediate long-range chromatin interactions and form complexes at gene regulatory elements, thus promoting epigenetic changes and transcription. ERs also participate in membrane-initiated steroid signaling to generate rapid responses. Estradiol and ER activity show profound dose- and context-dependent effects on innate immune signaling pathways and myeloid cell development. While estradiol most often promotes the production of type I interferon, innate pathways leading to pro-inflammatory cytokine production may be enhanced or dampened by ER activity. Regulation of innate immune cells and signaling by ERs may contribute to the reported sex differences in innate immune pathways. Here we review the recent literature and highlight several molecular mechanisms by which ERs regulate the development or functional responses of innate immune cells. (C) 2015 Elsevier Inc. All rights reserved.", 
"Evasion of immune system is a hallmark of cancer, which enables cancer cells to escape the attack from immune cells. Cancer cells can express many immune inhibitory signalling proteins to cause immune cell dysfunction and apoptosis. One of these inhibitory molecules is programmed death-ligand-1 (PD-L1), which binds to programmed death-1 (PD-1) expressed on T-cells, B-cells, dendritic cells and natural killer T-cells to suppress anti-cancer immunity. Therefore, anti-PD-L1 and anti-PD-1 antibodies have been used for the treatment of cancer, showing promising outcomes. However, only a proportion of patients respond to the treatments. Further understanding of the regulation of PD-L1 expression could be helpful for the improvement of anti-PD-L1 and anti-PD-1 treatments. Studies have shown that PD-L1 expression is regulated by signalling pathways, transcriptional factors and epigenetic factors. In this review, we summarise the recent progress of the regulation of PD-L1 expression in cancer cells and propose a regulatory model for unified explanation. Both PI3K and MAPK pathways are involved in PD-L1 regulation but the downstream molecules that control PD-L1 and cell proliferation may differ. Transcriptional factors hypoxia-inducible factor-1 alpha and signal transducer and activation of transcription-3 act on the promoter of PD-L1 to regulate its expression. In addition, microRNAs including miR-570, miR-513, miR-197, miR-34a and miR-200 negatively regulate PD-L1. Clinically, it could increase treatment efficacy of targeted therapy by choosing those molecules that control both PD-L1 expression and cell proliferation.", 
"Excessive fat deposition in obesity has a multifactorial aetiology, but is widely considered the result of disequilibrium between energy intake and expenditure. Despite specific public health policies and individual treatment efforts to combat the obesity epidemic, >2 billion people worldwide are overweight or obese. The central nervous system circuitry, fuel turnover and metabolism as well as adipose tissue homeostasis are important to comprehend excessive weight gain and associated comorbidities. Obesity has a profound impact on quality of life, even in seemingly healthy individuals. Diet, physical activity or exercise and lifestyle changes are the cornerstones of obesity treatment, but medical treatment and bariatric surgery are becoming important. Family history, food environment, cultural preferences, adverse reactions to food, perinatal nutrition, previous or current diseases and physical activity patterns are relevant aspects for the health care professional to consider when treating the individual with obesity. Clinicians and other health care professionals are often ill-equipped to address the important environmental and socioeconomic drivers of the current obesity epidemic. Finally, understanding the epigenetic and genetic factors as well as metabolic pathways that take advantage of 'omics' technologies could play a very relevant part in combating obesity within a precision approach.", 
"Macrophages have protective roles in immunity to pathogens, tissue development, homeostasis and repair following damage. Maladaptive immunity and inflammation provoke changes in macrophage function that are causative of disease. Despite a historical wealth of knowledge about macrophages, recent advances have revealed unknown aspects of their development and function. Following development, macrophages are activated by diverse signals. Such tissue microenvironmental signals together with epigenetic changes influence macrophage development, activation and functional diversity, with consequences in disease and homeostasis. We discuss here how recent discoveries in these areas have led to a multidimensional concept of macrophage ontogeny, activation and function. In connection with this, we also discuss how technical advances facilitate a new roadmap for the isolation and analysis of macrophages at high resolution.", 
"Progressive liver fibrosis, induced by chronic viral and metabolic disorders, leads to more than one million deaths annually via development of cirrhosis, although no antifibrotic therapy has been approved to date. Transdifferentiation (or activation) of hepatic stellate cells is the major cellular source of matrix protein-secreting myofibroblasts, the major driver of liver fibrogenesis. Paracrine signals from injured epithelial cells, fibrotic tissue microenvironment, immune and systemic metabolic dysregulation, enteric dysbiosis, and hepatitis viral products can directly or indirectly induce stellate cell activation. Dysregulated intracellular signaling, epigenetic changes, and cellular stress response represent candidate targets to deactivate stellate cells by inducing reversion to inactivated state, cellular senescence, apoptosis, and/or clearance by immune cells. Cell type- and target-specific pharmacological intervention to therapeutically induce the deactivation will enable more effective and less toxic precision antifibrotic therapies. (C) 2017 Elsevier B.V. All rights reserved.", 
"In recent years, there has been a tremendous and growing interest among researchers to investigate the role of mircoRNA (miRNA) in normal cellular as well as in disease processes. miRNAs are a family of small non-coding RNAs which were reported to regulate the expression of various oncogenes or tumor suppressor genes. The expression profiling of miRNAs has already entered into cancer clinics as diagnostic and prognostic biomarkers to assess tumor initiation, progression and response to treatment in cancer patients. This review summarizes: (i) the current understanding of interactions between miRNAs and their target genes, (ii) recent advances in the regulatory mechanisms that control the expression of genes related to carcinogenesis, and (iii) the role of miRNAs in cancer diagnosis and therapy.", 
"Irritable bowel syndrome (IBS) is a functional gastrointestinal disease with a high population prevalence. The disorder can be debilitating in some patients, whereas others may have mild or moderate symptoms. The most important single risk factors are female sex, younger age and preceding gastrointestinal infections. Clinical symptoms of IBS include abdominal pain or discomfort, stool irregularities and bloating, as well as other somatic, visceral and psychiatric comorbidities. Currently, the diagnosis of IBS is based on symptoms and the exclusion of other organic diseases, and therapy includes drug treatment of the predominant symptoms, nutrition and psychotherapy. Although the underlying pathogenesis is far from understood, aetiological factors include increased epithelial hyperpermeability, dysbiosis, inflammation, visceral hypersensitivity, epigenetics and genetics, and altered brain-gut interactions. IBS considerably affects quality of life and imposes a profound burden on patients, physicians and the health-care system. The past decade has seen remarkable progress in our understanding of functional bowel disorders such as IBS that will be summarized in this Primer.", 
"Approximately 50% of epithelial ovarian cancers (EOC) exhibit defective DNA repair via homologous recombination (HR) due to genetic and epigenetic alterations of HR pathway genes. Defective HR is an important therapeutic target in EOC as exemplified by the efficacy of platinum analogues in this disease, as well as the advent of PARP inhibitors, which exhibit synthetic lethality when applied to HR-deficient cells. Here, we describe the genotypic and phenotypic characteristics of HR-deficient EOCs, discuss current and emerging approaches for targeting these tumors, and present challenges associated with these approaches, focusing on development and overcoming resistance. Significance: Defective DNA repair via HR is a pivotal vulnerability of EOC, particularly of the high-grade serous histologic subtype. Targeting defective HR offers the unique opportunity of exploiting molecular differences between tumor and normal cells, thereby inducing cancer-specific synthetic lethality; the promise and challenges of these approaches in ovarian cancer are discussed in this review. (C)2015 AACR.", 
"Anticancer drugs resistance is a complex process that arises from altering in the drug targets. Advances in the DNA microarray, proteomics technology and the development of targeted therapies provide the new strategies to overcome the drug resistance. Although a design of the new chemotherapy agents is growing quickly, effective chemotherapy agent has not been discovered against the advanced stage of cancer (such as invasion and metastasis). The cancer cell resistance against the anticancer agents can be due to many factors such as the individual's genetic differences, especially in tumoral somatic cells. Also, the cancer drug resistance is acquired, the drug resistance can be occurred by different mechanisms, including multi-drug resistance, cell death inhibiting (apoptosis suppression), altering in the drug metabolism, epigenetic and drug targets, enhancing DNA repair and gene amplification. In this review, we outlined the mechanisms of cancer drug resistance and in following, the treatment failures by common chemotherapy agents in the different type of cancers.", 
"The endothelins comprise three structurally similar 21-amino acid peptides. Endothelin-1 and -2 activate two G-protein coupled receptors, ETA and ETB, with equal affinity, whereas endothelin-3 has a lower affinity for the ETA subtype. Genes encoding the peptides are present only among vertebrates. The ligand-receptor signaling pathway is a vertebrate innovation and may reflect the evolution of endothelin-1 as the most potent vasoconstrictor in the human cardiovascular system with remarkably long lasting action. Highly selective peptide ETA and ETB antagonists and ETB agonists together with radiolabeled analogs have accurately delineated endothelin pharmacology in humans and animal models, although surprisingly no ETA agonist has been discovered. ET antagonists (bosentan, ambrisentan) have revolutionized the treatment of pulmonary arterial hypertension, with the next generation of antagonists exhibiting improved efficacy (macitentan). Clinical trials continue to explore new applications, particularly in renal failure and for reducing proteinuria in diabetic nephropathy. Translational studies suggest a potential benefit of ETB agonists in chemotherapy and neuroprotection. However, demonstrating clinical efficacy of combined inhibitors of the endothelin converting enzyme and neutral endopeptidase has proved elusive. Over 28 genetic modifications have been made to the ET system in mice through global or cell-specific knockouts, knock ins, or alterations in gene expression of endothelin ligands or their target receptors. These studies have identified key roles for the endothelin isoforms and new therapeutic targets in development, fluid-electrolyte homeostasis, and cardiovascular and neuronal function. For the future, novel pharmacological strategies are emerging via small molecule epigenetic modulators, biologicals such as ETB monoclonal antibodies and the potential of signaling pathway biased agonists and antagonists.", 
"Exhausted CD8 T (Tex) cells are a distinct cell lineage that arise during chronic infections and cancers in animal models and humans. Tex cells are characterized by progressive loss of effector functions, high and sustained inhibitory receptor expression, metabolic dysregulation, poor memory recall and homeostatic self-renewal, and distinct transcriptional and epigenetic programs. The ability to reinvigorate Tex cells through inhibitory receptor blockade, such as alpha PD-1, highlights the therapeutic potential of targeting this population. Emerging insights into the mechanisms of exhaustion are informing immunotherapies for cancer and chronic infections. However, like other immune cells, Tex cells are heterogeneous and include progenitor and terminal subsets with unique characteristics and responses to checkpoint blockade. Here, we review our current understanding of Tex cell biology, including the developmental paths, transcriptional and epigenetic features, and cell intrinsic and extrinsic factors contributing to exhaustion and how this knowledge may inform therapeutic targeting of Tex cells in chronic infections, autoimmunity, and cancer.", 
"It is now well established that the immune system can recognize developing cancers and that therapeutic manipulation of immunity can induce tumor regression. The capacity to manifest remarkably durable responses in some patients has been ascribed in part to T cells that can (a) kill tumor cells directly, (b) orchestrate diverse antitumor immune responses, (c) manifest long-lasting memory, and (d) display remarkable specificity for tumor-derived proteins. This specificity stems from fundamental differences between cancer cells and their normal counterparts in that the former develop protein-altering mutations and undergo epigenetic and genetic alterations, resulting in aberrant protein expression. These events can result in formation of tumor antigens. The identification of mutated and aberrantly expressed self-tumor antigens has historically been time consuming and laborious. While mutant antigens are usually expressed in a tumor-specific manner, aberrantly expressed antigens are often shared between cancers and, therefore, in the past, have been the major focus of therapeutic cancer vaccines. However, advances in next-generation sequencing and epitope prediction now permit the rapid identification of mutant tumor neoantigens. This review focuses on a discussion of mutant tumor neoantigens and their use in personalizing cancer immunotherapies.", 
"Advances in single-cell RNA sequencing (scRNA-seq) have allowed for comprehensive analysis of the immune system. In this Review, we briefly describe the available scRNA-seq technologies together with their corresponding strengths and weaknesses. We discuss in depth how scRNA-seq can be used to deconvolve immune system heterogeneity by identifying novel distinct immune cell subsets in health and disease, characterizing stochastic heterogeneity within a cell population and building developmental 'trajectories' for immune cells. Finally, we discuss future directions of the field and present integrated approaches to complement molecular information from a single cell with studies of the environment, epigenetic state and cell lineage.", 
"Long-lived memory-like'' NK cells have been identified in individuals infected by human cytomegalovirus (HCMV), but little is known about how the memory-like NK cell pool is formed. Here, we have shown that HCMV-infected individuals have several distinct subsets of memory-like NK cells that are often deficient for multiple transcription factors and signaling proteins, including tyrosine kinase SYK, for which the reduced expression was stable over time and correlated with epigenetic modification of the gene promoter. Deficient expression of these proteins was largely confined to the recently discovered FcRg-deficient NK cells that display enhanced antibody-dependent functional activity. Importantly, FcRg-deficient NK cells exhibited robust preferential expansion in response to virus-infected cells (both HCMV and influenza) in an antibody-dependent manner. These findings suggest that the memory-like NK cell pool is shaped and maintained by a mechanism that involves both epigenetic modification of gene expression and antibody-dependent expansion.", 
"Extensive research over the past half a century indicates that reactive oxygen species (ROS) play an important role in cancer. Although low levels of ROS can be beneficial, excessive accumulation can promote cancer. One characteristic of cancer cells that distinguishes them from normal cells is their ability to produce increased numbers of ROS and their increased dependence on an antioxidant defense system. ROS are produced as a byproduct intracellularly by mitochondria and other cellular elements and exogenously by pollutants, tobacco, smoke, drugs, xenobiotics, and radiation. ROS modulate various cell signaling pathways, which are primarily mediated through the transcription factors NF-kappa B and STAT3, hypoxia-inducible factor-1 alpha, kinases, growth factors, cytokines and other proteins, and enzymes; these pathways have been linked to cellular transformation, inflammation, tumor survival, proliferation, invasion, angiogenesis, and metastasis of cancer. ROS are also associated with epigenetic changes in genes, which is helpful in diagnosing diseases. This review considers the role of ROS in the various stages of cancer development. Finally, we provide evidence that nutraceuticals derived from Mother Nature are highly effective in eliminating cancer cells. (C) 2016 Elsevier Ireland Ltd. All rights reserved.", 
"There are few in vitro models of exocrine pancreas development and primary human pancreatic adenocarcinoma (PDAC). We establish three-dimensional culture conditions to induce the differentiation of human pluripotent stem cells into exocrine progenitor organoids that form ductal and acinar structures in culture and in vivo. Expression of mutant KRAS or TP53 in progenitor organoids induces mutation-specific phenotypes in culture and in vivo. Expression of TP53(R175H) induces cytosolic SOX9 localization. In patient tumors bearing TP53 mutations, SOX9 was cytoplasmic and associated with mortality. We also define culture conditions for clonal generation of tumor organoids from freshly resected PDAC. Tumor organoids maintain the differentiation status, histoarchitecture and phenotypic heterogeneity of the primary tumor and retain patient-specific physiological changes, including hypoxia, oxygen consumption, epigenetic marks and differences in sensitivity to inhibition of the histone methyltransferase EZH2. Thus, pancreatic progenitor organoids and tumor organoids can be used to model PDAC and fat drug screening to identify precision therapy strategies.", 
"Arsenic (As) is ubiquitous in nature and humans being exposed to arsenic via atmospheric air, ground water and food sources are certain. Major sources of arsenic contamination could be either through geological or via anthropogenic activities. In physiological individuals, organ system is described as group of organs that transact collectively and associate with other systems for conventional body functions. Arsenic has been associated with persuading a variety of complications in body organ systems: integumentary, nervous, respiratory, cardiovascular, hematopoietic, immune, endocrine, hepatic, renal, reproductive system and development. In this review, we outline the effects of arsenic on the human body with a main focus on assorted organ systems with respective disease conditions. Additionally, underlying mechanisms of disease development in each organ system due to arsenic have also been explored. Strikingly, arsenic has been able to induce epigenetic changes (in utero) and genetic mutations (a leading cause of cancer) in the body. Occurrence of various arsenic induced health effects involving emerging areas such as epigenetics and cancer along with their respective mechanisms are also briefly discussed. (C) 2015 Elsevier B.V. All rights reserved.", 
"Parental environmental factors, including diet, body composition, metabolism, and stress, affect the health and chronic disease risk of people throughout their lives, as captured in the Developmental Origins of Health and Disease concept. Research across the epidemiological, clinical, and basic science fields has identified the period around conception as being crucial for the processes mediating parental influences on the health of the next generation. During this time, from the maturation of gametes through to early embryonic development, parental lifestyle can adversely influence long-term risks of offspring cardiovascular, metabolic, immune, and neurological morbidities, often termed developmental programming. We review periconceptional induction of disease risk from four broad exposures: maternal overnutrition and obesity; maternal undernutrition; related paternal factors; and the use of assisted reproductive treatment. Studies in both humans and animal models have demonstrated the underlying biological mechanisms, including epigenetic, cellular, physiological, and metabolic processes. We also present a meta-analysis of mouse paternal and maternal protein undernutrition that suggests distinct parental periconceptional contributions to postnatal outcomes. We propose that the evidence for periconceptional effects on lifetime health is now so compelling that it calls for new guidance on parental preparation for pregnancy, beginning before conception, to protect the health of offspring.", 
"Long non-coding RNAs (lncRNAs) refer to a group of RNAs that are usually more than 200 nucleotides and are not involved in protein generation. Instead, lncRNAs are involved in different regulatory processes, such as regulation of gene expression. Different lncRNAs exist throughout the genome. LncRNAs are also known for their roles in different human diseases such as cancer. HOTAIR is an lncRNA that plays a role as an oncogenic molecule in different cancer cells, such as breast, gastric, colorectal, and cervical cancer cells. Therefore, HOTAIR expression level is a potential biomarker for diagnostic and therapeutic purposes in several cancers. This RNA takes part in epigenetic regulation of genes and plays an important role in different cellular pathways by interacting with Polycomb Repressive Complex 2 (PRC2). In this review, we describe the molecular function and regulation of HOTAIR and its role in different types of cancers.", 
"Mutant isocitrate dehydrogenase (IDH) 1 and 2 proteins alter the epigenetic landscape in acute myeloid leukemia (AML) cells through production of the oncometabolite (R)-2-hydroxyglutarate (2-HG). Here we performed a large-scale RNA interference (RNAi) screen to identify genes that are synthetic lethal to the IDH1(R132H) mutation in AML and identified the anti-apoptotic gene BCL-2. IDH1- and IDH2-mutant primary human AML cells were more sensitive than IDH1/2 wildtype cells to ABT-199, a highly specific BCL-2 inhibitor that is currently in clinical trials for hematologic malignancies, both ex vivo and in xenotransplant models. This sensitization effect was induced by (R)-2-HG-mediated inhibition of the activity of cytochrome c oxidase (COX) in the mitochondrial electron transport chain (ETC); suppression of COX activity lowered the mitochondrial threshold to trigger apoptosis upon BCL-2 inhibition. Our findings indicate that IDH1/2 mutation status may identify patients that are likely to respond to pharmacologic BCL-2 inhibition and form the rational basis for combining agents that disrupt ETC activity with ABT-199 in future clinical studies.", 
"T cell dysfunction is a hallmark of many cancers, but the basis for T cell dysfunction and the mechanisms by which antibody blockade of the inhibitory receptor PD-1 (anti-PD-1) reinvigorates T cells are not fully understood. Here we show that such therapy acts on a specific subpopulation of exhausted CD8(+) tumor-infiltrating lymphocytes (TILs). Dysfunctional CD8(+) TILs possess canonical epigenetic and transcriptional features of exhaustion that mirror those seen in chronic viral infection. Exhausted CD8(+) TILs include a subpopulation of 'progenitor exhausted' cells that retain polyfunctionality, persist long term and differentiate into 'terminally exhausted' TILs. Consequently, progenitor exhausted CD8(+) TILs are better able to control tumor growth than are terminally exhausted T cells. Progenitor exhausted TILs can respond to anti-PD-1 therapy, but terminally exhausted TILs cannot. Patients with melanoma who have a higher percentage of progenitor exhausted cells experience a longer duration of response to checkpoint-blockade therapy. Thus, approaches to expand the population of progenitor exhausted CD8(+) T cells might be an important component of improving the response to checkpoint blockade.", 
"Apoptosis or programmed cell death is natural way of removing aged cells from the body. Most of the anticancer therapies trigger apoptosis induction and related cell death networks to eliminate malignant cells. However, in cancer, de-regulated apoptotic signaling, particularly the activation of an anti-apoptotic systems, allows cancer cells to escape this program leading to uncontrolled proliferation resulting in tumor survival, therapeutic resistance and recurrence of cancer. This resistance is a complicated phenomenon that emanates from the interactions of various molecules and signaling pathways. In this comprehensive review we discuss the various factors contributing to apoptosis resistance in cancers. The key resistance targets that are discussed include (1) BcI-2 and Mcl-1 proteins; (2) autophagy processes; (3) necrosis and necroptosis; (4) heat shock protein signaling; (5) the proteasome pathway; (6) epigenetic mechanisms; and (7) aberrant nuclear export signaling. The shortcomings of current therapeutic modalities are highlighted and a broad spectrum strategy using approaches including (a) gossypol; (b) epigallocatechin-3-gallate; (c) UMI-77 (d) triptolide and (e) selinexor that can be used to overcome cell death resistance is presented. This review provides a roadmap for the design of successful anti-cancer strategies that overcome resistance to apoptosis for better therapeutic outcome in patients with cancer. (C) 2015 Elsevier Ltd.", 
"Epigenetic dysregulation has emerged as an important mechanism in cancer. Alterations in epigenetic machinery have become a major focus for targeted therapies. The current report describes the discovery and biological activity of a cyclopropylamine containing inhibitor of Lysine Demethylase 1 (LSD1), GSK2879552. This small molecule is a potent, selective, orally bioavailable, mechanism-based irreversible inactivator of LSD1. A proliferation screen of cell lines representing a number of tumor types indicated that small cell lung carcinoma (SCLC) is sensitive to LSD1 inhibition. The subset of SCLC lines and primary samples that undergo growth inhibition in response to GSK2879552 exhibit DNA hypomethylation of a signature set of probes, suggesting this may be used as a predictive biomarker of activity.", 
"Over the last several decades, it has become clear that epigenetic abnormalities may be one of the hallmarks of cancer. Posttranslational modifications of histones, for example, may play a crucial role in cancer development and progression by modulating gene transcription, chromatin remodeling, and nuclear architecture. Histone acetylation, a well-studied post-translational histone modification, is controlled by the opposing activities of histone acetyl-transferases (HATs) and histone deacetylases (HDACs). By removing acetyl groups, HDACs reverse chromatin acetylation and alter transcription of oncogenes and tumor suppressor genes. In addition, HDACs deacetylate numerous nonhistone cellular substrates that govern a wide array of biological processes including cancer initiation and progression. This review will discuss the role of HDACs in cancer and the therapeutic potential of HDAC inhibitors (HDACi) as emerging drugs in cancer treatment.", 
"Histone modifications are key epigenetic regulatory features that have important roles in many cellular events. Lysine methylations mark various sites on the tail and globular domains of histones and their levels are precisely balanced by the action of methyltransferases ('writers') and demethylases ('erasers'). In addition, distinct effector proteins ('readers') recognize specific methyl-lysines in a manner that depends on the neighboring amino-acid sequence and methylation state. Misregulation of histone lysine methylation has been implicated in several cancers and developmental defects. Therefore, histone lysine methylation has been considered a potential therapeutic target, and clinical trials of several inhibitors of this process have shown promising results. A more detailed understanding of histone lysine methylation is necessary for elucidating complex biological processes and, ultimately, for developing and improving disease treatments. This review summarizes enzymes responsible for histone lysine methylation and demethylation and how histone lysine methylation contributes to various biological processes.", 
"DNA methylation is known to be abnormal in all forms of cancer, but it is not really understood how this occurs and what is its role in tumorigenesis. In this review, we take a wide view of this problem by analyzing the strategies involved in setting up normal DNA methylation patterns and understanding how this stable epigenetic mark works to prevent gene activation during development. Aberrant DNA methylation in cancer can be generated either prior to or following cell transformation through mutations. Increasing evidence suggests, however, that most methylation changes are generated in a programmed manner and occur in a subpopulation of tissue cells during normal aging, probably predisposing them for tumorigenesis. It is likely that this methylation contributes to the tumor state by inhibiting the plasticity of cell differentiation processes. (C) 2016 AACR.", 
"Cancer is driven by genetic and epigenetic alterations that allow cells to overproliferate and escape mechanisms that normally control their survival and migration. Many of these alterations map to signaling pathways that control cell growth and division, cell death, cell fate, and cell motility, and can be placed in the context of distortions of wider signaling networks that fuel cancer progression, such as changes in the tumor microenvironment, angiogenesis, and inflammation. Mutations that convert cellular proto-oncogenes to oncogenes can cause hyper-activation of these signaling pathways, whereas inactivation of tumor suppressors eliminates critical negative regulators of signaling. An examination of the PI3K-Akt and Ras-ERK pathways illustrates how such alterations dysregulate signaling in cancer and produce many of the characteristic features of tumor cells.", 
"Macrophages are immune cells of haematopoietic origin that provide crucial innate immune defence and have tissue-specific functions in the regulation and maintenance of organ homeostasis. Recent studies of macrophage ontogeny, as well as transcriptional and epigenetic identity, have started to reveal the decisive role of the tissue stroma in the regulation of macrophage function. These findings suggest that most macrophages seed the tissues during embryonic development and functionally specialize in response to cytokines and metabolites that are released by the stroma and drive the expression of unique transcription factors. In this Review, we discuss how recent insights into macrophage ontogeny and macrophage stroma interactions contribute to our understanding of the crosstalk that shapes macrophage function and the maintenance of organ integrity.", 
"Diversity and plasticity are hallmarks of cells of the monocyte-macrophage lineage. In response to IFNs, Toll-like receptor engagement, or IL-4/IL-13 signaling, macrophages undergo M1 (classical) or M2 (alternative) activation, which represent extremes of a continuum in a universe of activation states. Progress has now been made in defining the signaling pathways, transcriptional networks, and epigenetic mechanisms underlying M1-M2 or M2-like polarized activation. Functional skewing of mononuclear phagocytes occurs in vivo under physiological conditions (e.g., ontogenesis and pregnancy) and in pathology (allergic and chronic inflammation, tissue repair, infection, and cancer). However, in selected preclinical and clinical conditions, coexistence of cells in different activation states and unique or mixed phenotypes have been observed, a reflection of dynamic changes and complex tissue-derived signals. The identification of mechanisms and molecules associated with macrophage plasticity and polarized activation provides a basis for macrophage-centered diagnostic and therapeutic strategies.", 
"BACKGROUND Many mutations that contribute to the pathogenesis of acute myeloid leukemia (AML) are undefined. The relationships between patterns of mutations and epigenetic phenotypes are not yet clear. METHODS We analyzed the genomes of 200 clinically annotated adult cases of de novo AML, using either whole-genome sequencing (50 cases) or whole-exome sequencing (150 cases), along with RNA and microRNA sequencing and DNA-methylation analysis. RESULTS AML genomes have fewer mutations than most other adult cancers, with an average of only 13 mutations found in genes. Of these, an average of 5 are in genes that are recurrently mutated in AML. A total of 23 genes were significantly mutated, and another 237 were mutated in two or more samples. Nearly all samples had at least 1 nonsynonymous mutation in one of nine categories of genes that are almost certainly relevant for pathogenesis, including transcription-factor fusions (18% of cases), the gene encoding nucleophosmin (NPM1) (27%), tumor-suppressor genes (16%), DNA-methylation-related genes (44%), signaling genes (59%), chromatin-modifying genes (30%), myeloid transcription-factor genes (22%), cohesin-complex genes (13%), and spliceosome-complex genes (14%). Patterns of cooperation and mutual exclusivity suggested strong biologic relationships among several of the genes and categories. CONCLUSIONS We identified at least one potential driver mutation in nearly all AML samples and found that a complex interplay of genetic events contributes to AML pathogenesis in individual patients. The databases from this study are widely available to serve as a foundation for further investigations of AML pathogenesis, classification, and risk stratification.", 
"CD4 T cells play critical roles in mediating adaptive immunity to a variety of pathogens. They are also involved in autoimmunity, asthma, and allergic responses as well as in tumor immunity. During TCR activation in a particular cytokine milieu, naive CD4 T cells may differentiate into one of several lineages of T helper (Th) cells, including Th1, Th2, Th17, and iTreg, as defined by their pattern of cytokine production and function. In this review, we summarize the discovery, functions, and relationships among Th cells; the cytokine and signaling requirements for their development; the networks of transcription factors involved in their differentiation; the epigenetic regulation of their key cytokines and transcription factors; and human diseases involving defective CD4 T cell differentiation.", 
"The past decade has highlighted the central role of epigenetic processes in cancer causation, progression and treatment. Next-generation sequencing is providing a window for visualizing the human epigenome and how it is altered in cancer. This view provides many surprises, including linking epigenetic abnormalities to mutations in genes that control DNA methylation, the packaging and the function of DNA in chromatin, and metabolism. Epigenetic alterations are leading candidates for the development of specific markers for cancer detection, diagnosis and prognosis. The enzymatic processes that control the epigenome present new opportunities for deriving therapeutic strategies designed to reverse transcriptional abnormalities that are inherent to the cancer epigenome.", 
"Cancer-associated IDH mutations are characterized by neomorphic enzyme activity and resultant 2-hydroxyglutarate (2HG) production. Mutational and epigenetic profiling of a large acute myeloid leukemia (AML) patient cohort revealed that IDH1/2-mutant AMLs display global DNA hypermethylation and a specific hypermethylation signature. Furthermore, expression of 2HG-producing IDH alleles in cells induced global DNA hypermethylation. In the AML cohort, IDH1/2 mutations were mutually exclusive with mutations in the alpha-ketoglutarate-dependent enzyme TET2, and TET2 loss-of-function mutations were associated with similar epigenetic defects as IDH1/2 mutants. Consistent with these genetic and epigenetic data, expression of IDH mutants impaired TET2 catalytic function in cells. Finally, either expression of mutant IDH1/2 or Tet2 depletion impaired hematopoietic differentiation and increased stem/progenitor cell marker expression, suggesting a shared proleukemogenic effect.", 
"Interleukin-10 (IL-10), a cytokine with anti-inflammatory properties, has a central role in infection by limiting the immune response to pathogens and thereby preventing damage to the host. Recently, an increasing interest in how IL10 expression is regulated in different immune cells has revealed some of the molecular mechanisms involved at the levels of signal transduction, epigenetics, transcription factor binding and gene activation. Understanding the specific molecular events that regulate the production of IL-10 will help to answer the remaining questions that are important for the design of new strategies of immune intervention.", 
"Epigenetic mechanisms are essential for normal development and maintenance of tissue-specific gene expression patterns in mammals. Disruption of epigenetic processes can lead to altered gene function and malignant cellular transformation. Global changes in the epigenetic landscape are a hallmark of cancer. The initiation and progression of cancer, traditionally seen as a genetic disease, is now realized to involve epigenetic abnormalities along with genetic alterations. Recent advancements in the rapidly evolving field of cancer epigenetics have shown extensive reprogramming of every component of the epigenetic machinery in cancer including DNA methylation, histone modifications, nucleosome positioning and non-coding RNAs, specifically microRNA expression. The reversible nature of epigenetic aberrations has led to the emergence of the promising field of epigenetic therapy, which is already making progress with the recent FDA approval of three epigenetic drugs for cancer treatment. In this review, we discuss the current understanding of alterations in the epigenetic landscape that occur in cancer compared with normal cells, the roles of these changes in cancer initiation and progression, including the cancer stem cell model, and the potential use of this knowledge in designing more effective treatment strategies.", 
"Background Cancers arise from multiple acquired mutations, which presumably occur over many years. Early stages in cancer development might be present years before cancers become clinically apparent. Methods We analyzed data from whole-exome sequencing of DNA in peripheral-blood cells from 12,380 persons, unselected for cancer or hematologic phenotypes. We identified somatic mutations on the basis of unusual allelic fractions. We used data from Swedish national patient registers to follow health outcomes for 2 to 7 years after DNA sampling. Results Clonal hematopoiesis with somatic mutations was observed in 10% of persons older than 65 years of age but in only 1% of those younger than 50 years of age. Detectable clonal expansions most frequently involved somatic mutations in three genes (DNMT3A, ASXL1, and TET2) that have previously been implicated in hematologic cancers. Clonal hematopoiesis was a strong risk factor for subsequent hematologic cancer (hazard ratio, 12.9; 95% confidence interval, 5.8 to 28.7). Approximately 42% of hematologic cancers in this cohort arose in persons who had clonality at the time of DNA sampling, more than 6 months before a first diagnosis of cancer. Analysis of bone marrow-biopsy specimens obtained from two patients at the time of diagnosis of acute myeloid leukemia revealed that their cancers arose from the earlier clones. Conclusions Clonal hematopoiesis with somatic mutations is readily detected by means of DNA sequencing, is increasingly common as people age, and is associated with increased risks of hematologic cancer and death. A subset of the genes that are mutated in patients with myeloid cancers is frequently mutated in apparently healthy persons; these mutations may represent characteristic early events in the development of hematologic cancers.", 
"Type I interferons (IFNs) activate intracellular antimicrobial programmes and influence the development of innate and adaptive immune responses. Canonical type I IFN signalling activates the Janus kinase (JAK)-signal transducer and activator of transcription (STAT) pathway, leading to transcription of IFN-stimulated genes (ISGs). Host, pathogen and environmental factors regulate the responses of cells to this signalling pathway and thus calibrate host defences while limiting tissue damage and preventing autoimmunity. Here, we summarize the signalling and epigenetic mechanisms that regulate type I IFN-induced STAT activation and ISG transcription and translation. These regulatory mechanisms determine the biological outcomes of type I IFN responses and whether pathogens are cleared effectively or chronic infection or autoimmune disease ensues.", 
"Populations of tumour cells display remarkable variability in almost every discernable phenotypic trait, including clinically important phenotypes such as ability to seed metastases and to survive therapy. This phenotypic diversity results from the integration of both genetic and non-genetic influences. Recent technological advances have improved the molecular understanding of cancers and the identification of targets for therapeutic interventions. However, it has become exceedingly apparent that the utility of profiles based on the analysis of tumours en masse is limited by intra-tumour genetic and epigenetic heterogeneity, as characteristics of the most abundant cell type might not necessarily predict the properties of mixed populations. In this Review, we discuss both genetic and non-genetic causes of phenotypic heterogeneity of tumour cells, with an emphasis on heritable phenotypes that serve as a substrate for clonal selection. We discuss the implications of intra-tumour heterogeneity in diagnostics and the development of therapeutic resistance.", 
"The Janus kinases (JAKs) and signal transducer and activator of transcription (STAT) proteins, particularly STAT3, are among the most promising new targets for cancer therapy. In addition to interleukin-6 (IL-6) and its family members, multiple pathways, including G-protein-coupled receptors (GPCRs), Toll-like receptors (TLRs) and microRNAs were recently identified to regulate JAK-STAT signalling in cancer. Well known for its role in tumour cell proliferation, survival, invasion and immunosuppression, JAK-STAT3 signalling also promotes cancer through inflammation, obesity, stem cells and the pre-metastatic niche. In addition to its established role as a transcription factor in cancer, STAT3 regulates mitochondrion functions, as well as gene expression through epigenetic mechanisms. Newly identified regulators and functions of JAK-STAT3 in tumours are important targets for potential therapeutic strategies in the treatment of cancer.", 
"Idiopathic pulmonary fibrosis is a devastating, age-related lung disease of unknown cause that has few treatment options. This disease was once thought to be a chronic inflammatory process, but current evidence indicates that the fibrotic response is driven by abnormally activated alveolar epithelial cells (AECs). These cells produce mediators that induce the formation of fibroblast and myofibroblast foci through the proliferation of resident mesenchymal cells, attraction of circulating fibrocytes, and stimulation of the epithelial to mesenchymal transition. The fibroblast and myofibroblast foci secrete excessive amounts of extracellular matrix, mainly collagens, resulting in scarring and destruction of the lung architecture. The mechanisms that link idiopathic pulmonary fibrosis with ageing and aberrant epithelial activation are unknown; evidence suggests that the abnormal recapitulation of developmental pathways and epigenetic changes have a role. In this Seminar, we review recent data on the clinical course, therapeutic options, and underlying mechanisms thought to be involved in the pathogenesis of idiopathic pulmonary fibrosis.", 
"Glioblastoma (GBM) is a brain tumor that carries a dismal prognosis and displays considerable heterogeneity. We have recently identified recurrent H3F3A mutations affecting two critical amino acids (K27 and G34) of histone H3.3 in one-third of pediatric GBM. Here, we show that each H3F3A mutation defines an epigenetic subgroup of GBM with a distinct global methylation pattern, and that they are mutually exclusive with IDH1 mutations, which characterize a third mutation-defined subgroup. Three further epigenetic subgroups were enriched for hallmark genetic events of adult GBM and/or established transcriptomic signatures. We also demonstrate that the two H3F3A mutations give rise to GBMs in separate anatomic compartments, with differential regulation of transcription factors OLIG1, OLIG2, and FOXG1, possibly reflecting different cellular origins.", 
"Epigenetic regulation of gene expression is a dynamic and reversible process that establishes normal cellular phenotypes but also contributes to human diseases. At the molecular level, epigenetic regulation involves hierarchical covalent modification of DNA and the proteins that package DNA, such as histones. Here, we review the key protein families that mediate epigenetic signalling through the acetylation and methylation of histones, including histone deacetylases, protein methyltransferases, lysine demethylases, bromodomain-containing proteins and proteins that bind to methylated histones. These protein families are emerging as druggable classes of enzymes and druggable classes of protein-protein interaction domains. In this article, we discuss the known links with disease, basic molecular mechanisms of action and recent progress in the pharmacological modulation of each class of proteins.", 
"Cancers have the ability to develop resistance to traditional therapies, and the increasing prevalence of these drug resistant cancers necessitates further research and treatment development. This paper outlines the current knowledge of mechanisms that promote or enable drug resistance, such as drug inactivation, drug target alteration, drug efflux, DNA damage repair, cell death inhibition, and the epithelial-mesenchymal transition, as well as how inherent tumor cell heterogeneity plays a role in drug resistance. It also describes the epigenetic modifications that can induce drug resistance and considers how such epigenetic factors may contribute to the development of cancer progenitor cells, which are not killed by conventional cancer therapies. Lastly, this review concludes with a discussion on the best treatment options for existing drug resistant cancers, ways to prevent the formation of drug resistant cancers and cancer progenitor cells, and future directions of study.", 
"Cancer is associated with mutated genes, and analysis of tumour-linked genetic alterations is increasingly used for diagnostic, prognostic and treatment purposes. The genetic profile of solid tumours is currently obtained from surgical or biopsy specimens; however, the latter procedure cannot always be performed routinely owing to its invasive nature. Information acquired from a single biopsy provides a spatially and temporally limited snap-shot of a tumour and might fail to reflect its heterogeneity. Tumour cells release circulating free DNA (cfDNA) into the blood, but the majority of circulating DNA is often not of cancerous origin, and detection of cancer-associated alleles in the blood has long been impossible to achieve. Technological advances have overcome these restrictions, making it possible to identify both genetic and epigenetic aberrations. A liquid biopsy, or blood sample, can provide the genetic landscape of all cancerous lesions (primary and metastases) as well as offering the opportunity to systematically track genomic evolution. This Review will explore how tumour-associated mutations detectable in the blood can be used in the clinic after diagnosis, including the assessment of prognosis, early detection of disease recurrence, and as surrogates for traditional biopsies with the purpose of predicting response to treatments and the development of acquired resistance.", 
"All mammalian cells express 3 closely related Ras proteins, termed H-Ras, K-Ras, and N-Ras, that promote oncogenesis when they are mutationally activated at codon 12, 13, or 61. Although there is a high degree of similarity among the isoforms, K-Ras mutations are far more frequently observed in cancer, and each isoform displays preferential coupling to particular cancer types. We examined the mutational spectra of Ras isoforms curated from large-scale tumor profiling and found that each isoform exhibits surprisingly distinctive codon mutation and amino-acid substitution biases. These findings were unexpected given that these mutations occur in regions that share 100% amino-acid sequence identity among the 3 isoforms. Of importance, many of these mutational biases were not due to differences in exposure to mutagens, because the patterns were still evident when compared within specific cancer types. We discuss potential genetic and epigenetic mechanisms, as well as isoform-specific differences in protein structure and signaling, that may promote these distinct mutation patterns and differential coupling to specific cancers. Cancer Res; 72(10); 2457-67. (C) 2012 AACR.", 
"Epigenetic aberrations, which are recognized as key drivers of several human diseases, are often caused by genetic defects that result in functional deregulation of epigenetic proteins, their altered expression and/or their atypical recruitment to certain gene promoters. Importantly, epigenetic changes are reversible, and epigenetic enzymes and regulatory proteins can be targeted using small molecules. This Review discusses the role of altered expression and/or function of one class of epigenetic regulators histone deacetylases (HDACs) - and their role in cancer, neurological diseases and immune disorders. We highlight the development of small-molecule HDAC inhibitors and their use in the laboratory, in preclinical models and in the clinic.", 
"In the mammalian genome, DNA methylation is an epigenetic mechanism involving the transfer of a methyl group onto the C5 position of the cytosine to form 5-methylcytosine. DNA methylation regulates gene expression by recruiting proteins involved in gene repression or by inhibiting the binding of transcription factor(s) to DNA. During development, the pattern of DNA methylation in the genome changes as a result of a dynamic process involving both de novo DNA methylation and demethylation. As a consequence, differentiated cells develop a stable and unique DNA methylation pattern that regulates tissue-specific gene transcription. In this chapter, we will review the process of DNA methylation and demethylation in the nervous system. We will describe the DNA (de) methylation machinery and its association with other epigenetic mechanisms such as histone modifications and noncoding RNAs. Intriguingly, postmitotic neurons still express DNA methyltransferases and components involved in DNA demethylation. Moreover, neuronal activity can modulate their pattern of DNA methylation in response to physiological and environmental stimuli. The precise regulation of DNA methylation is essential for normal cognitive function. Indeed, when DNA methylation is altered as a result of developmental mutations or environmental risk factors, such as drug exposure and neural injury, mental impairment is a common side effect. The investigation into DNA methylation continues to show a rich and complex picture about epigenetic gene regulation in the central nervous system and provides possible therapeutic targets for the treatment of neuropsychiatric disorders. Neuropsychopharmacology Reviews (2013) 38, 23-38; doi:10.1038/npp.2012.112; published online 11 July 2012", 
"Genetic alterations and epigenetic dysregulation in cancer cells create a vast array of neoepitooes potentially recognizable by the immune system. Immune checkpoint blockade has the capacity to enhance and sustain endogenous immunity against non-mutated tumor-associated antigens as well as uniquely mutant antigens, establishing durable tumor control. Recent evidence from preclinical models highlights the pivotal role of the Programmed Death-1 (PD-1) T cell co-receptor and its ligands, B7-H1/PD-L1 and B7-DC/PD-L2, in maintaining an immunosuppressive tumor microenvironment. Encouraging early clinical results using blocking agents against components of the PD-1 pathway have validated its importance as a target for cancer immunotherapy.", 
"Epigenetic enzymes are often dysregulated in human tumors through mutation, altered expression, or inappropriate recruitment to certain loci. The identification of these enzymes and their partner proteins has driven the rapid development of small-molecule inhibitors that target the cancer epigenome. Herein, we discuss the influence of aberrantly regulated histone deacetylases (HDACs) in tumorigenesis. We examine HDAC inhibitors (HDACis) targeting class I, II, and IV HDACs that are currently under development for use as anticancer agents following the FDA approval of two HDACis, vorinostat and romidepsin.", 
"Epigenetics is one of the most promising and expanding fields in the current biomedical research landscape. Since the inception of epigenetics in the 1940s, the discoveries regarding its implications in normal and disease biology have not stopped, compiling a vast amount of knowledge in the past decade. The field has moved from just one recognized marker, DNA methylation, to a variety of others, including a wide spectrum of histone modifications. From the methodological standpoint, the successful initial single gene candidate approaches have been complemented by the current comprehensive epigenomic approaches that allow the interrogation of genomes to search for translational applications in an unbiased manner. Most important, the discovery of mutations in the epigenetic machinery and the approval of the first epigenetic drugs for the treatment of subtypes of leukemias and lymphomas has been an eye-opener for many biomedical scientists and clinicians. Herein, we will summarize the progress in the field of cancer epigenetics research that has reached mainstream oncology in the development of new biomarkers of the disease and new pharmacological strategies.", 
"With rare exceptions, spontaneous tumors originate from a single cell. Yet, at the time of clinical diagnosis, the majority of human tumors display startling heterogeneity in many morphological and physiological features, such as expression of cell surface receptors, proliferative and angiogenic potential. To a substantial extent, this heterogeneity might be attributed to morphological and epigenetic plasticity, but there is also strong evidence for the co-existence of genetically divergent tumor cell clones within tumors. In this perspective, we summarize the sources of intra-tumor phenotypic heterogeneity with emphasis on genetic heterogeneity. We review experimental evidence for the existence of both intra-tumor clonal heterogeneity as well as frequent evolutionary divergence between primary tumors and metastatic outgrowths. Furthermore, we discuss potential biological and clinical implications of intra-tumor clonal heterogeneity. (C) 2009 Elsevier B.V. All rights reserved.", 
"Thyroid cancer is a common endocrine malignancy. There has been exciting progress in understanding its molecular pathogenesis in recent years, as best exemplified by the elucidation of the fundamental role of several major signalling pathways and related molecular derangements. Central to these mechanisms are the genetic and epigenetic alterations in these pathways, such as mutation, gene copy-number gain and aberrant gene methylation. Many of these molecular alterations represent novel diagnostic and prognostic molecular markers and therapeutic targets for thyroid cancer, which provide unprecedented opportunities for further research and clinical development of novel treatment strategies for this cancer.", 
"During the course of malignant cancer progression, neoplastic cells undergo dynamic and reversible transitions between multiple phenotypic states, the extremes of which are defined by the expression of epithelial and mesenchymal phenotypes. This plasticity is enabled by underlying shifts in epigenetic regulation. A small cohort of pleiotropically acting transcription factors is widely recognized to effect these shifts by controlling the expression of a constituency of key target genes. These master regulators depend on complex epigenetic regulatory mechanisms, notably the induction of changes in the modifications of chromatin-associated histones, in order to achieve the widespread changes in gene expression observed during epithelial-mesenchymal transitions (EMTs). These associations indicate that an understanding of the functional interactions between such EMT-inducing transcription factors and the modulators of chromatin configuration will provide crucial insights into the fundamental mechanisms underlying cancer progression and may, in the longer term, generate new diagnostic and therapeutic modalities for treating high-grade malignancies.", 
"One-carbon metabolism involving the folate and methionine cycles integrates nutritional status from amino acids, glucose and vitamins, and generates diverse outputs, such as the biosynthesis of lipids, nucleotides and proteins, the maintenance of redox status and the substrates for methylation reactions. Long considered a 'housekeeping' process, this pathway has recently been shown to have additional complexity. Genetic and functional evidence suggests that hyperactivation of this pathway is a driver of oncogenesis and establishes a link to cellular epigenetic status. Given the wealth of clinically available agents that target one-carbon metabolism, these new findings could present opportunities for translation into precision cancer medicine.", 
"Lysine acetylation is a key mechanism that regulates chromatin structure; aberrant acetylation levels have been linked to the development of several diseases. Acetyl-lysine modifications create docking sites for bromodonnains, which are small interaction modules found on diverse proteins, some of which have a key role in the acetylation-dependent assembly of transcriptional regulator complexes. These complexes can then initiate transcriptional programmes that result in phenotypic changes. The recent discovery of potent and highly specific inhibitors for the BET (bromodonnain and extra-terminal) family of bromodomains has stimulated intensive research activity in diverse therapeutic areas, particularly in oncology, where BET proteins regulate the expression of key oncogenes and anti-apoptotic proteins. In addition, targeting BET bromodonnains could hold potential for the treatment of inflammation and viral infection. Here, we highlight recent progress in the development of bromodomain inhibitors, and their potential applications in drug discovery.", 
"Post-translational modification of histones provides an important regulatory platform for processes such as gene transcription and DNA damage repair. It has become increasingly apparent that the misregulation of histone modification, which is caused by the deregulation of factors that mediate the modification installation, removal and/or interpretation, actively contributes to human cancer. In this Review, we summarize recent advances in understanding the interpretation of certain histone methylations by plant homeodomain finger-containing proteins, and how misreading, miswriting and mis-erasing of histone methylation marks can be associated with oncogenesis and progression. These observations provide us with a greater mechanistic understanding of epigenetic alterations in human cancers and might also help direct new therapeutic interventions in the future.", 
"DNA methylation is one of the most intensely studied epigenetic modifications in mammals. In normal cells, it assures the proper regulation of gene expression and stable gene silencing. DNA methylation is associated with histone modifications and the interplay of these epigenetic modifications is crucial to regulate the functioning of the genome by changing chromatin architecture. The covalent addition of a methyl group occurs generally in cytosine within CpG dinucleotides which are concentrated in large clusters called CpG islands. DNA methyltransferases are responsible for establishing and maintenance of methylation pattern. It is commonly known that inactivation of certain tumor-suppressor genes occurs as a consequence of hypermethylation within the promoter regions and a numerous studies have demonstrated a broad range of genes silenced by DNA methylation in different cancer types. On the other hand, global hypomethylation, inducing genomic instability, also contributes to cell transformation. Apart from DNA methylation alterations in promoter regions and repetitive DNA sequences, this phenomenon is associated also with regulation of expression of noncoding RNAs such as microRNAs that may play role in tumor suppression. DNA methylation seems to be promising in putative translational use in patients and hypermethylated promoters may serve as biomarkers. Moreover, unlike genetic alterations, DNA methylation is reversible what makes it extremely interesting for therapy approaches. The importance of DNA methylation alterations in tumorigenesis encourages us to decode the human epigenome. Different DNA methylome mapping techniques are indispensable to realize this project in the future. (C) 2010, Elsevier Inc.", 
"Epigenetic and genetic alterations have long been thought of as two separate mechanisms participating in carcinogenesis. A recent outcome of whole exome sequencing of thousands of human cancers has been the unexpected discovery of many inactivating mutations in genes that control the epigenome. These mutations have the potential to disrupt DNA methylation patterns, histone modifications, and nucleosome positioning and hence, gene expression. Genetic alteration of the epigenome therefore contributes to cancer just as epigenetic process can cause point mutations and disable DNA repair functions. This crosstalk between the genome and the epigenome offers new possibilities for therapy.", 
"The epithelial-mesenchymal transition (EMT) confers mesenchymal properties on epithelial cells and has been closely associated with the acquisition of aggressive traits by carcinoma cells. EMT programs are orchestrated by a set of pleiotropically acting transcription factors (TFs). The actions of these EMT-TFs enable the early steps of metastasis: local invasion and subsequent dissemination of carcinoma cells to distant sites. However, in most malignancies, the subsequent outgrowth of micrometastatic deposits into macroscopic metastases has the greatest impact on clinical progression. Such metastatic colonization reflects the ability of disseminated tumor cells to adapt to a foreign tissue microenvironment. The outgrowth of a metastasis is also thought to be associated with self-renewal, the defining cellular trait of cancer stem cells (CSCs), also termed tumor-initiating cells. Importantly, molecular links between EMT-TFs and self-renewal have emerged, suggesting that EMT programs play critical roles both early and late in the metastatic cascade. The genetic and epigenetic mechanisms that regulate the activation of EMT-TFs and the traits they induce are areas under intensive investigation. Such studies may provide new opportunities for therapeutic intervention and help to overcome tumor heterogeneity and therapeutic resistance. (C) 2012 Elsevier Ltd. All rights reserved.", 
"The transcription factor Nrf2 (NF-E2-related factor 2) plays a vital role in maintaining cellular homeostasis, especially upon the exposure of cells to chemical or oxidative stress, through its ability to regulate the basal and inducible expression of a multitude of antioxidant proteins, detoxification enzymes and xenobiotic transporters. In addition, Nrf2 contributes to diverse cellular functions including differentiation, proliferation, inflammation and lipid synthesis and there is an increasing association of aberrant expression and/or function of Nrf2 with pathologies including cancer, neurodegeneration and cardiovascular disease. The activity of Nrf2 is primarily regulated via its interaction with Keap1 (Kelch-like ECH-associated protein 1), which directs the transcription factor for proteasomal degradation. Although it is generally accepted that modification (e.g. chemical adduction, oxidation, nitrosylation or glutathionylation) of one or more critical cysteine residues in Keap1 represents a likely chemico-biological trigger for the activation of Nrf2, unequivocal evidence for such a phenomenon remains elusive. An increasing body of literature has revealed alternative mechanisms of Nrf2 regulation, including phosphorylation of Nrf2 by various protein kinases (PKC, PI3K/Akt, GSK-3 beta, JNK), interaction with other protein partners (p21, caveolin-1) and epigenetic factors (micro-RNAs -144, -28 and -200a, and promoter methylation). These and other processes are potentially important determinants of Nrf2 activity, and therefore may contribute to the maintenance of cellular homeostasis. Here, we dissect evidence supporting these Keap1-dependent and -independent mechanisms of Nrf2 regulation. Furthermore, we highlight key knowledge gaps in this important field of biology, and suggest how these may be addressed experimentally. (C) 2012 Elsevier Inc. All rights reserved.", 
"It is now generally accepted that vitamin D deficiency is a worldwide health problem that affects not only musculoskeletal health but also a wide range of acute and chronic diseases. However, there remains cynicism about the lack of randomized controlled trials to support the association studies regarding the nonskeletal health benefits of vitamin D. This review was obtained by searching English-language studies published up to April 1, 2013, in PubMed, MEDLINE, and the Cochrane Central Register of Controlled Trials (search terms: vitamin D and supplementation) and focuses on recent challenges regarding the definition of vitamin D deficiency and how to achieve optimal serum 25-hydroxyvitamin D concentrations from dietary sources, supplements, and sun exposure. The effect of vitamin D on fetal programming epigenetics and gene regulation could potentially explain why vitamin D has been reported to have such wide-ranging health benefits throughout life. There is potentially a great upside to increasing the vitamin D status of children and adults worldwide for improving musculoskeletal health and reducing the risk of chronic illnesses, including some cancers, autoimmune diseases, infectious diseases, type 2 diabetes mellitus, neurocognitive disorders, and mortality. (c) 2013 Mayo Foundation for Medical Education and Research", 
"Hematopoietic cells, including lymphoid and myeloid cells, can develop into phenotypically distinct 'subpopulations' with different functions. However, evidence indicates that some of these subpopulations can manifest substantial plasticity (that is, undergo changes in their phenotype and function). Here we focus on the occurrence of phenotypically distinct subpopulations in three lineages of myeloid cells with important roles in innate and acquired immunity: macrophages, mast cells and neutrophils. Cytokine signals, epigenetic modifications and other microenvironmental factors can substantially and, in some cases, rapidly and reversibly alter the phenotype of these cells and influence their function. This suggests that regulation of the phenotype and function of differentiated hematopoietic cells by microenvironmental factors, including those generated during immune responses, represents a common mechanism for modulating innate or adaptive immunity.", 
"Cancer is a disease of aberrant gene expression. While the genetic causes of cancer have been intensively studied, it is becoming evident that a large proportion of cancer susceptibility cannot be attributed to variation in protein-coding sequences. This is highlighted by genome-wide association studies in cancer that reveal that more than 80% of cancer-associated SNPs occur in noncoding regions of the genome. In this review, we posit that a significant fraction of the genetic aetiology of cancer is exacted by noncoding regulatory sequences, particularly by long noncoding RNAs (lncRNAs). Recent studies indicate that several cancer risk loci are transcribed into lncRNAs and these transcripts play key roles in tumorigenesis. We discuss the epigenetic and other mechanisms through which lncRNAs function and how they contribute to each stage of cancer progression, understanding of which will be crucial for realising new opportunities in cancer diagnosis and treatment. Long noncoding RNAs play important roles in almost every aspect of cell biology from nuclear organisation and epigenetic regulation to post-transcriptional regulation and splicing, and we link these processes to the hallmarks and genetics of cancer. Finally, we highlight recent progress and future potential in the application of lncRNAs as therapeutic targets and diagnostic markers.", 
"We hypothesized that DNA methylation distributes into specific patterns in cancer cells, which reflect critical biological differences. We therefore examined the methylation profiles of 344 patients with acute myeloid leukemia (AML). Clustering of these patients by methylation data segregated patients into 16 groups. Five of these groups defined new AML subtypes that shared no other known feature. In addition, DNA methylation profiles segregated patients with CEBPA aberrations from other subtypes of leukemia, defined four epigenetically distinct forms of AML with NPM1 mutations, and showed that established AML1-ETO, CBFb-MYH11, and PML-RARA leukemia entities are associated with specific methylation profiles. We report a 15 gene methylation classifier predictive of overall survival in an independent patient cohort (p < 0.001, adjusted for known covariates).", 
"The histone 3 lysine 79 (H3K79) methyltransferase Dot1l has been implicated in the development of leukemias bearing translocations of the Mixed Lineage Leukemia (MLL) gene. We identified the MLL-fusion targets in an MLL-AF9 leukemia model, and conducted epigenetic profiling for H3K79me2, H3K4me3, H3K27me3, and H3K36me3 in hematopoietic progenitor and leukemia stem cells (LSCs). We found abnormal profiles only for H3K79me2 on MLL-AF9 fusion target loci in LSCs. Inactivation of Dot1l led to downregulation of direct MLL-AF9 targets and an MLL translocation-associated gene expression signature, whereas global gene expression remained largely unaffected. Suppression of MLL translocation-associated gene expression corresponded with dependence of MLL-AF9 leukemia on Dot1l in vivo. These data point to DOT1L as a potential therapeutic target in MLL-rearranged leukemia.", 
"The unique abilities of human pluripotent stem cells to self-renew and to differentiate into cells of the three germ layers make them an invaluable tool for the future of regenerative medicine. However, the same properties also make them tumorigenic, and therefore hinder their clinical application. Hence, the tumorigenicity of human embryonic stem cells (HESCs) has been extensively studied. Until recently, it was assumed that human induced pluripotent stem cells (HiPSCs) would behave like their embryonic counterparts in respect to their tumorigenicity. However, a rapidly accumulating body of evidence suggests that there are important genetic and epigenetic differences between these two cell types, which seem to influence their tumorigenicity.", 
"Type 2 diabetes is now a pandemic and shows no signs of abatement. In this Seminar we review the pathophysiology of this disorder, with particular attention to epidemiology, genetics, epigenetics, and molecular cell biology. Evidence is emerging that a substantial part of diabetes susceptibility is acquired early in life, probably owing to fetal or neonatal programming via epigenetic phenomena. Maternal and early childhood health might, therefore, be crucial to the development of effective prevention strategies. Diabetes develops because of inadequate islet beta-cell and adipose-tissue responses to chronic fuel excess, which results in so-called nutrient spillover, insulin resistance, and metabolic stress. The latter damages multiple organs. Insulin resistance, while forcing beta cells to work harder, might also have an important defensive role against nutrient-related toxic effects in tissues such as the heart. Reversal of ovemutrition, healing of the beta cells, and lessening of adipose tissue defects should be treatment priorities.", 
"Since the discovery of vitamin C, the number of its known biological functions is continually expanding. Both the names ascorbic acid and vitamin C reflect its antiscorbutic properties due to its role in the synthesis of collagen in connective tissues. Ascorbate acts as an electron-donor keeping iron in the ferrous state thereby maintaining the full activity of collagen hydroxylases; parallel reactions with a variety of dioxygenases affect the expression of a wide array of genes, for example via the HIF system, as well as via the epigenetic landscape of cells and tissues. In fact, all known physiological and biochemical functions of ascorbate are due to its action as an electron donor. The ability to donate one or two electrons makes AscH(-) an excellent reducing agent and antioxidant. Ascorbate readily undergoes pH-dependent autoxidation producing hydrogen peroxide (H2O2). In the presence of catalytic metals this oxidation is accelerated. In this review, we show that the chemical and biochemical nature of ascorbate contribute to its antioxidant as well as its prooxidant properties. Recent pharmacokinetic data indicate that intravenous (i.v.) administration of ascorbate bypasses the tight control of the gut producing highly elevated plasma levels; ascorbate at very high levels can act as prodrug to deliver a significant flux of H2O2 to tumors. This new knowledge has rekindled interest and spurred new research into the clinical potential of pharmacological ascorbate. Knowledge and understanding of the mechanisms of action of pharmacological ascorbate bring a rationale to its use to treat disease especially the use of i.v. delivery of pharmacological ascorbate as an adjuvant in the treatment of cancer. Published by Elsevier B.V.", 
"Along with the wide use of pesticides in the world, the concerns over their health impacts are rapidly growing. There is a huge body of evidence on the relation between exposure to pesticides and elevated rate of chronic diseases such as different types of cancers, diabetes, neurodegenerative disorders like Parkinson, Alzheimer, and amyotrophic lateral sclerosis (ALS), birth defects, and reproductive disorders. There is also circumstantial evidence on the association of exposure to pesticides with some other chronic diseases like respiratory problems, particularly asthma and chronic obstructive pulmonary disease (COPD), cardiovascular disease such as atherosclerosis and coronary artery disease, chronic nephropathies, autoimmune diseases like systemic lupus erythematous and rheumatoid arthritis, chronic fatigue syndrome, and aging. The common feature of chronic disorders is a disturbance in cellular homeostasis, which can be induced via pesticides' primary action like perturbation of ion channels, enzymes, receptors, etc., or can as well be mediated via pathways other than the main mechanism. In this review, we present the highlighted evidence on the association of pesticide's exposure with the incidence of chronic diseases and introduce genetic damages, epigenetic modifications, endocrine disruption, mitochondrial dysfunction, oxidative stress, endoplasmic reticulum stress and unfolded protein response (UPR), impairment of ubiquitin proteasome system, and defective autophagy as the effective mechanisms of action. (C) 2013 Elsevier Inc. All rights reserved.", 
"Phosphorylation of the Ser-139 residue of the histone variant H2AX, forming gamma H2AX, is an early cellular response to the induction of DNA double-strand breaks. Detection of this phosphorylation event has emerged as a highly specific and sensitive molecular marker for monitoring DNA damage initiation and resolution. Further, analysis of gamma H2AX foci has numerous other applications including, but not limited to, cancer and aging research. Quantitation of gamma H2AX foci has also been applied as a useful tool for the evaluation of the efficacy of various developmental drugs, particularly, radiation modifying compounds. This review focuses on the current status of gamma H2AX as a marker of DNA damage and repair in the context of ionizing radiation. Although the emphasis is on gamma-radiation-induced gH2AX foci, the effects of other genotoxic insults including exposure to ultraviolet rays, oxidative stress and chemical agents are also discussed. Leukemia (2010) 24, 679-686; doi: 10.1038/leu.2010.6; published online 4 February 2010", 
"Chromosomal rearrangements fusing the androgen-regulated gene TMPRSS2 to the oncogenic ETS transcription factor ERG occur in approximately 50% of prostate cancers, but how the fusion products regulate prostate cancer remains unclear. Using chromatin immunoprecipitation coupled with massively parallel sequencing, we found that ERG disrupts androgen receptor (AR) signaling by inhibiting AR expression, binding to and inhibiting AR activity at gene-specific loci, and inducing repressive epigenetic programs via direct activation of the H3K27 methyltransferase EZH2, a Polycomb group protein. These findings provide a working model in which TMPRSS2-ERG plays a critical role in cancer progression by disrupting lineage-specific differentiation of the prostate and potentiating the EZH2-mediated dedifferentiation program.", 
"Purpose of review TGF-beta acts as a potent driver of cancer progression through the induction of epithelial-mesenchymal transition (EMT), in which epithelial cells acquire mesenchymal phenotype, leading to enhanced motility and invasion. Recent reports highlight the fundamental roles of TGF-beta-induced EMT in multiple aspects of cancer progression. In this review, we focus on the novel insights into the roles of TGF-beta-induced EMT in cancer progression and the underlying mechanisms that enable TGF-beta to activate this epithelial plasticity response at transcription, translation, and posttranslational levels. Recent findings Smad-mediated transcription regulation is known to activate TGF-beta-induced EMT. More recently, novel mechanisms of epigenetic control, alternative splicing, miRNAs, translation control, and posttranslational modifications have been shown to play key roles in the control of EMT. In addition to initiating carcinoma cell invasion, TGF-beta-induced EMT can guide cancer cells to de-differentiate and gain cancer stem-cell-like properties. EMT also allows the generation of stromal cells that support and instruct cancer progression. Summary The differentiation plasticity of epithelial cells that mediates TGF-beta-induced EMT and reversion from mesenchymal to epithelial phenotype are increasingly seen as integral aspects of cancer progression that contribute to survival and dissemination of cancer cells. Further mechanistic insights under physiological conditions may lead to new therapeutic or prognostic strategies in cancer treatment.", 
"Metastases arise from residual disseminated tumour cells (DTCs). This can happen years after primary tumour treatment because residual tumour cells can enter dormancy and evade therapies. As the biology of minimal residual disease seems to diverge from that of proliferative lesions, understanding the underpinnings of this new cancer biology is key to prevent metastasis. Analysis of approximately 7 years of literature reveals a growing focus on tumour and normal stem cell quiescence, extracellular and stromal microenvironments, autophagy and epigenetics as mechanisms that dictate tumour cell dormancy. In this Review, we attempt to integrate this information and highlight both the weaknesses and the strengths in the field to provide a framework to understand and target this crucial step in cancer progression.", 
"Colorectal cancer (CRC) is a heterogeneous disease, including at least three major forms: hereditary, sporadic and colitis-associated CRC. A large body of evidence indicates that genetic mutations, epigenetic changes, chronic inflammation, diet and lifestyle are the risk factors for CRC. As elevated cyclooxygenase-2 (COX-2) expression was found in most CRC tissue and is associated with worse survival among CRC patients, investigators have sought to evaluate the effects of nonsteroidal anti-inflammatory drugs (NSAIDs) and selective COX-2 inhibitors (COXIBs) on CRC. The epidemiological studies, clinical trials and animal experiments indicate that NSAIDs are among the most promising chemopreventive agents for this disease. NSAIDs exert their antiinflammatory and antitumor effects primarily by reducing prostaglandin production by inhibition of COX-2 activity. In this review, we highlight breakthroughs in our understanding of the roles of COX-2 in CRC and inflammatory bowel disease. These recent data provide a rationale for re-evaluating COX-2 as both the prognostic and the predictive marker in a wide variety of malignancies and for renewing the interest in evaluating relative benefits and risk of COXIBs in appropriately selected patients for cancer prevention and treatment. Oncogene (2010) 29, 781-788; doi:10.1038/onc.2009.421; published online 30 November 2009", 
"Glioblastoma is the most frequent and malignant brain tumor. The vast majority of glioblastomas (similar to 90%) develop rapidly de novo in elderly patients, without clinical or histologic evidence of a less malignant precursor lesion (primary glioblastomas). Secondary glioblastomas progress from low-grade diffuse astrocytoma or anaplastic astrocytoma. They manifest in younger patients, have a lesser degree of necrosis, are preferentially located in the frontal lobe, and carry a significantly better prognosis. Histologically, primary and secondary glioblastomas are largely indistinguishable, but they differ in their genetic and epigenetic profiles. Decisive genetic signposts of secondary glioblastoma are IDH1 mutations, which are absent in primary glioblastomas and which are associated with a hypermethylation phenotype. IDH1 mutations are the earliest detectable genetic alteration in precursor low-grade diffuse astrocytomas and in oligodendrogliomas, indicating that these tumors are derived from neural precursor cells that differ from those of primary glioblastomas. In this review, we summarize epidemiologic, clinical, histopathologic, genetic, and expression features of primary and secondary glioblastomas and the biologic consequences of IDH1 mutations. We conclude that this genetic alteration is a definitive diagnostic molecular marker of secondary glioblastomas and more reliable and objective than clinical criteria. Despite a similar histologic appearance, primary and secondary glioblastomas are distinct tumor entities that originate from different precursor cells and may require different therapeutic approaches. Clin Cancer Res; 19(4); 764-72. (c) 2012 AACR.", 
"Myriad genetic and epigenetic alterations are required to drive normal cells toward malignant transformation. These somatic events commandeer many signaling pathways that cooperate to endow aspiring cancer cells with a full range of biological capabilities needed to grow, disseminate and ultimately kill its host. Cancer genomes are highly rearranged and are characterized by complex translocations and regional copy number alterations that target loci harboring cancer-relevant genes. Efforts to uncover the underlying mechanisms driving genome instability in cancer have revealed a prominent role for telomeres. Telomeres are nucleoprotein structures that protect the ends of eukaryotic chromosomes and are particularly vulnerable due to progressive shortening during each round of DNA replication and, thus, a lifetime of tissue renewal places the organism at risk for increasing chromosomal instability. Indeed, telomere erosion has been documented in aging tissues and hyperproliferative disease states-conditions strongly associated with increased cancer risk. Telomere dysfunction can produce the opposing pathophysiological states of degenerative aging or cancer with the specific outcome dictated by the integrity of DNA damage checkpoint responses. In most advanced cancers, telomerase is reactivated and serves to maintain telomere length and emerging data have also documented the capacity of telomerase to directly regulate cancer-promoting pathways. This review covers the role of telomeres and telomerase in the biology of normal tissue stem/progenitor cells and in the development of cancer.", 
"Conventional chemotherapeutics and targeted antineoplastic agents have been developed based on the simplistic notion that cancer constitutes a cell-autonomous genetic or epigenetic disease. However, it is becoming clear that many of the available anticancer drugs that have collectively saved millions of life-years mediate therapeutic effects by eliciting de novo or reactivating pre-existing tumor-specific immune responses. Here, we discuss the capacity of both conventional and targeted anticancer therapies to enhance the immunogenic properties of malignant cells and to stimulate immune effector cells, either directly or by subverting the immunosuppressive circuitries that preclude antitumor immune responses in cancer patients. Accumulating evidence indicates that the therapeutic efficacy of several antineoplastic agents relies on their capacity to influence the tumor-host interaction, tipping the balance toward the activation of an immune response specific for malignant cells. We surmise that the development of successful anticancer therapies will be improved and accelerated by the immunological characterization of candidate agents.", 
"Alkylating agents constitute a major class of frontline chemotherapeutic drugs that inflict cytotoxic DNA damage as their main mode of action, in addition to collateral mutagenic damage. Numerous cellular pathways, including direct DNA damage reversal, base excision repair (BER) and mismatch repair (MMR), respond to alkylation damage to defend against alkylation-induced cell death or mutation. However, maintaining a proper balance of activity both within and between these pathways is crucial for a favourable response of an organism to alkylating agents. Furthermore, the response of an individual to alkylating agents can vary considerably from tissue to tissue and from person to person, pointing to genetic and epigenetic mechanisms that modulate alkylating agent toxicity.", 
"MicroRNA (miRNA) alterations are involved in the initiation and progression of human cancer. The causes of the widespread differential expression of miRNA genes in malignant compared with normal cells can be explained by the location of these genes in cancer-associated genomic regions, by epigenetic mechanisms and by alterations in the miRNA processing machinery. MiRNA-expression profiling of human tumours has identified signatures associated with diagnosis, staging, progression, prognosis and response to treatment. In addition, profiling has been exploited to identify miRNA genes that might represent downstream targets of activated oncogenic pathways, or that target protein-coding genes involved in cancer.", 
"BACKGROUND: Epigenetic silencing of the MGMT (O(sup 6)-methylguanine-DNA methyltransferase) DNA-repair gene by promoter methylation compromises DNA repair and has been associated with longer survival in patients with glioblastoma who receive alkylating agents. METHODS: We tested the relationship between MGMT silencing in the tumor and the survival of patients who were enrolled in a randomized trial comparing radiotherapy alone with radiotherapy combined with concomitant and adjuvant treatment with temozolomide. The methylation status of the MGMT promoter was determined by methylation-specific polymerase-chain-reaction analysis. RESULTS: The MGMT promoter was methylated in 45 percent of 206 assessable cases. Irrespective of treatment, MGMT promoter methylation was an independent favorable prognostic factor (P<0.001 by the log-rank test; hazard ratio, 0.45; 95 percent confidence interval, 0.32 to 0.61). Among patients whose tumor contained a methylated MGMT promoter, a survival benefit was observed in patients treated with temozolomide and radiotherapy; their median survival was 21.7 months (95 percent confidence interval, 17.4 to 30.4), as compared with 15.3 months (95 percent confidence interval, 13.0 to 20.9) among those who were assigned to only radiotherapy (P=0.007 by the log-rank test). In the absence of methylation of the MGMT promoter, there was a smaller and statistically insignificant difference in survival between the treatment groups. CONCLUSIONS: Patients with glioblastoma containing a methylated MGMT promoter benefited from temozolomide, whereas those who did not have a methylated MGMT promoter did not have such a benefit.", 
" ", 
"Transitions between epithelial and mesenchymal states have crucial roles in embryonic development. Emerging data suggest a role for these processes in regulating cellular plasticity in normal adult tissues and in tumours, where they can generate multiple, distinct cellular subpopulations contributing to intratumoural heterogeneity. Some of these subpopulations may exhibit more differentiated features, whereas others have characteristics of stem cells. Owing to the importance of these tumour-associated phenotypes in metastasis and cancer-related mortality, targeting the products of such cellular plasticity is an attractive but challenging approach that is likely to lead to improved clinical management of cancer patients.", 
"Histone deacetylases (HDACs) are enzymes involved in the remodelling of chromatin, and have a key role in the epigenetic regulation of gene expression. In addition, the activity of non-histone proteins can be regulated through HDAC-mediated hypoacetylation. In recent years, inhibition of HDACs has emerged as a potential strategy to reverse aberrant epigenetic changes associated with cancer, and several classes of HDAC inhibitors have been found to have potent and specific anticancer activities in preclinical studies. However, such studies have also indicated that the effects of HDAC inhibitors could be considerably broader and more complicated than originally understood. Here we summarize recent advances in the understanding of the molecular events that underlie the anticancer effects of HDAC inhibitors, and discuss how such information could be used in optimizing the development and application of these agents in the clinic, either as monotherapies or in combination with other anticancer drugs.", 
" ", 
"Histone deacetylases ( HDACs) are considered to be among the most promising targets in drug development for cancer therapy, and first- generation histone deacetylase inhibitors ( HDACi) are currently being tested in phase I/ II clinical trials. A wide- ranging knowledge of the role of HDACs in tumorigenesis, and of the action of HDACi, has been achieved. However, several basic aspects are not yet fully understood. Investigating these aspects in the context of what we now understand about HDACi action both in vitro and in vivo will further improve the design of optimized clinical protocols.", 
"MicroRNAs (miRNAs) are small, noncoding RNAs with important functions in development, cell differentiation, and regulation Of Cell cycle and apoptosis. MiRNA expression is deregulated in cancer by a variety of mechanisms including amplification, deletion, mutation, and epigenetic silencing. Several studies have now shown that miRNAs arc involved in the initiation and progression of cancer. In this review, we briefly describe miRNA biogenesis and discuss how miRNAs can act as oncogenes and tumor Suppressors. We also address the role of miRNAs in the diagnosis, prognosis, and treatment of cancer.", 
"Resistance of human tumors to anticancer drugs is most often ascribed to gene mutations, gene amplification, or epigenetic changes that influence the uptake, metabolism, or export of drugs from single cells. Another important yet little-appreciated cause of anticancer drug resistance is the limited ability of drugs to penetrate tumor tissue and to reach all of the tumor cells in a potentially lethal concentration. To reach all viable cells in the tumor, anticancer drugs must be delivered efficiently through the tumor vasculature, cross the vessel wall, and traverse the tumor tissue. In addition, heterogeneity within the tumor microenvironment leads to marked gradients in the rate of cell proliferation and to regions of hypoxia and acidity, all of which can influence the sensitivity of the tumor cells to drug treatment. In this review, we describe how the tumor microenvironment may be involved in the resistance of solid tumors to chemotherapy and discuss potential strategies to improve the effectiveness of drug treatment by modifying factors relating to the tumor microenvironment.", 
"Chromatin alterations have been associated with all stages of tumour formation and progression. The best characterized are epigenetically mediated transcriptional-silencing events that are associated with increases in DNA methylation - particularly at promoter regions of genes that regulate important cell functions. Recent evidence indicates that epigenetic changes might 'addict' cancer cells to altered signal-transduction pathways during the early stages of tumour development. Dependence on these pathways for cell proliferation or survival allows them to acquire genetic mutations in the same pathways, providing the cell with selective advantages that promote tumour progression. Strategies to reverse epigenetic gene silencing might therefore be useful in cancer prevention and therapy.", 
"Epithelial ovarian cancer (EOC) is the sixth most common cancer in women worldwide and, despite advances in detection and therapies, it still represents the most lethal gynecologic malignancy in the industrialized countries. Unfortunately, still relatively little is known about the molecular events that lead to the development of this highly aggressive disease. The relatively recent discovery of microRNAs (miRNA), a class of small noncoding RNAs targeting multiple mRNAs and triggering translation repression and/or RNA degradation, has revealed the existence of a new level of gene expression regulation. Multiple studies involving various types of human cancers proved that miRNAs have a causal role in tumorigenesis. Here we show that, in comparison to normal ovary, miRNAs are aberrantly expressed in human ovarian cancer. The overall miRNA expression could clearly separate normal versus cancer tissues. The most significantly overexpressed miRNAs were miR-200a, miR-141, miR-200c, and miR200b, whereas miR-199a, miR-140, miR-145, and miR-125b1 were among the most down-modulated miRNAs. We could also identify miRNAs whose expression was correlated with specific ovarian cancer biopathologic features, such as histotype, lymphovascular and organ invasion, and involvement of ovarian surface. Moreover, the levels of miR-21, miR-203, and miR-205, up-modulated in ovarian carcinomas compared with normal tissues, were significantly increased after 5-aza-2 '-deoxycytidine demethylating treatment of OVCAR3 cells, suggesting that the DNA hypomethylation could be the mechanism responsible for their overexpression. Our results indicate that miRNAs might play a role in the pathogenesis of human EOC and identify altered miRNA gene methylation as a possible epigenetic mechanism involved in their aberrant expression.", 
"Polycomb group (PcG) proteins are epigenetic gene silencers that are implicated in neoplastic development. Their oncogenic function might be associated with their well-established role in the maintenance of embryonic and adult stem cells. In this review, we discuss new insights into the possible mechanisms by which PcGs regulate cellular identity, and speculate how these functions might be relevant during tumorigenesis.", 
"Expression profiling of T24 cells revealed that 17 out of 313 human miRNAs were upregulated more than 3-fold by simultaneous treatment with the chromatin-modifying drugs 5-aza-2'-deoxycytidine and 4-phenylbutyric acid. One of these, miR-127, is embedded in a CpG island and is highly induced from its own promoter after treatment. miR-127 is usually expressed as part of a miRNA cluster in normal cells but not in cancer cells, suggesting that it is subject to epigenetic silencing. In addition, the proto-oncogene BCL6, a potential target of miR-127, was translationally downregulated after treatment. These results suggest that DNA demethylation and histone deacetylase inhibition can activate expression of miRNAs that may act as tumor suppressors.", 
"The initiation and progression of cancer is controlled by both genetic and epigenetic events. Unlike genetic alterations, which are almost impossible to reverse, epigenetic aberrations are potentially reversible, allowing the malignant cell population to revert to a more normal state. With the advent of numerous drugs that target specific enzymes involved in the epigenetic regulation of gene expression, the utilization of epigenetic targets is emerging as an effective and valuable approach to chemotherapy as well as chemoprevention of cancer.", 
"Interferon-gamma (IFN-,gamma) is crucial for immunity against intracellular pathogens and for tumor control. However, aberrant IFN-gamma expression has been associated with a number of autoinftammatory and autoimmune diseases. This cytokine is produced predominantly by natural killer (NK) and natural killer T (NKT) cells as part of the innate immune response, and by Th1 CD4 and CD8 cytotoxic T lymphocyte (CTL) effector T cells once antigen-specific immunity develops. Herein, we briefly review the functions of IFN-gamma, the cells that produce it, the cell extrinsic signals that induce its production and influence the differentiation of naive T cells into IFN-gamma-producing effector T cells, and the signaling pathways and transcription factors that facilitate, induce, or repress production of this cytokine. We then review and discuss recent insights regarding the molecular regulation of IFN-gamma, focusing on work that has led to the identification and characterization of distal regulatory elements and epigenetic modifications with the IFN-gamma locus (lfng) that govern its expression. The epigenetic modifications and three-dimensional structure of the lfng locus in naive CD4 T cells, and the modifications they undergo as these cells differentiate into effector T cells, suggest a model whereby the chromatin architecture of lfng is poised to facilitate either rapid opening or silencing during Th1 or Th2 differentiation, respectively.", 
"Multipotential naive CD4(+) T cells differentiate into distinct lineages including T helper 1 (Th1), Th2, Th17, and inducible T regulatory (iTreg) cells. The remarkable diversity of CD4(+) T cells begs the question whether the observed changes reflect terminal differentiation with heritable epigenetic modifications or plasticity in T cell responses. We generated genome-wide histone H3 lysine 4 (H3K4) and lysine 27 (H3K27) trimethylation maps in naive, Th1, Th2, Th17, iTreg, and natural Treg (nTreg) cells. We found that although modifications of signature-cytokine genes (Ifng, II4, and II17) partially conform to the expectation of lineage commitment, genes encoding transcription factors like Tbx21 exhibit a broad spectrum of epigenetic states, consistent with our demonstration of T-bet and interferon-gamma induction in nTreg cells. Our data suggest an epigenetic mechanism underlying the specificity and plasticity of effector and regulatory T cells and also provide a framework for understanding complexity of CD4(+) T helper cell differentiation.", 
"Translocations that involve the mixed lineage leukaemia (MLL) gene identify a unique group of acute leukaemias, and often predict a poor prognosis. The MLL gene encodes a DNA-binding protein that methylates histone H3 lysine 4 (H3K4), and positively regulates gene expression including multiple Hox genes. Leukaemogenic MLL translocations encode MLL fusion proteins that have lost H3K4 methyltransferase activity. A key feature of MLL fusion proteins is their ability to efficiently transform haematopoietic cells into leukaemia stem cells. The link between a chromatin modulator and leukaemia stem cells provides support for epigenetic landscapes as an important part of leukaemia and normal stem-cell development.", 
"Cyclin-dependent kinases (cdks) are critical regulators of cell cycle progression and RNA transcription. A variety of genetic and epigenetic events cause universal overactivity of the cell cycle cdks in human cancer, and their inhibition can lead to both cell cycle arrest and apoptosis. However, built-in redundancy may limit the effects of highly selective cdk inhibition. Cdk4/6 inhibition has been shown to induce potent G, arrest in vitro and tumor regression in vivo; cdk2/1 inhibition has the most potent effects during the S and G(2) phases and induces E2F transcription factor-dependent cell death. Modulation of cdk2 and cdk1 activities also affects survival checkpoint responses after exposure to DNA-damaging and microtubule-stabilizing agents. The transcriptional cdks phosphorylate the carboxy-terminal domain of RNA polymerase II, facilitating efficient transcriptional initiation and elongation. Inhibition of these cdks primarily affects the accumulation of transcripts with short half-lives, including those encoding antiapoptosis family members, cell cycle regulators, as well as p53 and nuclear factor-kappa B-responsive gene targets. These effects may account for apoptosis induced by cdk9 inhibitors, especially in malignant hematopoietic cells, and may also potentiate cytotoxicity mediated by disruption of a variety of pathways in many transformed cell types. Current work is focusing on overcoming pharmacokinetic barriers that hindered development of flavopiridol, a pan-cdk inhibitor, as well as assessing novel classes of compounds potently targeting groups of cell cycle cdks (cdk4/6 or cdk2/1) with variable effects on the transcriptional cdks 7 and 9. These efforts will establish whether the strategy of cdk inhibition is able to produce therapeutic benefit in the majority of human tumors.", 
"Mechanisms of constitutive NF-kappa B signaling in multiple myeloma are unknown. An inhibitor of I kappa B kinase beta(IKK beta) targeting the classical NF-kappa B pathway was lethal to many myeloma cell lines. Several cell lines had elevated expression of NIK due to genomic alterations or protein stabilization, while others had inactivating mutations of TRAF3; both kinds of abnormality triggered the classical and alternative NF-kappa B pathways. A majority of primary myeloma patient samples and cell lines had elevated NF-kappa B target gene expression, often associated with genetic or epigenetic alteration of NIK, TRAF3, CYLD, BIRC2/BIRC3, CD40, NFKB1, or NFKB2. These data demonstrate that addiction to the NF-kappa B pathway is frequent in myeloma and suggest that IKK beta inhibitors hold promise for the treatment of this disease.", 
"The mechanisms underlying microRNA (miRNA) disruption in human disease are poorly understood. In cancer cells, the transcriptional silencing of tumor suppressor genes by CpG island promoter hypermethylation has emerged as a common hallmark. We wondered if the same epigenetic disruption can hit miRNAs in transformed cells. To address this issue, we have used cancer cells genetically deficient for the DNA methyltransferase enzymes in combination with a miRNA expression profiling. We have observed that DNA hypomethylation induces a release of miRNA silencing in cancer cells. One of the main targets is miRNA-124a, which undergoes transcriptional inactivation by CpG island hypermethylation in human tumors from different cell types. Interestingly, we functionally link the epigenetic loss of miRNA-124a with the activation of cyclin D kinase 6, a bona fide oncogenic factor, and the phosphorylation of the retinoblastoma, a tumor suppressor gene.", 
"Cruciferous vegetables are a rich source of glucosinolates and their hydrolysis products, including indoles and isothiocyanates, and high intake of cruciferous vegetables has been associated with lower risk of lung and colorectal cancer in some epidemiological studies. Glucosinolate hydrolysis products alter the metabolism or activity of sex hormones in ways that could inhibit the development of hormone-sensitive cancers, but evidence of an inverse association between cruciferous vegetable intake and breast or prostate cancer in humans is limited and inconsistent. Organizations such as the National Cancer Institute recommend the consumption of five to nine servings of fruits and vegetables daily, but separate recommendations for cruciferous vegetables have not been established. Isothiocyanates and indoles derived from the hydrolysis of glucosinolates, such as sulforaphane and indole-3-carbinol (I3C), have been implicated in a variety of anti carcinogenic mechanisms, but deleterious effects also have been reported in some experimental protocols, including tumor promotion over prolonged periods of exposure. Epidemiological studies indicate that human exposure to isothiocyanates and indoles through cruciferous vegetable consumption may decrease cancer risk, but the protective effects may be influenced by individual genetic variation (polymorphisms) in the metabolism and elimination of isothiocyanates from the body. Cooking procedures also affect the bioavailability and intake of glucosinolates and their derivatives. Supplementation with I3C or the related dimer 3,3'-diindolylmethane (DIM) alters urinary estrogen metabolite profiles in women, but the effects of I3C and DIM on breast cancer risk are not known. Small preliminary trials in humans suggest that I3C supplementation may be beneficial in treating conditions related to human papilloma virus infection, such as cervical intraepithelial neoplasia and recurrent respiratory papillomatosis, but larger randomized controlled trials are needed. (C) 2007 Elsevier Ltd. All rights reserved.", 
"The current goal of diabetes therapy is to reduce time-averaged mean levels of glycemia, measured as HbA1c, to prevent diabetic complications. However, HbA1c only explains <25% of the variation in risk of developing complications. Because HbA1c does not correlate with glycemic variability when adjusted for mean blood glucose, we hypothesized that transient spikes of hyperglycemia may be an HbA1c-independent risk factor for diabetic complications. We show that transient hyperglycemia induces long-lasting activating epigenetic changes in the promoter of the nuclear factor kappa B (NF-kappa B) subunit p65 in aortic endothelial cells both in vitro and in nondiabetic mice, which cause increased p65 gene expression. Both the epigenetic changes and the gene expression changes persist for at least 6 d of subsequent normal glycemia, as do NF-kappa B-induced increases in monocyte chemoattractant protein 1 and vascular cell adhesion molecule 1 expression. Hyperglycemia-induced epigenetic changes and increased p65 expression are prevented by reducing mitochondrial superoxide production or superoxide-induced alpha-oxoaldehydes. These results highlight the dramatic and long-lasting effects that short-term hyperglycemic spikes can have on vascular cells and suggest that transient spikes of hyperglycemia may be an HbA1c -independent risk factor for diabetic complications.", 
"Purpose Epigenetic processes are implicated in cancer causation and progression. The acetylation status of histones regulates access of transcription factors to DNA and influences levels of gene expression. Histone deacetylase (HDAC) activity diminishes acetylation of histones, causing compaction of the DNA/histone complex. This compaction blocks gene transcription and inhibits differentiation, providing a rationale for developing HDAC inhibitors. Methods In this review, we explore the biology of the HDAC enzymes, summarize the pharmacologic properties of HDAC inhibitors, and examine results of selected clinical trials. We consider the potential of these inhibitors in combination therapy with targeted drugs and with cytotoxic chemotherapy. Results HDAC inhibitors promote growth arrest, differentiation, and apoptosis of tumor cells, with minimal effects on normal tissue. In addition to decompaction of the histone/DNA complex, HDAC inhibition also affects acetylation status and function of nonhistone proteins. HDAC inhibitors have demonstrated antitumor activity in clinical trials, and one drug of this class, vorinostat, is US Food and Drug Administration approved for the treatment of cutaneous T-cell lymphoma. Other inhibitors in advanced stages of clinical development, including depsipeptide and MGCD0103, differ from vorinostat in structure and isoenzyme specificity, and have shown activity against lymphoma, leukemia, and solid tumors. Promising preclinical activity in combination with cytotoxics, inhibitors of heat shock protein 90, and inhibitors of proteasome function have led to combination therapy trials. Conclusion HDAC inhibitors are an important emerging therapy with single-agent activity against multiple cancers, and have significant potential in combination use.", 
"Infection and chronic inflammation contribute to about 1 in 4 of all cancer cases. Mediators of the inflammatory response, e-g., cytokines, free radicals, prostaglandins and growth factors, can induce genetic and epigenetic changes including point mutations in tumor suppressor genes, DNA methylation and post-translational modifications, causing alterations in critical pathways responsible for maintaining the normal cellular homeostasis and leading to the development and progression of cancer. Recent discovery of an interaction between microRNAs and innate immunity during inflammation has further strengthened the association between inflammation and cancer. (C) 2007 Wiley-Liss, Inc.", 
"Gain-of-function mutations in NOTCH1 are common in T- cell lymphoblastic leukemias and lymphomas ( T- ALL), making this receptor a promising target for drugs such as gamma-secretase inhibitors, which block a proteolytic cleavage required for NOTCH1 activation. However, the enthusiasm for these therapies has been tempered by tumor resistance and the paucity of information on the oncogenic programs regulated by oncogenic NOTCH1. Here we show that NOTCH1 regulates the expression of PTEN ( encoding phosphatase and tensin homolog) and the activity of the phosphoinositol- 3 kinase ( PI3K)-AKT signaling pathway in normal and leukemic T cells. Notch signaling and the PI3K-AKT pathway synergize in vivo in a Drosophila melanogaster model of Notch- induced tumorigenesis, and mutational loss of PTEN is associated with human T- ALL resistance to pharmacological inhibition of NOTCH1. Overall, these findings identify transcriptional control of PTEN and regulation of the PI3K-AKT pathway as key elements of the leukemogenic program activated by NOTCH1 and provide the basis for the design of new therapeutic strategies for T- ALL.", 
"Cancer cells contain multiple genetic and epigenetic abnormalities. Despite this complexity, their growth and survival can often be impaired by the inactivation of a single oncogene. This phenomenon, called oncogene addiction, provides a rationale for molecular targeted therapy. The efficacy of this strategy requires novel methods, including integrative genomics and systems biology, to identify the state of oncogene addiction (i.e., the Achilles heel) in specific cancers. Combination therapy may also be required to prevent the escape of cancers from a given state of oncogene addiction.", 
"NFATc1 and NFATc2 are functionally redundant in the immune system, but it was suggested that NFATc1 is required exclusively for differentiation of osteoclasts in the skeletal system. Here we provide genetic evidence that NFATc1 is essential for osteoclast differentiation in vivo by adoptive transfer of NFATc1(-/-) hematopoietic stem cells to osteoclast-deficient Fos(-/-) mice, and by Fos(-/-) blastocyst complementation, thus avoiding the embryonic lethality of NFATc1(-/-) mice. However, in vitro osteoclastogenesis in NFATc1-deficient cells was rescued by ectopic expression of NFATc2. The discrepancy between the in vivo essential role of NFATc1 and the in vitro effect of NFATc2 was attributed to selective autoregulation of the NFATc1 gene by NFAT through its promoter region. This suggested that an epigenetic mechanism contributes to the essential function of NFATc1 in cell lineage commitment. Thus, this study establishes that NFATc1 represents a potential therapeutic target for bone disease and reveals a mechanism that underlies the essential role of NFATc1 in bone homeostasis.", 
"A cell undergoes many genetic and epigenetic changes as it transitions to malignancy. Malignant transformation is also accompanied by a progressive loss of tissue homeostasis and perturbations in tissue architecture that ultimately culminates in tumor cell invasion into the parenchyma and metastasis to distant organ sites. Increasingly, cancer biologists have begun to recognize that a critical component of this transformation journey involves marked alterations in the mechanical phenotype of the cell and its surrounding microenvironment. These mechanical differences include modifications in cell and tissue structure, adaptive force-induced changes in the environment, altered processing of micromechanical cues encoded in the extracellular matrix (ECM), and cell-directed remodeling of the extracellular stroma. Here, we review critical steps in this force journey, including mechanical contributions to tissue dysplasia, invasion of the ECM, and metastasis. We discuss the biophysical basis of this force journey and present recent advances in the measurement of cellular mechanical properties in vitro and in vivo. We end by describing examples of molecular mechanisms through which tumor cells sense, process and respond to mechanical forces in their environment. While our understanding of the mechanical components of tumor growth, survival and motility remains in its infancy, considerable work has already yielded valuable insight into the molecular basis of force-dependent tumor pathophysiology, which offers new directions in cancer chemotherapeutics.", 
"The balance of histone acetylation and deacetylation is an epigenetic layer with a critical role in the regulation of gene expression, Histone acetylation induced by histone acetyl transferases (HATs) is associated with gene transcription, while histone hypoacetylation induced by histone deacetylase (HDAC) activity is associated with gene silencing. Altered expression and mutations of genes that encode HDACs have been linked to tumor development since they both induce the aberrant transcription of key genes regulating important cellular functions such as cell proliferation, cell-cycle regulation and apoptosis. Thus, HDACs are among the most promising therapeutic targets for cancer treatment, and they have inspired researchers to study and develop HDAC inhibitors. (C) 2007 Federation of European Biochemical Societies. Published by Elsevier B.V. All rights reserved.", 
"Early developmental perturbations have been linked to adult-onset prostate pathology, including excessive exposure to estrogenic compounds; however, the molecular basis for this imprinting event is not known. An important and controversial health concern is whether low-dose exposures to hormonally active environmental estrogens, such as bisphenol A, can promote human diseases, including prostate cancer. Here, we show that transient developmental exposure of rats to low, environmentally relevant doses of bisphenol A or estradiol increases prostate gland susceptibility to adult-onset precancerous lesions and hormonal carcinogenesis. We found permanent alterations in the DNA methylation patterns of multiple cell signaling genes, suggesting an epigenetic basis for estrogen imprinting. For phosphodiesterase type 4 variant 4 (PDE4D4), an enzyme responsible for cyclic AMP breakdown, a specific methylation cluster was identified in the 5'-flanking CpG island that was gradually hypermethylated with aging in normal prostates, resulting in loss of gene expression. Early and prolonged hypomethylation at this site following neonatal estradiol or bisphenol A exposure resulted in continued, elevated PDE4D4 expression. Cell line studies confirmed that site-specific methylation is involved in transcriptional silencing of the PDE4D4 gene and showed hypomethylation of this gene in prostate cancer cells. Importantly, the PDE4D4 alterations in the estrogen-exposed prostates were distinguishable before histopathologic changes of the gland, making PDE4D4 a candidate molecular marker for prostate cancer risk assessment as a result of endocrine disruptors. In total, these findings indicate that low-dose exposures to ubiquitous environmental estrogens affect the prostate epigenome during development and, in so doing, promote prostate disease with aging.", 
"Resistance to alkylating agents via direct DNA repair by O-6-methylguanine methyltransferase (MGMT) remains a significant barrier to the successful treatment of patients with malignant glioma. The relative expression of MGMT in the tumor may determine response to alkylating agents, and epigenetic silencing of the MGMT gene by promoter methylation plays an important role in regulating MGMT expression in gliomas. MGMT promoter methylation is correlated with improved progression-free and overall survival in patients treated with alkylating agents. Strategies to overcome MGMT-mediated chemoresistance are being actively investigated. These include treatment with nontoxic pseudosubstrate inhibitors of MGMT, such as O6-benzylguanine, or RNA interference-mediated gene silencing of MGMT. However, systemic application of MGMT inhibitors is limited by an increase in hematologic toxicity. Another strategy is to deplete MGMT activity in tumor tissue using a dose-dense temozolomide schedule. These alternative schedules are well tolerated; however, it remains unclear whether they are more effective than the standard dosing regimen or whether they effectively deplete MGMT activity in tumor tissue. Of note, not all patients with glioblastoma having MGMT promoter methylation respond to alkylating agents, and even those who respond will inevitably experience relapse. Herein we review the data supporting MGMT as a major mechanism of chemotherapy resistance in malignant gliomas and describe ongoing studies that are testing resistance-modulating strategies.", 
"MicroRNAs (miR) show characteristic expression signatures in various cancers and can profoundly affect cancer cell behavior. We carried out miR expression profiling of human glioblastoma specimens versus adjacent brain devoid of tumor. This revealed several significant alterations, including a pronounced reduction of miR-128 in tumor samples. miR-128 expression significantly reduced glioma cell proliferation in vitro and glioma xenograft growth in vivo. miR-128 caused a striking decrease in expression of the Bmi-1 oncogene, by direct regulation of the Bmi-1 mRNA 3'-untranslated region, through a single miR-128 binding site. In a panel of patient glioblastoma specimens, Bmi-1 expression was significantly up-regulated and miR-128 was down-regulated compared with normal brain. Bmi-1 functions in epigenetic silencing of certain genes through epigenetic chromatin modification. We found that miR-128 expression caused a decrease in histone methylation (H3K27me(3)) and Akt phosphorylation, and up-regulation of p21(CIP1) levels, consistent with Bmi-1 down-regulation. Bmi-1 has also been shown to promote stem cell self-renewal; therefore, we investigated the effects of miR-128 overexpression in human glioma neurosphere cultures, possessing features of glioma stem-like cells. This showed that miR-128 specifically blocked glioma self-renewal consistent with Bmi-1 down-regulation. This is the first example of specific regulation by a miR of a neural stem cell self-renewal factor, implicating miRs that may normally regulate brain development as important biological and therapeutic targets against the stem cell-like characteristics of glioma. [Cancer Res 2008;68(22):9125-30]", 
"Compelling evidence suggests that Foxp3-expressing CD25(+)CD4(+) regulatory T cells (Treg) are generated within the thymus as a separate lineage. However, Foxp3(+)CD4(+) Treg can also be generated de novo in a TGF-beta-dependent process from naive T cells by TCR triggering. Recently, we have shown that naturally occurring, but not in vitro TGF-beta-induced Foxp3(+) Treg display stable Foxp3 expression that was associated with selective demethylation of an evolutionarily conserved element within the Foxp3 locus named TSDR (Treg-specific demethylated region). Here, we report that inhibition of DNA methylation by azacytidine, even in absence of exogenous TGF-beta, not only promoted de novo induction of Foxp3 expression during priming, but also conferred stability of Foxp3 expression upon restimulation. Most notably, such stable Foxp3 expression was found only for cells displaying enhanced TSDR demethylation. In contrast, in vitro TSDR methylation diminished its transcriptional activity. Foxp3(+) Treg generated in vivo by DEC-205-mediated targeting of agonist ligands to dendritic cells showed long-term survival in the absence of the inducing antigen and exhibited efficient TSDR demethylation. Together, our data suggest that TSDR is an important methylation-sensitive element regulating Foxp3 expression and demonstrate that epigenetic imprinting in this region is critical for establishment of a stable Treg lineage.", 
"WNT signals are transduced to the canonical pathway for cell fate determination, and to the noncanonical pathway for control of cell movement and tissue polarity. Canonical WNTsignals are transduced through Frizzled family receptors and LRP5/LRP6 coreceptor to the beta-catenin signaling cascade. Microtubule affinity-regulating kinase (PAR-1) family kinases, casein kinase le (CKl epsilon), and FRAT are positive regulators of the canonical WNT pathway, whereas APC, AXIN1, AXIN2, CKl alpha, NKD1, NKD2, beta TRCP1, beta TRCP2, ANKRD6, Nemo-like kinase (NLK), and peroxisome proliferator-activated receptor gamma (PPAR gamma) are negative regulators. Nuclear complex, consisting of T-cell factor/lymphoid enhancer factor, p-catenin, BCL9/BCL9L, and PYGO, activates transcription of canonical WNT target genes such as FGF20, DKK1, WISP1 MYC, CCND1, and Glucagon (GCG). Noncanonical WNTsignals are transduced through Frizzled family receptors and ROR2/RYK coreceptors to the Dishevelled-dependent (Rho family GTPases and c-jun NH(2)-terminal kinase) or the Ca(2+)-dependent (NLK and nuclear factor of activated Tcells) signaling cascades. WNTsignals are context-dependently transduced to both pathways based on the expression profile of WNT SFRP, WIF, DKK, Frizzled receptors, coreceptors, and the activity of intracellular WNT signaling regulators. Epigenetic silencing and loss-of-function mutation of negative regulators of the canonical WNT pathway occur in a variety of human cancer. WNT, fibroblast growth factor (FGF), Notch, Hedgehog, and transforming growth factor beta/bone morphogenetic protein signaling network are implicated in the maintenance of tissue homeostasis by regulating self-renewal of normal stem cells as well as proliferation or differentiation of progenitor (transit-amplifying) cells. Breakage of the stem cell signaling network leads to carcinogenesis. Nonsteroidal anti-inflammatory drugs and PPAR gamma agonists with the potential to inhibit the canonical WNT signaling pathway are candidate agents for chemoprevention. ZTM000990 and PKF118-310 are lead compounds targeted to the canonical WNT signaling cascade. Anti-WNT1 and anti-WNT2 monoclonal antibodies show in vitro effects in cancer treatment. After the optimization, derivatives of small-molecule compound and human monoclonal antibody targeted to the WNT signaling pathway could be used in cancer medicine.", 
"Cellular transformation and tumor development result from an accumulation Of mutational and epigenetic changes that alter normal cell growth and survival pathways. For the last 100 years, there has been a vigorous debate (is to whether the unmanipulated immune system can detect and eliminate such altered host derived cells despite the fact that cancer cells frequently express either abnormal proteins or abnormal levels of normal cellular proteins that function as tumor antigens. In this review, toe discuss the current state of this argument and point out some of the recent key experiments demonstrating that immunity not only protects the host from cancer development (i.e., provides a cancer immunosurveillance but also can promote tumor growth, sometimes by generating more aggressive tumors. The terminology cancer immunoediting has been used to describe this dual host protective and tumor promoting action Of immunity, and herein we summarize the ever-increasing experimental and clinical data that support the validity of this concept.", 
"The transcription factor FOXP3 is critical for development and function of regulatory T cells (Treg). Their number and functioning appears to be crucial in the prevention of autoinmumity and allergy, but also to be a negative prognostic marker for various solid tumors. Although expression of the transcription factor FOXP3 currently constitutes the best-known marker for Treg, in humans, transient expression is also observed in activated non-Treg. Extending our recent findings for the murine foxp3 locus, we observed epigenetic modification of several regions in the human FOXP3 locus exclusively occurring in Treg. Importantly, activated conventional CD4(+) T cells and TGF-beta-treated cells displayed no FOXP3 DNA demethylation despite expression of FOXP3, whereas subsets of Treg stable even upon extended in vitro expansion remained demethylated. To investigate whether a whole set of genes might be epigenetically imprinted in the Treg lineage, we conducted a genome-wide differential methylation hybridization analysis. Several genes were found displaying differential methylation between Treg and conventional T cells, but none beside FOXP3 turned out to be entirely specific to Treg when tested on a broad panel of cells and tissues. We conclude that FOXP3 DNA demethylation constitutes the most reliable criterion for natural Treg available at present.", 
"Naive CD4(+) T cells give rise to T-helper-cell subsets with functions that are tailored to their respective roles in host defence. The specification of T-helper-cell subsets is controlled by networks of lineage-specifying transcription factors, which bind to regulatory elements in genes that encode cytokines and other transcription factors. The nuclear context in which these transcription factors act is affected by epigenetic processes, which allow programmes of gene expression to be inherited by progeny cells that at the same time retain the potential for change in response to altered environmental signals. In this Review, we describe these epigenetic processes and discuss how they collaborate to govern the fate and function of T helper cells.", 
"Helper T cells coordinate immune responses through the production of cytokines. Th2 cells express the closely linked 114, 1113, and 115 cytokine genes, whereas these same genes are silenced in the Th1 lineage. The Th1/Th2 lineage choice has become a textbook example for the regulation of cell differentiation, and recent discoveries have further refined and expanded our understanding of how Th2 differentiation is initiated and reinforced by signals from antigen-presenting cells and cytokine-driven feedback loops. Epigenetic changes that stabilize the active or silent state of the 114 locus in differentiating helper T cells have been a major focus of recent research. Overall, the field is progressing toward an integrated model of the signaling and transcription factor networks, cis-regulatory elements, epigenetic modifications, and RNA interference mechanisms that converge to determine the lineage fate and gene expression patterns of differentiating helper T cells.", 
"There has been considerable progress in the systemic treatment of cancer because of the rapid development and clinical application of molecular targeted agents. Although patients with a particular type and stage of cancer are often treated as a single group, more-specific therapy is being considered, as subsets of these patients who are more likely to benefit from treatment with particular agents are being identified. We previously introduced the concept of 'oncogene addiction' to explain how some cancers that contain multiple genetic, epigenetic, and chromosomal abnormalities are dependent on or 'addicted' to one or a few genes for both maintenance of the malignant phenotype and cell survival. Thus, reversal of only one or a few of these abnormalities can inhibit cancer cell growth and in some cases translate to improved survival rates. This review summarizes current experimental and clinical evidence for the concept of oncogene addiction and describes molecular mechanisms that may explain this phenomenon. In addition, we discuss how high-throughput screening methods, including gene-expression profiling and proteomics, and emerging methods for analyzing complex cellular networks can be used to identify the state of oncogene addiction, i.e. the 'Achilles' heel,' in specific cancers. Finally, we discuss the use of molecular targeted agents in combination with other anticancer agents as a strategy to optimize therapy and prevent disease recurrence.", 
"Acetylation is a key posttranslational modification of many proteins responsible for regulating critical intracellular pathways. Although histones are the most thoroughly studied of acetylated protein substrates, histone acetyltransferases (HATs) and deacetylases (HDACs) are also responsible for modifying the activity of diverse types of nonhistone proteins, including transcription factors and signal transduction mediators. HDACs have emerged as uncredentialed molecular targets for the development of enzymatic inhibitors to treat human cancer, and six structurally distinct drug classes have been identified with in vivo bioavailability and intracellular capability to inhibit many of the known mammalian members representing the two general types of NAD+-independent yeast HDACs, Rpd3 (HDACs 1, 2, 3, 8) and Hda1 (HDACs 4,5, 6, 7, 9a, 9b, 10). Initial clinical trials indicate that HDAC inhibitors from several different structural classes are very well tolerated and exhibit clinical activity against a variety of human malignancies; however, the molecular basis for their anticancer selectivity remains largely unknown. HDAC inhibitors have also shown preclinical promise when combined with other therapeutic agents, and innovative drug delivery strategies, including liposome encapsulation, may further enhance their clinical development and anticancer potential. An improved understanding of the mechanistic role of specific HDACs in human tumorigenesis, as well as the identification of more specific HDAC inhibitors, will likely accelerate the clinical development and broaden the future scope and utility of HDAC inhibitors for cancer treatment.", 
"Altered expression of microRNA (miRNA) is strongly implicated in cancer, and recent studies have shown that, in cancer, expression of some miRNAs cells is silenced in association with CpG island hypermethylation. To identify epigenetically silenced miRNAs in colorectal cancer (CRC), we screened for miRNAs induced in CRC cells by 5-aza-2'-deoxycytidine (DAC) treatment or DNA methyltransferase knockout. We found that miRNA-34b (miR-34b) and miR-34c, two components of the p53 network, are epigenetically silenced in CRC; that this down-regulation of miR-34b/c is associated with hypermethylation of the neighboring CpG island; and that DAC treatment rapidly restores miR-34b/c expression. Methylation of the miR-34b/c CpG island was frequently observed in CRC cell lines (nine of nine, 100%) and in primary CRC tumors (101 of 111, 90%), but not in normal colonic mucosa. Transfection of precursor miR-34b or miR-34c into CRC cells induced dramatic changes in the gene expression profile, and there was significant overlap between the genes down-regulated by miR-34b/c and those down-regulated by DAC. We also found that the miR-34b/c CpG island is a bidirectional promoter which drives expression of both miR-34b/c and B-cell translocation gene 4 (BTG4); that methylation of the CpG island is also associated with transcriptional silencing of BTG4; and that ectopic expression of BTG4 suppresses colony formation by CRC cells. Our results suggest that miR-34b/c and BTG4 are novel tumor suppressors in CRC and that the miR-34b/c CpG island, which bidirectionally regulates miR-34b/c and BTG4, is a frequent target of epigenetic silencing in CRC.", 
"The cytosine analogues 5-azacytosine (azacytidine) and 2'-deoxy-5-azacytidine (decitabine) are the currently most advanced drugs for epigenetic cancer therapies. These compounds function as DNA methyltransferase inhibitors and have shown substantial potency in reactivating epigenctically silenced tumor suppressor genes in vitro. However, it has been difficult to define the mode of action of these drugs in patients and it appears that clinical responses are influenced both by epigenetic alterations and by apoptosis induction. To maximize the clinical efficacy of azacytidine and decitabine it will be important to understand the molecular changes induced by these drugs. In this review, we examine the pharmacological properties of azanucleosides and their interactions with various cellular pathways. Because azacytidine and decitabine are prodrugs, an understanding of the cellular mechanisms mediating transmembrane transport and metabolic activation will be critically important for optimizing patient responses. We also discuss the mechanism of DNA methyltransferase inhibition and emphasize the need for the identification of predictive biomarkers for the further advancement of epigenetic therapies. (C) 2008 Wiley-Liss, Inc.", 
"Context Changes over time in epigenetic marks, which are modifications of DNA such as by DNA methylation, may help explain the late onset of common human diseases. However, changes in methylation or other epigenetic marks over time in a given individual have not yet been investigated. Objectives To determine whether there are longitudinal changes in global DNA methylation in individuals and to evaluate whether methylation maintenance demonstrates familial clustering. Design, Setting, and Participants We measured global DNA methylation by luminometric methylation assay, a quantitative measurement of genome- wide DNA methylation, on DNA sampled at 2 visits on average 11 years apart in 111 individuals from an Icelandic cohort ( 1991 and 2002- 2005) and on average 16 years apart in 126 individuals from a Utah sample ( 1982- 1985 and 1997- 2005). Main Outcome Measure Global methylation changes over time. Results Twenty- nine percent of Icelandic individuals showed greater than 10% methylation change over time ( P <. 001). The family- based Utah sample also showed intra-individual changes over time, and further demonstrated familial clustering of methylation change ( P=. 003). The family showing the greatest global methylation loss also demonstrated the greatest loss of gene- specific methylation by a separate methylation assay. Conclusion These data indicate that methylation changes over time and suggest that methylation maintenance may be under genetic control.", 
"The ubiquitous transcription factor Yin Yang 1 (YY1) is known to have a fundamental role in normal biologic processes such as embryogenesis, differentiation, replication, and cellular proliferation. YY1 exerts its effects on genes involved in these processes via its ability to initiate, activate, or repress transcription depending upon the context in which it binds. Mechanisms of action include direct activation or repression, indirect activation or repression via cofactor recruitment, or activation or repression by disruption of binding sites or conformational DNA changes. YY1 activity is regulated by transcription factors and cytoplasmic proteins that have been shown to abrogate or completely inhibit YY1-mediated activation or repression; however, these mechanisms have not yet been fully elucidated. Since expression and function of YY1 are known to be intimately associated with progression through phases of the cell cycle, the physiologic significance of YY1 activity has recently been applied to models of tumor biology. The majority of the data are consistent with the hypothesis that YY1 overexpression and/or activation is associated with unchecked cellular proliferation, resistance to apoptotic stimuli, tumorigenesis and metastatic potential. Studies involving hemato-poetic tumors, epithelial-based tumors, endocrine organ malignancies, hepatocellular carcinoma, and retinoblastoma support this hypothesis. Molecular mechanisms that have been investigated include YY1-mediated downregulation of p53 activity, interference with poly-ADP-ribose polymerase, alteration in c-myc and nuclear factor-kappa B (NF-kappa B) expression, regulation of death genes and gene products, and differential YY1 binding in the presence of inflammatory mediators. Further, recent findings implicate YY1 in the regulation of tumor cell resistance to chemotherapeutics and immune-mediated apoptotic stimuli. Taken together, these findings provide strong support of the hypothesis that YY1, in addition to its regulatory roles in normal biologic processes, may possess the potential to act as an initiator of tumorigenesis and may thus serve as both a diagnostic and prognostic tumor marker; furthermore, it may provide an effective target for antitumor chemotherapy and/or immunotherapy.", 
"Many chemotherapy regiments are successfully used to treat breast cancer; however, often breast cancer cells develop drug resistance that usually leads to a relapse and worsening of prognosis. We have shown recently that epigenetic changes such as DNA methylation and histone modifications play an important role in breast cancer cell resistance to chemotherapeutic agents. Another mechanism of gene expression control is mediated via the function of small regulatory RNA, particularly microRNA (miRNA); its role in cancer cell drug resistance still remains unexplored. In the present study, we investigated the role of miRNA in the resistance of human MCF-7 breast adenocarcinoma cells to doxorubicin (DOX). Here, we for the first time show that DOX-resistant MCF-7 cells (MCF-7/DOX) exhibit a considerable dysregulation of the miRNAome profile and altered expression of miRNA processing enzymes Dicer and Argonaute 2. The mechanistic link of miRNAome deregulation and the multidrug-resistant phenotype of MCF-7/DOX cells was evidenced by a remarkable correlation between specific miRNA expression and corresponding changes in protein levels of their targets, specifically those ones that have a documented role in cancer drug resistance. Furthermore, we show that microRNA-451 regulates the expression of multidrug resistance 1 gene. More importantly, transfection of the MCF-7/DOX-resistant cells with microRNA-451 resulted in the increased sensitivity of cells to DOX, indicating that correction of altered expression of miRNA may have significant implications for therapeutic strategies aiming to overcome cancer cell resistance.", 
"In the last few years, microRNAs (miRNA) have started a revolution in molecular biology and emerged as key players in the carcinogenesis. They have been identified in various tumor types, showing that different sets of miRNAs are usually deregulated in different cancers. To identify the miRNA signature that was specific for oral squamous cell carcinoma (OSCC), we first examined expression profiles of 148 miRNAs in a panel of 18 OSCC cell lines and the immortalized oral keratinocyte line RT7 as a control. Compared with RT7, the expression of 54 miRNAs (36.5%) was frequently down-regulated in OSCC lines (<0.5-fold expression, >= 66.7% of 18 lines). Among these 54 miRNAs, we further analyzed four of these miRNAs (i.e., miR-34b, miR-137, miR-193a, and miR-203), located around CpG islands, to identify tumor-suppressive miRNAs silenced through aberrant DNA methylation. The expression of those four genes was restored by treatment with 5-aza-2'-deoxycytidine in OSCC cells lacking their expression, In addition, expression levels of the four miRNAs were inversely correlated with their DNA methylation status in the OSCC lines. In primary tumors of OSCC with paired normal oral mucosa, down-regulation of miRNA expression through tumor-specific hypermethylation was more frequently observed for miR-137 and miR-193a than for miR-34b and miR-203. Moreover, the ectopic transfection of miR-137 or miR-193a into OSCC lines lacking their expressions significantly reduced cell growth, with down-regulation of the translation of cyclin-dependent kinase 6 or E2F transcription factor 6, respectively. Taken together, our results clearly show that miR-137 and miR-193a are tumor suppressor miRNAs epigenetically silenced during oral carcinogenesis.", 
"Intrauterine growth retardation (IUGR) has been linked to the onset of diseases in adulthood, including type 2 diabetes, and has been proposed to result from altered gene regulation patterns due to epigenetic modifications of developmental genes. To determine whether epigenetic modifications may play a role in the development of adult diabetes following IUGR, we used a rodent model of IUGR that expresses lower levels of Pdx1, a pancreatic and duodenal homeobox 1 transcription factor critical for 0 cell function and development, which develops diabetes in adulthood. We found that expression of Pdx1 was permanently reduced in IUGR beta cells and underwent epigenetic modifications throughout development. The fetal IUGR state was characterized by loss of USF-1 binding at the proximal promoter of Pdx1, recruitment of the histone deacetylase 1 (HDAC1) and the corepressor Sin3A, and deacetylation of histones H3 and H4. Following birth, histone 3 lysine 4 (H3K4) was demethylated and histone 3 lysine 9 (H3K9) was methylated. During the neonatal period, these epigenetic changes and the reduction in Pdx1 expression could be reversed by HDAC inhibition. After the onset of diabetes in adulthood, the CpG island in the proximal promoter was methylated, resulting in permanent silencing of the Pdx1 locus. These results provide insight into the development of type 2 diabetes following IUGR and we believe they are the first to describe the ontogeny of chromatin remodeling in vivo from the fetus to the onset of disease in adulthood.", 
"The epidemic of obesity took off from about 1980 and in almost all countries has been rising inexorably ever since. Only in 1997 did WHO accept that this was a major public health problem and, even then, there was no accepted method for monitoring the problem in children. It was soon evident, however, that the optimum population body mass index is about 21 and this is particularly true in Asia and Latin America where the populations are very prone to developing abdominal obesity, type 2 diabetes and hypertension. These features are now being increasingly linked to epigenetic programming of gene expression and body composition in utero and early childhood, both in terms of fat/lean tissue ratios and also in terms of organ size and metabolic pathway regulation. New Indian evidence suggests that insulin resistance at birth seems linked to low birth weight and a higher proportion of body fat with selective B12 deficiency and abnormalities of one carbon pool metabolism potentially responsible and affecting 75% of Indians and many populations in the developing world. Biologically there are also adaptive biological mechanisms which limit weight loss after weight gain and thereby in part account for the continuing epidemic despite the widespread desire to slim. Logically, the burden of disease induced by inappropriate diets and widespread physical inactivity can be addressed by increasing physical activity (PA), but simply advocating more leisure time activity is unrealistic. Substantial changes in urban planning and diet are needed to counter the removal of any every day need for PA and the decades of misdirected food policies which with free market forces have induced our current 'toxic environment'. Counteracting this requires unusual policy initiatives.", 
"Environmental effects on the materno-foetal interaction determine birth outcomes that predict health over the lifespan. Thus, maternal undernutrition or stress associate with low birth weight, leading to an increased risk of metabolic and cardiovascular illness in the offspring. We argue that these effects are, in part, mediated by direct and indirect effects on the hypothalamic-pituitary-adrenal (HPA) axis such that (i) the effect of maternal adversity on foetal growth is mediated by adrenal glucocorticoids and (ii) environmental adversity alters maternal physiology and behaviour, which then programs HPA activity in the offspring.", 
"While cisplatin and carboplatin are active versus most common cancers, epithelial malignancies are incurable when metastatic. Even if an initial response occurs, acquired resistance due to mutations and epigenetic events limits efficacy. Resistance may be due to excess of a resistance factor, to saturation of factors required for tumor cell killing, or to mutation or alteration of a factor required for tumor cell killing. Platinum resistance could arise from decreased tumor blood flow, extracellular conditions, reduced platinum uptake, increased efflux, intracellular detoxification by glutathione, etc., decreased binding (e.g., due to high intracellular pH), DNA repair, decreased mismatch repair, defective apoptosis, antiapoptotic factors, effects of several signaling pathways, or presence of quiescent non-cycling cells. In lung cancer, flattening of dose-response Curves at higher doses suggests that efficacy is limited by exhaustion of something required for cell killing, and several clinical observations suggest epigenetic events may play a major role in resistance. (c) 2007 Elsevier Ireland Ltd. All rights reserved.", 
"We are in an era where the potential exists for deriving comprehensive profiles of DNA alterations characterizing each form of human cancer. Such profiles would provide invaluable insight into mechanisms underlying the evolution of each tumor type and will provide molecular markers, which could radically improve cancer detection. To date, no one type of DNA change has been defined which accomplishes this purpose. Herein, by using a candidate gene approach, we shaw that one category of DNA alteration, aberrant methylation of gene promoter regions, can enormously contribute to the above goals. We have now analyzed a series of promoter hypermethylation changes in 12 genes (p16(INK4a), p15(INK4b), p14(ARF), p73, APC(5), BRCA1, hMLH1, GSTP1, MGMT, CDH1, TIMP3, and DAPK) each rigorously characterized for association with abnormal gene silencing in cancer, in DNA from over 600 primary tumor samples representing 15 major tumor types. The genes play known important roles in processes encompassing tumor suppression, cell cycle regulation, apoptosis, DNA repair, and metastastic potential. A unique profile of promoter hypermethylation exists for each human cancer in which some gene changes are shared and others are cancer-type specific. The hypermethylatian of the genes occurs independently to the extent that a panel of three to four markers defines an abnormality in 70-90% of each cancer type. Our results provide an unusual view of the pervasiveness of DNA alterations, in this case an epigenetic change, in human cancer and a powerful set of markers to outline the disruption of critical pathways in tumorigenesis and for derivation of sensitive molecular detection strategies for virtually every human tumor type.", 
"Since its discovery in 1983, the epigenetics of human cancer has been in the shadows of human cancer genetics. But this area has become increasingly visible with a growing understanding of specific epigenetic mechanisms and their role in cancer, including hypomethylation, hypermethylation, loss of imprinting and chromatin modification. This timeline traces the field from its conception to the present day. It also addresses the genetic basis of epigenetic changes - an emerging area that promises to unite cancer genetics and epigenetics, and might serve as a model for understanding the epigenetic basis of human disease more generally.", 
"The past few years have seen an explosion of interest in the epigenetics of cancer. This has been a consequence of both the exciting coalescence of the chromatin and DNA methylation fields, and the realization that DNA methylation changes are involved in human malignancies. The ubiquity of DNA methylation changes has opened the way to a host of innovative diagnostic and therapeutic strategies. Recent advances attest to the great promise of DNA methylation markers as powerful future tools in the clinic.", 
"Cancer-associated DNA hypomethylation is as prevalent as cancer-linked hypermethylation, but these two types of epigenetic abnormalities usually seem to affect different DNA sequences. Much more of the genome is generally subject to undermethylation rather than overmethylation. Genomic hypermethylation in cancer has been observed most often in CpG islands in gene regions. In contrast, very frequent hypomethylation is seen in both highly and moderately repeated DNA sequences in cancer, including heterochromatic DNA repeats, dispersed retrotransposons, and endogenous retroviral elements. Also, unique sequences, including transcription control sequences, are often subject to cancer-associated undermethylation. The high frequency of cancer-linked DNA hypomethylation, the nature of the affected sequences, and the absence of associations with DNA hypermethylation are consistent with an independent role for DNA undermethylation in cancer formation or tumor progression. Increased karyotypic instability and activation of tumor-promoting genes by cis or trans effects, that might include altered heterochromatin-euchromatin interactions, may be important consequences of DNA hypomethylation which favor oncogenesis. The relationship of DNA hypomethylation to tumorigenesis is important to be considered in the light of cancer therapies involving decreasing DNA methylation. Inducing DNA hypomethylation may have short-term anticancer effects, but might also help speed tumor progression from cancer cells surviving the DNA demethylation chemotherapy.", 
"5-Azacytidine was first synthesized almost 40 years ago. It was demonstrated to have a wide range of anti-metabolic activities when tested against cultured cancer cells and to be an effective chemotherapeutic agent for acute myelogenous leukemia. However, because of 5-azacytidine's general toxicity, other nucleoside analogs were favored as therapeutics. The finding that 5-azacytidine was incorporated into DNA and that, when present in DNA, it inhibited DNA methylation, led to widespread use of 5-azacytidine and 5-aza-2'-deoxycytidine (Decitabine) to demonstrate the correlation between loss of methylation in specific gene regions and activation of the associated genes. There is now a revived interest in the use of Decitabine as a therapeutic agent for cancers in which epigenetic silencing of critical regulatory genes has occurred. Here, the current status of our understanding of the mechanism(s) by which 5-azacytosine residues in DNA inhibit DNA methylation is reviewed with an emphasis on the interactions of these residues with bacterial and mammalian DNA (cytosine-C5) methyltransferases. The implications of these mechanistic studies for development of less toxic inhibitors of DNA methylation are discussed.", 
"We have come a long way since the first reports of the existence of aberrant DNA methylation in human cancer. Hypermethylation of CpG islands located in the promoter regions of tumor suppressor genes is now firmly established as an important mechanism for gene inactivation. CpG island hypermethylation has been described in almost every tumor type. Many cellular pathways are inactivated by this type of epigenetic lesion: DNA repair (hMLH1, MGMT), cell cycle (p16(INK4a), p15(INK4b), p14(ARF)), apoptosis (DAPK), cell adherence (CDH1, CDH13), detoxification (GSTP1), etc... However, we still know little of the mechanisms of aberrant methylation and why certain genes are selected over others. Hypermethylation is not an isolated layer of epigenetic control, but is linked to the other pieces of the puzzle such as methyl-binding proteins, DNA methyltransferases and histone deacetylase, but our understanding of the degree of specificity of these epigenetic layers in the silencing of specific tumor suppressor genes remains incomplete. The explosion of user-friendly technologies has given rise to a rapidly increasing list of hypermethylated genes. Careful functional and genetic studies are necessary to determine which hypermethylation events are truly relevant for human tumorigenesis. The development of CpG island hypermethylation profiles for every form of human tumors has yielded valuable pilot clinical data in monitoring and treating cancer patients based in our knowledge of DNA methylation. Basic and translational will both be needed in the near future to fully understand the mechanisms, roles and uses of CpG island hypermethylation in human cancer. The expectations are high.", 
"DNA methylation is an important regulator of gene transcription, and its role in carcinogenesis has been a topic of considerable interest in the last few years. Alterations in DNA methylation are common in a variety of tumors as well as in development. Of all epigenetic modifications, hypermethylation, which represses transcription of the promoter regions of tumor suppressor genes leading to gene silencing, has been most extensively studied. However, global hypomethylation has also been recognized as a cause of oncogenesis. New information concerning the mechanism of methylation and its control has led to the discovery of many regulatory proteins and enzymes. The contribution of dietary folate and methylene terahydrofolate reductase polymorphisms to methylation patterns in normal and cancer tissues is under intense investigation. As methylation occurs early and can be detected in body fluids, it may be of potential use in early detection of tumors and for determining the prognosis. Because DNA methylation is reversible, drugs like 5'-azacytidine, decitabine, and histone deacetylase inhibitors are being used to treat a variety of tumors. Novel demethylating agents such as antisense DNA methyl transferase and small interference RNA are being developed, making the field of DNA methylation wider and more exciting. (C) 2004 by American Society of Clinical Oncology.", 
"The role of DNA methylation and of the maintenance DNA methyltransferase Dnmt1 in the epigenetic regulation of developmental stage- and cell lineage-specific gene expression in vivo is uncertain. This is addressed here through the generation of mice in which Dnmt1 was inactivated by Cre/loxP-mediated deletion at sequential stages of T cell development. Deletion of Dnmt1 in early double-negative thymocytes led to impaired survival of TCR alpha beta (+) cells and the generation of atypical CD8(+)TCR gamma delta (+) cells. Deletion of Dnmt1 in double-positive thymocytes impaired activation-induced proliferation but differentially enhanced cytokine mRNA expression by naive peripheral T cells. We conclude that Dnmt1 and DNA methylation are required for the proper expression of certain genes that define fate and determine function in T cells.", 
"DNA hypermethylation in CpG-rich promoters is now recognized as a common feature of human neoplasia. However, the pathophysiology of hypermethylation (why, when, where) remains obscure. Cancers can be classified according to their degree of methylation, and those cancers with high degrees of methylation (the CpG island methylator phenotype, or CIMP) represent a clinically and aetiologically distinct group that is characterized by 'epigenetic instability'. Furthermore, CIMP-associated cancers seem to have a distinct epidemiology, a distinct histology, distinct precursor lesions and distinct molecular features.", 
"The idea that tumors must escape from immune recognition contains the implicit assumption that tumors can be destroyed by immune responses either spontaneously or as the result of immunotherapeutic intervention. Simply put, there is no need for tumor escape without immunological pressure. Here, we review evidence supporting the immune escape hypothesis and critically explore the mechanisms that may allow such escape to occur. We discuss the idea that the central engine for generating immunoresistant tumor cell variants is the genomic instability and dysregulation that is characteristic of the transformed genome. Natural selection of heterogeneous tumor cells results in the survival and proliferation of variants that happen to possess genetic and epigenetic traits that facilitate their growth and immune evasion. Tumor escape variants are likely to emerge after treatment with increasingly effective immunotherapies.", 
"Background: The recently identified RASSF1 locus is located within a 120-kilobase region of chromosome 3p21.3 that frequently undergoes allele loss in lung and breast cancers. We explored the hypothesis that RASSF1 encodes a tumor suppressor gene for lung and breast cancers. Methods: We assessed expression of two RASSF1 gene products, RASSF1A and RASSF1C, and the methylation status of their respective promoters in 27 non-small-cell lung cancer (NSCLC) cell lines, in 107 resected NSCLCs, in 47 small-cell Lung cancer (SCLC) cell lines, in 22 breast cancer cell lines, in 39 resected breast cancers, in 104 nonmalignant lung samples, and in three breast and lung epithelial cultures, We also transfected a lung cancer cell line that lacks RASSF1A expression with vectors containing RASSF1A complementary DNA to determine whether exogenous expression of RASSF1A would affect in vitro growth and in vivo tumorigenicity of this cell line. Ail statistical tests were two-sided. Results: RASSF1A messenger RNA was expressed in nonmalignant epithelial cultures but not in 100% of the SCLC, in 65% of the NSCLC, or in 60% of the breast cancer lines. By contrast, RASSF1C was expressed in all nonmalignant cell cultures and in nearly all cancer cell lines. RASSF1A promoter hypermethylation was detected in 100% of SCLC, in 63% of NSCLC, in 63% of breast cancer lines, in 30% of primary NSCLCs, and in 39% of primary breast tumors but in none of the nonmalignant lung tissues, RASSF1A promoter hypermethylation in resected NSCLCs was associated with impaired patient survival (P =.046), Exogenous expression of RASSF1A in a cell line lacking expression decreased in vitro colony formation and in vivo tumorigenicity. Conclusion: RASSF1A is a potential tumor suppressor gene that undergoes epigenetic inactivation in lung and breast cancers through hypermethylation of its promoter region.", 
"Purpose: In the setting of a prospective clinical trial, we determined the predictive value of the methylation status of the O-6-methylguanine-DNA methyltransferase (MGMT) promoter for outcome in glioblastoma patients treated with the alkylating agent temozolomide. Expression of this excision repair enzyme has been associated with resistance to alkylating chemotherapy. Experimental Design: The methylation status of MGMT in the tumor biopsies was evaluated in 38 patients undergoing resection for newly diagnosed glioblastoma and enrolled in a Phase 11 trial testing concomitant and adjuvant temozolomide and radiation. The epigenetic silencing of the MGMT gene was determined using methylation-specific PCR. Results: Inactivation of the MGMT gene by promoter methylation was associated with longer survival (P = 0.0051; Log-rank test). At 18 months, survival was 62% (16 of 26) for patients testing positive for a methylated MGMT promoter but reached only 8% (1 of 12) in absence of methylation (P = 0.002; Fisher's exact test). In the presence of other clinically relevant factors, methylation of the MGMT promoter remains the only significant predictor (P = 0.017; Cox regression). Conclusions: This prospective clinical trial identifies MGMT-methylation status as an independent predictor for glioblastoma patients treated with a methylating agent. The association of the epigenetic inactivation of the DNA repair gene MGMT with better outcome in this homogenous cohort may have important implications for the design of future trials and supports efforts to deplete MGMT by O-6-benzylguanine, a noncytotoxic substrate of this enzyme.", 
"The Cyclin D-Cdk4,6/lNK4/Rb/E2F pathway plays a key role in controlling cell growth by integrating multiple mitogenic and antimitogenic stimuli. The components of this pathway are gene families with a high level of structural and functional redundancy and are expressed in an overlapping fashion in most tissues and cell types. Using classical transgenic technology as well as gene-targeting in ES cells, a series of mouse models have been developed to study the in vivo function of individual components of this pathway in both normal homeostasis and tumor development. These models have proven to be useful to define specific as well as redundant roles among members of these cell cycle regulatory gene families. This pathway is deregulated in the vast majority of human tumors by genetic and epigenetic alterations that target at least some of its key members such as Cyclin D1, Cdk4, INK4a and INK4b, pRb etc. As a consequence, some of these molecules are currently being considered as targets for cancer therapy, and several novel molecules, such as Cdk inhibitors, are under development as potential anti-cancer drugs. (C) 2002 Elsevier Science B.V. All rights reserved.", 
"Given the vast number of genetic and epigenetic changes associated with carcinogenesis, it is clear that tumors express many neoantigens. A central question in cancer immunology is whether recognition of tumor antigens by the immune system leads to activation (i.e., surveillance) or tolerance. Paradoxically, while strong evidence exists that specific immune surveillance systems operate at early stages of tumorigenesis, established tumors primarily induce immune tolerance. A unifying hypothesis posits that the fundamental processes of cancer progression, namely tissue invasion and metastasis, are inherently proinflammatory and thus activating for innate and adaptive antitumor immunity. To elude immune surveillance, tumors must develop mechanisms that block the elaboration and sensing of proinflammatory danger signals, thereby shifting the balance from activation to tolerance induction. Elucidation of these mechanisms provides new strategies for cancer immunotherapy.", 
"Cancer is an epigenetic disease at the same level that it can be considered a genetic disease. In fact, epigenetic changes, particularly DNA methylation, are susceptible to change and are excellent candidates to explain how certain environmental factors may increase the risk of cancer. The delicate organization of methylation and chromatin states that regulates the normal cellular homeostasis. of gene expression patterns becomes unrecognizable in the cancer cell. The genome of the transformed cell undergoes simultaneously a global genomic hypomethylation and a dense hypermethylation of the CpG islands associated with gene regulatory regions. These dramatic changes may lead to chromosomal instability, activation of endogenous parasitic sequences, loss of imprinting, illegitimate expression, aneuploidy, and mutations, and may contribute to the transcriptional silencing of tumour suppressor genes. The hypermethylation-associated inactivation affects virtually all of the pathways in the cellular network, such as DNA repair (hMLH1, BRCA1, MGMT,...), the cell cycle (p16(INK4a), p14(ARF), p15(INK4b),...), and apoptosis (DAPK, APAF-1,...). The aberrant CpG island methylation can also be used as a biomarker of malignant cells and as a predictor of their behaviour, and may constitute a good target for future therapies. Copyright (C) 2002 John Wiley Sons, Ltd.", 
"The field of epigenetics has recently moved to the forefront of studies relating to diverse processes such as transcriptional regulation, chromatin structure, genome integrity, and tumorigenesis. Recent work has revealed how DNA methylation and chromatin structure are linked at the molecular level and how methylation anomalies play a direct causal role in tumorigenesis and genetic disease. Much new information has also come to light regarding the cellular methylation machinery, known as the DNA methyltransferases, in terms of their roles in mammalian development and the types of proteins they are known to interact with, This information has forced a new view for the role of DNA methyltransferases. Rather than enzymes that act in isolation to copy methylation patterns after replication, the types of interactions discovered thus far indicate that DNA methyltransferases may be components of larger complexes actively involved in transcriptional control and chromatin structure modulation. These new findings will likely enhance our understanding of the myriad roles of DNA methylation in disease as well as point the may to novel therapies to prevent or repair these defects.", 
"The epigenome is defined by DNA methylation patterns and the associated posttranslational modifications of histones. This histone code determines the expression status of individual genes dependent upon their localization on the chromatin. The silencing of gene expression is associated with deacetylated histones, which are often found to be associated with regions of DNA methylation as well as methylation at the lysine 4 residue of histone 3. In contrast, the activation of gene expression is associated with acetylated histones and methylation at the lysine 9 residue of histone 3. The histone deactylases play a major role in keeping the balance between the acetylated and deacetylated states of chromatin. Histone deacetylases (HDACs) are divided into three classes: class I HDACs (HDACs 1, 2, 3, and 8) are similar to the yeast RPD3 protein and localize to the nucleus; class 11 HDACs (HDACs 4, 5, 6, 7, 9, and 10) are homologous to the yeast HDA1 protein and are found in both the nucleus and cytoplasm; and class III HDACs form a structurally distinct class of NAD-dependent enzymes that are similar to the yeast SIR2 proteins. Since inappropriate silencing of critical genes can result in one or both hits of tumor suppressor gene (TSG) inactivation in cancer, theoretically the reactivation of affected TSGs could have an enormous therapeutic value in preventing and treating cancer. Indeed, several HDAC inhibitors are currently being developed and tested for their potency in cancer chemotherapy. Importantly, these agents are also potentially applicable to chemoprevention if their toxicity can be minimized. Despite the toxic side effects and lack of specificity of some of the inhibitors, progress is being made. With the elucidation of the structures, functions and modes of action of HDACs, finding agents that may be targeted to specific HDACs and potentially reactivate expression of only a defined set of affected genes in cancer will be more attainable.", 
"CpG island hypermethylation is a mechanism of gene silencing that can be usurped by neoplastic cells to inactivate undesirable genes. In the colon, hypermethylation often starts in normal mucosa as a function of age and Is markedly increased in cancer. To test the hypothesis that subjects at increased risk of colon canter have higher levels of methylation in their nonneoplastic mucosa, we studied methylation patterns of five genes in the normal and dysplastic mucosa of patients with ulcerative colitis (UC), a condition associated with a marked increased risk of colon cancer. One gene (Mlh1) was unmethylated in all tissues examined, All four remaining genes had low but detectable levels of methylation in the epithelium of UC patients without evidence of dysplasia, and this methylation was not different from non-UC controls. BS contrast, all four genes were highly methylated in dysplastic epithelium from high-grade dysplasia (HGD)/ cancer patients with UC; methylation in HGD versus controls averaged 40.0% versus 7.4% (P = 0.00003) for ER, 44.0% versus 3.0% (P < 0.00003) for MYOD, 9.4% versus 2.4% (P = 0.03) for p16 exon 1, and 57.5% versus 30.6% (P = 0.01) fur CSPG2, Importantly, three of the four genes were also highly methylated in the normal appearing (nondysplastic) epithelium from these same HGD/cancer patients, indicating that methylation precedes dysplasia and is widespread in these patients. Compared with controls, methylation averaged 20.1% versus 7.2% (P = 0.07) for ER, 18.4% versus 3.0% (P < 0.008) for MYOD, and 7.9% versus 2.4% (P = 0.007) for p16 exon 1. These results are consistent with the hypothesis that age-related methylation marks (and may lead to) the field defect that reflects acquired predisposition to colorectal neoplasia, Furthermore, the data suggest that chronic inflammation is associated with high levels of methylation, perhaps as a result of increased cell turnover, and that UC can be viewed as resulting in premature aging of colorectal epithelial cells.", 
"O-6-Methylguanine DNA methyltransferase (MGMT) is a DNA repair protein that removes mutagenic and cytotoxic adducts from the O-6 position of guanine, O-6-methylguanine mispairs with thymine during replication, and if the adduct is not removed, this results in conversion from a guanine-cytosine pair to an adenine-thymine pair. In vitro assays show that MGMT expression avoids G to A mutations and MGMT transgenic mice are protected against G to A transitions at ras genes. We have recently demonstrated that the MGMT gene is silenced by promoter methylation in many human tumors, including colorectal carcinomas. To study the relevance of defective MGMT function by aberrant methylation in relation to the presence of K-ras mutations, we studied 244 colorectal tumor samples for MGMT promoter hypermethylation and K-ras mutational status. Our results show a clear association between the inactivation of MGMT by promoter hypermethylation and the appearance of G to A mutations at K-ras: 71% (36 of 51) of the tumors displaying this particular type of mutation had abnormal MGMT methylation, whereas only 32% (12 of 37) of those with other K-ras mutations not involving G to A transitions and 35% (55 of 156) of the tumors without R-ras mutations demonstrated MGMT methylation (P = 0.002), In addition, MGMT loss associated with hypermethylation was observed in the small adenomas, including those that do not yet contain K-ras mutations. Hypermethylation of other genes such as p16(INK4 alpha) and p14(ARF) was not associated with either MGMT hypermethylation or K-ras mutation. Our data suggest that epigenetic silencing of MGMT by promoter hypermethylation may Lead to a particular genetic change in human cancer, specifically G to A transitions in the K-ras oncogene.", 
"In children, cancer probably arises from a combination of inherited genetic mutations and genetic alterations that are acquired during the rapid cellular expansion that occurs during embryogenesis, and it is rarely associated with immune cell infiltrates. Conversely, in adults, cancer is frequently preceded by a long period of subclinical inflammatory disease and micronecrosis that provides a setting in which the epigenetic regulation of genes, cell death, cell proliferation and mutagenesis occurs. Here, we provocatively suggest that adult cancer results from rounds of disordered and unscheduled necrotic cell death, subsequent epithelial proliferation and the resulting suppressed immunity, rather than from a process that is dictated solely by cell growth. This paradigm shift regarding the development of cancer and this 'sixth sense' of the immune system indicates new strategies for cancer prevention and therapy.", 
"The INK4a/ARF locus encodes two cell cycle-regulatory proteins, p16(INK4a) and p14(ARF), which share an exon using different reading frames. p14(ARF) antagonizes MDM2-dependent p53 degradation. However, no point mutations in p14(ARF) not altering p16(INK4a) have been described in primary tumors. We report that p14(ARF) is epigenetically inactivated in several colorectal cell lines, and its expression is restored by treatment with demethylating agents. In primary colorectal carcinomas, p14(ARF) promoter hypermethylation was found in 31 of 110 (28%) of the tumors and observed in 13 or 41 (32%) colorectal adenomas but was not present in any normal tissues. p14(ARF) methylation appears in the context of an adjacent unmethylated p16(INK4a) promoter in 16 of 31 (52%) of the carcinomas methylated at p14(ARF). Although p14(ARF) hypermethylation was slightly overrepresented in tumors with wild-type p53 compared to tumors harboring p53 mutations [19 of 55 (34%) versus 12 of 55 (22%)], this difference did not reach statistical significance, p14(ARF) aberrant methylation was not related to the presence of K-ras mutations. Our results demonstrate that p14(ARF) promoter hypermethylation is frequent in colorectal cancer and occurs independently of the p16(INK4a) methylation status and only marginally in relation to the p53 mutational status.", 
"Polycomb group protein Ezh2 is an essential epigenetic regulator of embryonic development in mice, but its role in the adult organism is unknown. High expression of Ezh2 in developing murine lymphocytes suggests Ezh2 involvement in lymphopoiesis. Using Cre-mediated conditional mutagenesis, we demonstrated a critical role for Ezh2 in early B cell development and rearrangement of the immunoglobulin heavy chain gene (Igh). We also revealed Ezh2 as a key regulator of histone H3 methylation in early B cell progenitors. Our data suggest Ezh2-dependent histone H3 methylation as a novel regulatory mechanism controlling Igh rearrangement during early murine B cell development.", 
"Esophageal adenocarcinoma (EAC) arises after normal squamous mucosa undergoes metaplasia to specialized columnar epithelium (intestinal metaplasia or Barrett's esophagus), which can then ultimately progress to dysplasia and subsequent malignancy. Epigenetic studies of this model have thus far been limited to the DNA methylation analysis of a few genes. In this study, we analyzed a panel of 20 genes using a quantitative, high-throughput methylation assay, MethyLight. Ne used this broader approach to gain insight into concordant methylation behavior between genes and to generate epigenomic fingerprints for the different histological stages of EAC. Our study included a total of 104 tissue specimens from 51 patients with different stages of Barrett's esophagus and/or associated adenocarcinoma. We screened 84 of these samples with the full panel of 20 genes and found distinct classes of methylation patterns in the different types of tissue. The most informative genes,were those with an intermediate frequency of significant hypermethylation [ranging from 15% (CDKN2A) to 60% (MGMT) of the samples]. This group could be further subdivided into three classes, according to the absence (CDKN2A, ESRI, and MYODI) or presence (CALCA, MGMT, and TIMP3) of methylation in normal esophageal mucosa and stomach, or the infrequent methylation of normal esophageal mucosa accompanied by methylation in all normal stomach samples (APC). The other genes were less informative, because the frequency of hypermethylation was below 5% (ARF, CDH1, CDKN2B, GSTP1, MLH1, PTGS2, and THBS1), completely absent (CTNNB1, RBI, TGFBR2, and TYMS1), or ubiquitous (H1C1 and MTHFR), regardless of tissue type. Each class undergoes unique epigenetic changes at different steps of disease progression of EAC, suggesting a step-wise loss of multiple protective barriers against CpG island hypermethylation. The aberrant hypermethylation occurs at many different loci in the same tissues, suggestive of an overall deregulation of methylation control in EAC tumorigenesis. However, we did not find evidence for a distinct group of tumors with a CpG island methylator phenotype. Finally,we found that normal and metaplastic tissues from patients with evidence of associated dysplasia or cancer had a significantly higher incidence of hypermethylation than similar tissues from patients with no further progression of their disease. The fact that the samples from these two groups of patients were histologically indistinguishable, yet molecularly distinct, suggests that the occurrence of such hypermethylation may provide a clinical tool to identify patients with premalignant Barrett's who are at risk for further progression.", 
"Promoter hypermethylation is an important. pathway for repression of gene transcription in cancer cells. We analyzed aberrant DNA methylation at four genes in primary tumors from 95 head and neck cancer patients and then used the presence of this methylation as a marker for cancer cell detection in serum DNA, These four genes were tested by ethylation-specific PCR and included: p16 (CDKN2A) O-6-methylguanine-DNA-methyltransferase, glutathione S-transferase P1, and death-associated protein kinase (DAP-kinase), Fifty-five % (52 of 95) of the primary tumors displayed promoter hypermethylation in at least one of the genes studied: 27% (26/95) at p16, 33% (31 of 95) at O-6-methylguanine-DNA-methyltransferase; and 18% (17 of 92) at DAP-kinase. No promoter hypermethylation was observed at the glutathione S-transferase P1 gene promoter. We detected a statistically significant correlation between the presence of DAP-kinase gene promoter hypermethylation and lymph node involvement (P = 0.014) and advanced disease stage (P = 0.016). In 50 patients with paired serum available for epigenetic analysis, the same methylation pattern was detected in the corresponding serum DNA of 21 (42%) cases. Among the patients with methylated serum DNA, 5 developed distant metastasis compared with the occurrence of metastasis in only 1 patient negative for serum promoter hypermethylation (P = 0.056). Promoter hypermethylation of key genes in critical pathways is common in head and neck cancer and represents a promising serum marker for monitoring affected patients.", 
"The base sequence of DNA provides the genetic code for proteins. The regulation of expression or suppression of gene transcription is largely determined by the structure of the chromatin-referred to as epigenetic gene regulation (Agaliori et al., 2002: Jenuwein and Allis, 2001; Richards and Elgin, 2002; Spotswood and Turner, 2002; Zhang and Reinberg, 2001). Posttranslational modifications of the histories of chromatin play an important role in regulating gene expression. Some of the most extensively studied epigenetic modifications involve acetylation/deacetylation of lysines in the tails of the core histones, which is controlled by the action of histone deacetylases (HDACs) and histone aceryltransferases (HATs). A controlled balance between histone acerylation and deacetylation appears to be essential for normal cell growth (Waterborg. 2002). Alterations in the structure or expression of HATs and HDACs occur in many cancers (Jones and Baylin, 2002; Marks et al., 2001, 2003; Timmermann et al.. 2001; Wang et al., 2001). A structurally diverse group of molecules has been developed that can inhibit HDACs (HDACi) (Arts et al., 2003; Bouchain and Delorme, 2003; Curtin and Glaser, 2003; Johnstone and Licht, 2003; Marks et al., 2003, Remiszewski, 2003: Richon e. al., 1998; Yoshida et al., 2003). These inhibitors induce growth arrest, differentiation, and/or apoptosis of cancer cells in vitro and in in vivo tumor-bearing animal models. Clinical trials with several of these agents have shown that certain HDACi have antitumor activity against various cancers at doses that are well tolerated by patients (Gottlicher et al.,. 200 1: Kelly et al., 2002a,b; Piekarz et al., 2001; Wozniak et al.. 1999). (C) 2004 Elsevier Inc.", 
"Silencing of genes by aberrant promoter hypermethylation is now recognized as a crucial component in cancer initiation and progression. Highly sensitive assays have been developed to assess gene-promoter methylation in biological fluids. The detection of methylated genes in sputum could lead to the development of a screening test to non-invasively identify early cancer in high-risk people.", 
"Aberrant DNA methylation patterns may be the earliest somatic genome changes in prostate cancer. Using real-time methylation-specific PCR, we assessed the extent of hypermethylation at 16 CpG islands in DNA from seven prostate cancer cell lines (LNCaP, PC-3, DU-145, LAPC-4, CWR22Rv1, VCaP, and C42B), normal prostate epithelial cells, normal prostate stromal cells, 73 primary prostate cancers, 91 metastatic prostate cancers, and 25 noncancerous prostate tissues. We found that CpG islands at GSTP1, APC, RASSF1a, PTGS2, and MDR1 were hypermethylated in >85% of prostate cancers and cancer cell lines but not in normal prostate cells and tissues; CpG islands at EDNRB, ESR1, CDKN2a, and hMLH1 exhibited low to moderate rates of hypermethylation in prostate cancer tissues and cancer cell lines but were entirely unmethylated in normal tissues; and CpG islands at DAPK1, TIMP3, MGMT, CDKN2b,p14/ARF, and CDH1 were not abnormally hypermethylated in prostate cancers. Receiver operator characteristic curve analyses suggested that CpG island hypermethylation changes at GSTP1, APC, RASSF1a, PTGS2, and MDR1 in various combinations can distinguish primary prostate cancer from benign prostate tissues with sensitivities of 97.3-100% and specificities of 92-100%. Hypermethylation of the CpG island at EDNRB was correlated with the grade and stage of the primary prostate cancers. PTGS2 CpG island hypermethylation portended an increased risk of recurrence. Furthermore, CpG island hypermethylation patterns in prostate cancer metastases were very similar to the primary prostate cancers and tended to show greater differences between cases than between anatomical sites of metastasis.", 
"Background: Gene silencing by abnormal methylation of promoter regions of regulatory genes is commonly associated with cancer. Silenced tumor suppressor genes are obvious targets for reactivation by methylation inhibitors such as 5-azacytidine (5-Aza-CR) and 5-aza-2'-deoxycytidine (5-Aza-CdR). However, both compounds are chemically unstable and toxic and neither can be given orally. We characterized a new demethylating agent, zebularine [1-(beta-D-ribofuranosyl)-1,2-dihydropyrimidin-2-one], which is a chemically stable cytidine analog. Methods: We tested the ability of zebularine to reactivate a silenced Neurospora crassa gene using a hygromycin gene reactivation assay. We then analyzed the ability of zebularine to inhibit DNA methylation in C3H 10T1/2 C18 (10T1/2) mouse embryo cells as assayed by induction of a myogenic phenotype and in T24 human bladder carcinoma cells, using the methylation-sensitive single nucleotide primer extension (Ms-SNuPE) assay. We also evaluated the effects of zebularine (administered orally or intraperitoneally) on growth of EJ6 human bladder carcinoma cells grown in BALB/c nu/nu mice (five mice per group) and the in vivo reactivation of a methylated p16 gene in these cells. All statistical tests were two-sided. Results: In N. crassa, zebularine inhibited DNA methylation and reactivated a gene previously silenced by methylation. Zebularine induced the myogenic phenotype in 10T1/2 cells, which is a phenomenon unique to DNA methylation inhibitors. Zebularine reactivated a silenced p16 gene and demethylated its promoter region in T24 bladder carcinoma cells in vitro and in tumors grown in mice. Zebularine was only slightly cytotoxic to T24 cells in vitro (1 mM zebularine for 48 hours decreased plating efficiency by 17% [95% confidence interval (CI) = 12.8% to 21.2%]) and to tumor-bearing mice (average maximal weight change in mice treated with 1000 mg/kg zebularine = 11% [95% CI = 4% to 19%]). Compared with those in control mice, tumor volumes were statistically significantly reduced in mice treated with high-dose zebularine administered by intraperitoneal injection (P<.001) or by oral gavage (P<.001). Conclusions: Zebularine is a stable DNA demethylating agent and the first drug in its class able to reactivate an epigenetically silenced gene by oral administration.", 
"Human Forkhead-box (FOX) gene family consists of at least 43 members, including FOXA1, FOXA2, FOXA3, FOXB1, FOXC1, FOXC2, FOXD1, FOXD2, FOXD3, FOXD4, FOXD5 (FOXD4L1), FOXD6 (FOXD4L3), FOXE1, FOXE2, FOXE3, FOXF1, FOXF2, FOXG1 (FOXG1B), FOXH1, FOXI1, FOXJ1, FOXJ2, FOXJ3, FOXK1, FOXK2, FOXL1, FOXL2, FOXM1, FOXN1, FOXN2 (HTLF), FOXN3 (CHES1), FOXN4, FOXN5 (FOXR1), FOXN6 (FOXR2), FOXO1 (FOXO1A), FOXO2 (FOXO6), FOXO3 (FOXO3A), FOXO4 (MLLT7), FOXP1, FOXP2, FOXP3, FOXP4, and FOXQ1. FOXE3-FOXD2 (1p33), FOXQ1-FOXF2-FOXC1 (6p25.3), and FOXF1-FOXC2-FOXL1 (16q24.1) loci are FOX gene clusters within the human genome. Members of FOX subfamilies A-G, I-L and Q were grouped into class I FOX proteins, while members of FOX subfamilies H and M-P were grouped into class 2 FOX proteins. C-terminal basic region within the FOX domain was the common feature of class I FOX proteins. FOXH1 and FOXO1 mRNAs are expressed in human embryonic stem (ES) cells. FOXC1, FOXC2, FOXE1, FOXE3, FOXL2, FOXN1, FOXP2 and FOXP3 genes are mutated in human congenital disorders. FOXA1 gene is amplified and over-expressed in esophageal and lung cancer. FOXM1 gene is up-regulated in pancreatic cancer and basal cell carcinoma due to the transcriptional regulation by Sonic Hedgehog (SHH) pathway. FOXO1 gene is fused to PAX3 or PAX7 genes in rhabdomyosarcoma. FOXO3 and FOXO4 genes are fused to MLL gene in hematological malignancies. Deregulation of FOX family genes leads to congenital disorders, diabetes mellitus, or carcinogenesis. Expression profiles, genetic alterations and epigenetic changes of FOX family genes as well as binding proteins and target genes of FOX family transcription factors should be comprehensively investigated to develop novel therapeutics and preventives for human diseases.", 
" ", 
"A role for DNA demethylation in transcriptional regulation of genes expressed in differentiated somatic cells remains controversial. Here, we define a small region in the promoter-enhancer of the interleukin-2 (II2) gene that demethylates in T lymphocytes following activation, and remains demethylated thereafter. This epigenetic change was necessary and sufficient to enhance transcription in reporter plasmids. The demethylation process started as early as 20 minutes after stimulation and was not prevented by a G1 to S phase cell cycle inhibitor that blocks DNA replication. These results imply that this demethylation process proceeds by an active enzymatic mechanism.", 
"Recent studies from our laboratory suggest that gene-specific methylation changes in sputum could be good intermediate markers for the early detection of lung cancer and defining the efficacy of chemopreventive interventions. The purpose of our study was to determine the prevalence for aberrant promoter methylation of the p16, O-6-methylguanine-DNA methyltransferase (MGMT), death-associated protein (DAP) kinase, and Ras effector homologue (RASSFIA) genes in nonmalignant bronchial epithelial cells from current and former smokers in a hospital-based, case control study of lung cancer. The relationship between loss of heterozygosity, at 9p and p16 methylation in bronchial epithelium and the prevalence for methylation of these four genes in sputum from cancer-free, current and former smokers were also determined. Aberrant promoter methylation of p16 was seen in at least one bronchial epithelial site from 44% of cases and controls. Methylation of the DAP kinase gene was seen in only I site from 5 cases and 4 controls, whereas methylation of the RASSFIA was not detected in the bronchial epithelium. Promoter methylation for p16 and DAP kinase was seen as frequently in bronchial epithelium from current smokers as from former smokers. No promoter methylation of these genes was detected in bronchial epithelium from never-smokers. Methylation of the p16 gene was detected in sputum from 23 of 66 controls. DAP kinase gene promoter methylation was also seen in sputum from 16 controls, and 8 of these subjects were positive for p16 methylation. Methylation of the MGMT gene was seen in sputum from 9 controls, whereas RASSFIA promoter methylation was only seen in 2 controls. The correlation between p16 status in the bronchial epithelium obtained from lung lobes that did not contain the primary tumor and the tumor itself was examined. Seventeen of 18 tumors (94%) showed an absolute concordance, being either methylated in the tumor and at least I bronchial epithelial site, or unmethylated in both tumor and bronchial epithelium. These results indicate that aberrant promoter hypermethylation of the p16 gene, and to a lesser extent, DAP kinase, occurs frequently in the bronchial epithelium of lung cancer cases and cancer-free controls and persists after smoking cessation. The strong association seen between p16 methylation in the bronchial epithelium and corresponding primary tumor substantiates that inactivation of this gene,although not transforming by itself. is likely permissive for the acquisition of additional genetic and epigenetic changes leading to lung cancer.", 
"Cell-cell adhesion determines the polarity of cells and participates in the maintenance of the cell societies called tissues. Cell-cell adhesiveness is generally reduced in human cancers. Reduced intercellular adhesiveness allows cancer cells to disobey the social order, resulting in destruction of histological structure, which is the morphological hallmark of malignant tumors. Reduced intercellular adhesiveness is also indispensable for cancer invasion and metastasis. A tumor-suppressor gene product, E-cadherin, and its undercoat proteins, catenins, which connect cadherins to actin filaments, are located at lateral borders, concentrating on adherens junctions, of epithelial cells and establish firm cell-cell adhesion. The E-cadherin cell adhesion system in cancer cells is inactivated by various mechanisms that reflect the morphological and biological characteristics of the tumor. Silencing of the E-cadherin gene by DNA hypermethylation around the promoter region occurs frequently, even in precancerous conditions. In diffuse infiltrating cancers, mutations are found in the genes for E-cadherin and alpha- and beta-catenins. At the invading front of cancers, the E-cadherin cell adhesion system is inactivated by tyrosine phosphorylation of beta-catenin; an oncogene product, c-erbB-2 protein, is found to associate directly with beta-catenin. The E-cadherin cell adhesion system cross-talks with the Wingless/Wnt signaling pathway through beta-catenin, and expression of genes, which participate in cancer morphogenesis, may be regulated in conjunction with the Wingless/Wnt signaling pathway. Dysadherin, a newly identified cancer-associated cell membrane glycoprotein, down-regulates E-cadherin and promotes cancer metastasis. In conclusion, inactivation of the E-cadherin cell adhesion system by both genetic and epigenetic mechanisms plays a significant role during multistage human carcinogenesis.", 
"Background Hyperhomocysteinaemia occurs in several genetically determined and acquired disorders and is highly prevalent in patients with uraemia. In these disorders, homocysteine precursor S-adenosylhomocysteine, a powerful competitive inhibitor of S-adenosylmethionine-dependent methyltransferases, is increased, suggesting unbalanced methylation. We aimed to investigate whether DNA hypomethylation is present in patients with uraemia who also have hyperhomocysteinaemia and whether regulation of specific classes of genes, dependent on DNA methylation, is compromised. Methods We selected men with hyperhomocysteinaemia and uraemia who were having standard haemodialysis treatment, and compared them with healthy male controls. We measured the homocysteine concentration from plasma samples and obtained DNA and RNA samples from peripheral mononuclear cells. DNA methylation was assessed by cytosine extension assay and by Southern blotting. Allelic expression of pseudoautosomal and imprinted genes was investigated by analysis of suitable restriction fragment length polymorphisms. Findings Total DNA hypomethylation was higher in patients than in controls (z score -4.593, p=0.0006) and allelic expression was changed in both sex-linked and imprinted genes. The shift from monoallelic to biallelic expression was dependent on homocysteine concentrations. Folate therapy, a common method to reduce hyperhomocysteinaemia, restored DNA methylation to normal levels and corrected the patterns of gene expression. Interpretation Our results suggest that hyperhomocysteinaemia affects epigenetic control of gene expression, which can be reverted by folate treatment. Our data support the hypothesis that the toxic action of homocysteine can be mediated by macromolecule hypomethylation.", 
"DNA methylation is an epigenetic feature of DNA that influences cellular development and function, and aberrations of DNA methylation are a candidate mechanism for the development of cancer. Methylenetetrahydrofolate reductase (MTHFR) catalyzes the synthesis of 5-methyltetrahydrofolate, the methyl donor for methionine synthesis and the precursor of S-adenosylmethionine. S-adenosylmethionine is the universal methyl donor for methylation reactions, including that of DNA methylation. In the present study, we investigated whether a common C677T mutation in the MTHFR gene, which results in reduced enzyme activity in vitro, affects genomic DNA methylation. We selected 9 subjects homozygous for the wild-type MTHFR and 10 subjects homozygous for the mutation (T/T), Genomic DNA methylation was determined by an established enzymatic assay that measures the capacity of DNA to accept methyl groups in vitro, which is inversely related to endogenous methylation. DNA from subjects with the T/T MTHFR genotype had a significantly higher methyl group acceptance capacity (12,615 +/- 1836 dpm/2 mu g of DNA) compared with wild-type MTHFR (7843 1043 dpm/2 mu g of DNA; P < 0.05), indicating DNA hypomethylation in the T/T genotype, Furthermore, DNA methylation was directly and significantly related to RBC folate concentrations in persons with the T/T genotype, but not in those with wild-type MTHFR, These data are consistent with prior observations, which suggest that the T/T genotype is associated with impaired MTHFR activity in vivo and that the cellular impact of this impairment is determined, in part, by folate status. The relationship of genomic DNA hypomethylation in persons with the T/T MTHFR genotype to the development of cancer remains to be defined.", 
" ", 
"Methylation of cytosines within the CpG dinucleotide by DNA methyltransferases is involved in regulating transcription and chromatin structure, controlling the spread of parasitic elements, maintaining genome stability in the face of vast amounts of repetitive DNA, and X chromosome inactivation. Cellular DNA methylation is highly compartmentalized over the mammalian genome and this compartmentalization is essential for embryonic development. When the complicated mechanisms that control which DNA sequences become methylated go awry, a number of inherited genetic diseases and cancer may result. Much new information has recently come to light regarding how cellular DNA methylation patterns may be established during development and maintained in somatic cells. Emerging evidence indicates that various chromatin states such as histone modifications (acetylation and methylation) and nucleosome positioning (modulated by ATP-dependent chromatin remodeling machines) determine DNA methylation patterning. Additionally, various regulatory factors interacting with the DNA methyltransferases may direct them to specific DNA sequences, regulate their enzymatic activity, and allow their use as transcriptional repressors. Continued studies of the connections between DNA methylation and chromatin structure and the DNA methyltransferase-associated proteins, will likely reveal that many, if not all, epigenetic modifications of the genome are directly connected. Such studies should also yield new insights into treating diseases involving aberrant DNA methylation.", 
"Antigen and cytokine receptor signals act in synergy to direct the differentiation of CD4(+) T cells. These signals initiate reciprocal activation and silencing of the interferon-gamma (IFN-gamma) and interleukin 4 (IL-4) cytokine gene loci, changes that are heritably maintained in the resulting T helper type I (T(H)1) or T(H)2 cells and their progeny. Early, unpolarized transcription and chromatin remodeling of the poised cytokine genes of naive T cells is followed by consolidation and spreading of epigenetic changes and the establishment of self-reinforcing transcription factor networks. Recent studies have begun to elucidate the molecular mechanisms that establish and maintain polarized cytokine gene expression, and thus the cellular identity of differentiated helper T cells.", 
"We attempted to answer two central questions about epigenetic silencing of the tumor suppressor gene p16(INK4a) in this study: (1) whether the maintenance of associated histone modifications is dependent on DNA methylation and (2) whether such histone modifications can occur prior to DNA methylation. By coupling chromatin immunoprecipitation with gene targeting and the analysis of specific alleles, we found that elimination of DNA methylation from a p16(INK4a) allele resulted in profound changes in surrounding histones. After continued passage of such cells, methylation of histone H3 lysine-9 occurred in conjunction with re-silencing in the absence of DNA methylation. These results have important implications for understanding the biochemical events underlying the silencing of tumor suppressor genes and the resultant growth suppression.", 
"Genome-wide losses of DNA methylation have been regarded as a common epigenetic event in malignancies and may play crucial roles in carcinogenesis. Limited information is available on the global methylation status in normal tissues and other cancer types beyond colonic carcinoma. Here we applied the combined bisulfite restriction analysis PCR to evaluate the methylation status of LINE-1 repetitive sequences in genomic DNA derived from microdissected samples from several human normal and neoplastic tissues. We found that methylation of LINE-1 in leukocytes was independent of age and gender. In contrast, normal tissues from different organs showed tissue-specific levels of methylated LINE-1. Globally, most carcinomas including breast, colon, lung, head and neck, bladder, esophagus, liver, prostate, and stomach, revealed a greater percentage of hypomethylation than their normal tissue counterparts. Furthermore, DNA derived from sera of patients with carcinoma displayed more LINE-1 hypomethylation than those of noncarcinoma individuals. Finally, in a colonic carcinogenesis model, we detected significantly greater hypomethylation in carcinoma than those of dysplastic polyp and histological normal colonic epithelium. Thus, the methylation status is a unique feature of a specific tissue type and the global hypomethylation is a common epigenetic process in cancer, which may progressively evolve during multistage carcinogenesis.", 
"Cancer is a process driven by the accumulation of abnormalities in gene function. While many of these changes are genetic, epigenetically mediated changes in gene expression are being increasingly appreciated. This latter process emphasizes the need to understand two key components of heritable, but reversible, modulation of gene promoter function that are closely tied to one another - formation of chromatin which modulates transcription and establishing patterns of DNA methylation, The link lies first in the recruitment to methylated cytosines of a family of methyl-CpG binding domain proteins (MBDs), which are direct transcriptional repressors and can complex with transcriptional core-pressors including histone deacetylases (HDACs), Additionally, the proteins that catalyze DNA methylation, DNA methyltransferases (DNMTs), also directly repress transcription and associate with HDACs. Regulation of these above chromatin-DNA methylation interactions as a function of DNA replication timing is emerging as a key event in the inheritance of transcriptionally repressed domains of the genome. Importantly, synergy between HDAC activity and DNA methylation is operative for a key epigenetic abnormality in cancer cells, transcriptional silencing of tumor suppressor genes. This change has now been recognized for genes that are essential for normal regulation of virtually every major cell function including cell growth, differentiation, apoptosis, DNA repair, and cell-cell, cell-substratum interaction. Understanding the molecular determinants of both normal and abnormal patterns of chromatin formation and DNA methylation thus holds great promise for our understanding of cancer and for means to better diagnose, prevent, and treat this means to better disease.", 
"Post-translational modification of the histories of chromatin has a fundamental role in regulating gene expression. Enzymes involved in these epigenetic events include histone deacetylases (class I and class II), which can be inhibited by a structurally diverse group of small molecules. These histone deacetylase inhibitors induce growth arrest, differentiation and/or apoptosis of cancer cells in vitro and in vivo. Results of clinical trials with several of these agents have indicated that they are well tolerated at doses that have anti-tumour activity.", 
"The concept of stem-cell plasticity received strong support from a recent observation that extensively passaged, clonally derived neural stem cells could contribute to hematopoiesis. We investigated whether hematopoietic potential was a consistent or unusual feature of neural stem cells, and whether it depended on the extent of in vitro passaging before transplantation. Here we transplanted over 128 x 10(6) neurosphere cells into 128 host animals; however, we never observed contribution to hematopoiesis, irrespective of the number of passages and despite the use of an assay that could detect the contribution of a single blood stem cell to hematopoietic repopulation. Although extensively cultured neurosphere cells continued to generate neural progeny, marked changes in their growth properties occurred, including changes in growth-factor dependence, cell-cycle kinetics, cell adhesion and gene expression. Our results exclude hematopoietic competence as a consistent property of intravenously infused neural stem cells. However, the consistent changes that occurred during extended passaging are compatible with genetic or epigenetic alterations and suggest that rare transformation events may account for the neural-to-blood fate switch originally reported.", 
"Epigenetic modifications of cytosine residues in DNA and the amino termini of histone proteins have emerged as key mechanisms in chromatin remodeling, impacting both the transcriptional regulation and the establishment of chromosomal domains. Using the chromatin immunoprecipitation (ChIP) assay, we demonstrate that aberrantly silenced genes in cancer cells exhibit a heterochromatic structure that is characterized by histone H3 lysine 9 (H3-K9) hypermethylation and histone H3 lysine 4 (H3-K4) hypomethylation. This aberrant heterochromatin is incompatible with transcriptional initiation but does not inhibit elongation by RNA polymerase II. H3-K9 methylation may, therefore, play a role in the silencing of tumor-suppressor genes in cancer. Treatment with 5-aza-2'-deoxycytidine (5-Aza-CdR), previously known for its ability to inhibit cytosine methylation, induced a rapid and substantial remodeling of the heterochromatic domains of the p14ARF/p16INK4a locus in T24 bladder cancer cells, reducing levels of dimethylated H3-K9 and increasing levels of dimethylated H3-K4 at this locus. In addition, 5-Aza-CdR increased acetylation and H3-K4 methylation at the unmethylated p14 promoter, suggesting it can induce chromatin remodeling independently of its effects on cytosine methylation.", 
"E-cadherin is a cell-cell adhesion protein fulfilling a prominent role in epithelial differentiation. Data from model systems suggest that E-cadherin is a potent invasion/tumor suppressor of breast cancer. Consistent with this role in breast cancer progression, partial or complete loss of E-cadherin expression has been found to correlate with poor prognosis in breast cancer patients. The E-cadherin gene (CDH1) is located on human chromosome 16q22.1, a region frequently affected with loss of heterozygosity in sporadic breast cancer. Invasive lobular breast carcinomas, which are typically completely E-cadherin-negative, often show inactivating mutations in combination with loss of heterozygosity of the wild-type CDH1 allele. Mutations were found at early noninvasive stages, thus associating E-cadherin mutations with loss of cell growth control and defining CDH1 as the tumor suppressor for the lobular breast cancer subtype. Ductal breast cancers in general show heterogeneous loss of E-cadherin expression, associated with epigenetic transcriptional downregulation. It is proposed that the microenvironment at the invasive front is transiently downregulating E-cadherin transcription. This can be associated with induction of nonepithelial cadherins.", 
"Epigenetic alterations in human cancers include global DNA hypomethylation, gene hypomethylation and promoter hypermethylation, and loss of imprinting (LOI) of the insulin-like growth factor-II gene (IGF2). A mechanism for LOI described previously is hypermethylation of a differentially methylated region (DMR) upstream of the H19 gene, allowing activation of the normally silent maternal allele of IGF2. Here we show that this mechanism does not apply to colorectal cancers, which show hypomethylation of the H19 DMR as well as a DMR upstream of exon 3 of IGF2. This hypomethylation is found in both colorectal cancers and normal mucosa from the same patients, and in cell lines with somatic cell knockout of DNA methyltransferases DNMT1 and DNMT3B. These data suggest that hypomethylation is a mechanism for LOI, that the popular IGF2-H19 enhancer competition model for IGF2 imprinting does not apply to the human colon, and that an alternative model for LOI would involve a transcriptional repressor acting on the normally silent maternal allele of IGF2.", 
"DNA methylation is an important epigenetic mechanism of transcriptional control. DNA methylation plays an essential role in maintaining cellular function, and changes in methylation patterns may contribute to the development of cancer. Aberrant methylation of DNA (global hypomethylation accompanied by region-specific hypermethylation) is frequently found in tumor cells. Global hypomethylation can result in chromosome instability, and hypermethylation has been associated with the inaction of tumor suppressor genes. Preclinical and clinical studies suggest that part of the cancer-protective effects associated with several bioactive food components may relate to DNA methylation patterns. Dietary factors that are involved in one-carbon metabolism provide the most compelling data for the interaction of nutrients and DNA methylation because they influence the supply of methyl groups, and therefore the biochemical pathways of methylation processes. These nutrients include folate, vitamin B-12, vitamin B-6, methionine, and choline. However, looking at individual nutrients may be too simplistic. Dietary methyl (folate, choline, and methionine) deficiency in combination causes decreased tissue S-adenosylmethionine, global DNA hypomethylation, hepatic steatosis, cirrhosis, and ultimately hepatic tumorigenesis in rodents in the absence of carcinogen treatment. Other dietary components such as vitamin B-12, alcohol, and selenium may modify the response to inadequate dietary folate.", 
"Cadmium is a human carcinogen that likely acts via epigenetic mechanisms. Since DNA methylation alterations represent an important epigenetic event linked to cancer, the effect of cadmium on DNA methyltransferase (MeTase) activity was examined using in vitro (TRL1215 rat liver cells) and ex vivo (M.SssI DNA MeTase) systems. Cadmium effectively inhibited DNA MeTases in a manner that was noncompetitive with respect to substrate (DNA), indicating an interaction with the DNA binding domain rather than the active site. Based on these results, the effects of prolonged cadmium exposure on DNA MeTase and genomic DNA methylation in TRL1215 cells were studied. After I week of exposure to 0-2.5 muM cadmium, DNA MeTase activity was reduced (up to 40%) in a concentration-dependent fashion, while genomic DNA methylation showed slight but significant reductions at the two highest concentrations. After 10 weeks of exposure, the cells exhibited indications of transformation, including hyperproliferation, increased invasiveness, and decreased serum dependence. Unexpectedly, these cadmium-transformed cells exhibited significant increases in DNA methylation and DNA MeTase activity. These results indicate that, while cadmium is an effective inhibitor of DNA MeTase and initially induces DNA hypomethylation, prolonged exposure results in DNA hypermethylation and enhanced DNA MeTase activity. (C) 2003 Elsevier Science (USA). All rights reserved.", 
"The microenvironment of solid tumours contains regions of poor oxygenation and high acidity. Growing evidence from clinical and experimental studies points to a fundamental role for hypoxia in metastatic progression. Prolonged hypoxia increases genomic instability, genomic heterogeneity, and may act as a selective pressure for tumour cell variants. Hypoxia can also act in an epigenetic fashion, altering the expression of genes. Hypoxia-induced changes in gene expression alter non-specific stress responses, anaerobic metabolism, angiogenesis, tissue remodeling, and cell-cell contacts. Experimental studies have demonstrated that inhibition of proteins involved in these processes can modify metastasis formation, suggesting a causal role in metastatic progression. Recent advances in high-throughput screening techniques have allowed identification of many hypoxia-induced genes that are involved in the processes associated with metastasis. Here we review the epigenetic control of gene expression by the hypoxic microenvironment and its potential contribution to metastatic progression.", 
"The neoplastic process may involve a cancer stem cell. This concept has emerged largely from the careful analysis of tumour biopsy systems from haematological, breast and brain tumours. However, the experimental systems necessary to provide the cellular and molecular evidence to support this important concept have been lacking. We have used adult mesenchymal stem cells (hMSC) transduced with the telomerase hTERT gene to investigate the neoplastic potential of adult stem cells. The hTERT-transduced line, hMSC-TERT20 at population doubling level (PDL) 256 showed loss of contact inhibition, anchorage independence and formed tumours in 10/10 mice. hMSC-TERT4 showed loss of contact inhibition at PDL 95, but did not exhibit anchorage independence and did not form tumours in mice. Both lines had a normal karyotype but showed deletion of the Ink4a/ARF locus. At later passage, hMSC-TERT4 also acquired an activating mutation in KRAS. In hMSC-TERT20, expression of the cell cycle-associated gene, DBCCR1 was lost due to promoter hypermethylation. This epigenetic event correlated with acquisition of tumorigenicity. These data suggest that the adult hMSCs can be targets for neoplastic transformation and have implications for the development of novel anticancer therapeutics and for the use of hMSC in tissue engineering and transplantation protocols.", 
"The DNA repair protein O-6-methylguanine DNA methyltransferase (MGMT) removes alkyl adducts from the O-6 position of guanine, MGMT expression is decreased in some tumor tissues, and lack of activity has been observed in some cell lines. Loss of expression is rarely due to deletion, mutation, or rearrangement of the MGMT gene, but methylation of discrete regions of the CpG island of MGMT has been associated with the silencing of the gene in cell lines. We used methylation-specific PCR to study the promoter methylation of the MGMT gene. All normal tissues and expressing dancer cell lines were unmethylated, whereas nonexpressing cancer cell lines were methylated. Among the more than 500 primary human tumors examined, MGMT hypermethylation was present in a subset of specific types of cancer. In gliomas and colorectal carcinomas, aberrant methylation was detected in 40% of the tumors, whereas in non-small cell lung carcinomas, lymphomas, and head and neck carcinomas, this alteration was found in 25% of the tumors. MGMT methylation was found rarely or not at all in other tumor types. We also analyzed MGMT expression by immunohistochemistry in relation to the methylation status in 31 primary tumors. The presence of aberrant hypermethylation was associated with loss of MGMT protein, in contrast to retention of protein in the majority of tumors without aberrant hypermethylation, Our results suggest that epigenetic inactivation of MGMT plays an important role in primary human neoplasia.", 
"Helper T (Th) cell differentiation is highly regulated by cytokines but initiated by mitogens. By examining gene expression in discrete generations of dividing cells, we have delineated the relationship between proliferation and differentiation. Initial expression of IL-2 is cell cycle-independent, whereas effector cytokine expression is cell cycle-dependent. IFN gamma expression increases in frequency with successive cell cycles, while IL-4 expression requires three cell divisions. Cell cycle progression and cytokine signaling act in concert Po relieve epigenetic repression and can be supplanted by agents that hyperacetylate histones and demethylate DNA. Terminally differentiated cells exhibit stable epigenetic modification and cell cycle-independent gene expression. These data reveal a novel mechanism governing Th cell fate that initially integrates proliferative and differentiative signals and subsequently maintains stability of the differentiated state.", 
"Endoglin (CD105), an accessory protein of the TGF-beta receptor superfamily, is highly expressed on endothelial cells. Hereditary hemorrhagic telangiectasia type 1 (HHT1) is associated with mutations in the Endoglin gene, leading to haploinsufficiency. To generate a disease model and ascertain the role of endoglin in development, we generated mice lacking 1 or both copies of the gene. Endoglin null embryos die at gestational day 10.0-10.5 due to defects in vessel and heart development. Vessel formation appears normal until hemorrhage occurs in yolk sacs and embryos. The primitive vascular plexus of the yolk sac fails to mature into defined vessels, and vascular channels dilate and rupture. Internal bleeding is seen in the peritoneal cavity, implying fragile vessels. Heart development is arrested at day 9.0, and the atrioventricular canal endocardium fails to undergo mesenchymal transformation and cushion-tissue formation. These data suggest that endoglin is critical for both angiogenesis and heart valve formation. Some heterozygotes, either with an inbred 123/Ola or mixed C57BL/6-129/Ola background, show signs of HHT, such as telangiectases or recurrent nosebleeds. In this murine model of HHT, it appears that epigenetic factors and modifier genes, some of which are present in 129/Ola, contribute to disease heterogeneity.", 
"L-Glutamic acid is a major excitatory neurotransmitter in the mammalian central nervous system. The termination of the glutamatergic transmission and the clearance of the excessive, neurotoxic concentrations of glutamate is ensured by a high affinity glutamate uptake system. Four homologous types of Na/K-dependent high affinity glutamate transporters, glutamate/aspartate transporter, glutamate transporter 1, excitatory amino acid carrier 1, and excitatory amino acid transporter 4, have recently been cloned and were assigned to a separate gene family, together with two neutral amino acid carriers, alanine/serine/cysteine transporter 1/serine/alanine/threonine transporter and adipocyte amino acid transporter. The genomic organization of these transporters is still under investigation. Very little is known about the nature of the factors and molecular mechanisms that regulate developmental, regional, and cell type-specific expression of the glutamate transporters and their aberrant functioning in neurodegenerative diseases (e.g., amyotrophic lateral sclerosis and Alzheimer's disease). Some experimental conditions (e.g., ischemia, corticostriatal lesions, hyperosmolarity, culturing conditions) and several naturally occurring and synthetic compounds (e.g., glutamate receptor agonists, dopamine, alpha(1)- and beta-adrenergic agonists, cAMP, phorbol esters, arachidonic acid, nitric oxide, oxygen free radicals, amyloid beta-peptide, tumor necrosis factor-alpha, glucocorticosteroids, unidentified neuronal factors) affect the molecular expression and activity of glutamate transporters. Further elucidation of the molecular events that link epigenetic signals with transcriptional and post-transcriptional mechanisms (e.g., alternative splicing, translation and post-translational modifications) is crucial for the development of selective pharmacological tools and strategies interfering with the expression of the individual glutamate transporters.", 
"Mutation of DNA mismatch repair genes has rarely been documented in sporadic gastric carcinoma with microsatellite instability (MSI), In sporadic colorectal carcinoma, hMLH1 promoter methylation associated with protein loss is found in the majority of high-frequency MSI cases. We investigated a series of 35 sporadic gastric carcinomas stratified into high-frequency MSI (MSI-H), low-frequency MSI (MSI-L) and microsatellite stable (MSS) groups and found that hypermethylation of the CPG island in the hMLH1 promoter region was present in 100% of MSI-H sporadic gastric carcinomas. In 90% of cases, there was an associated complete loss of hMLH1 protein, as detected by immunohistochemistry, and a markedly lowered hMLH1 mRNA level. This loss of hMLH1 protein occurred in the MSI-H invasive tumor but not in the adjacent carcinoma-irt situ or dysplastic components that were MSS. The MSI-L and MSS forms of gastric carcinoma all showed predominantly unmethylated hll;hMLH1 promoter, positive hMLH1 protein and high hMLH1 mRNA level. On the other hand, hMSH2 protein was expressed in all of the tumors irrespective of the MSI status. Our results suggest that high-frequency MSI in sporadic gastric cancer is mostly due to epigenetic inactivation of hMLH1 in association with promoter methylation, and the loss of hMLH1 protein is a significant event in the development of invasive tumor.", 
"Human gastric carcinoma shows a higher prevalence of microsatellite instability (MSI) than does any other type of sporadic human cancer. The reasons for this high Frequency of MSI are not get known. In contrast to endometrial and colorectal carcinoma, mutations of the DNA mismatch repair (MMR) genes hMLH1 or hMSH2 have not been described in gastric carcinoma. However, hypermethylation of the hMLH1 MMR gene promoter is quite common in MSI-positive endometrial and colorectal cancers. This hypermethylation has been associated with hMLH1 transcriptional blockade, which is reversible with demethylation, suggesting that an epigenetic mechanism underlies hMLH1 gene inactivation and MMR deficiency. Therefore, we studied the prevalence of hMLH1 promoter hypermethylation in a total of 65 gastric tumors: 18 with frequent MSI (MSI-H), 8 with infrequent MSI(MSI-L), and 39 that were MSI negative. We found a striking association between hMLH1 promoter hypermethylation and MSI: of 18 MSI-H tumors, 14 (77.8%) showed hg permethylation, whereas 6 of 8 MSI-L tumors (75%) were hypermethylated at hMLH1. In contrast, only; 1 of 39 (2.6%) MSI-negative tumors demonstrated hMLH1 hypermethylation (P < 0.0001 for MSI-H or MSI-L, versus MSI-negative). Moreover, hypermethylated cancers demonstrated diminished expression of hMLH1 protein by both immunohistochemistry and Western blotting, whereas nonhypermethylated tumors expressed abundant hMLH1 protein. These data indicate that hypermethylation of hMLH1 is strongly associated with MSI in gastric cancers and suggest an epigenetic mechanism by which defective MMR occurs in this group of cancers.", 
"Mutations of the BRCA1 gene in tumor DNA from patients with sporadic breast cancer have not yet been observed. Nevertheless, BRCA1 activity is markedly decreased in invasive breast tumors, Previous reports have shown that hypermethylation of the promoter region is an alternative mechanism to mutation for the inactivation of tumor suppressor genes. We examined the BRCA1 promoter region for hypermethylation by Southern blotting, Hypermethylation was observed in two of seven sporadic breast carcinomas but not in any normal tissues, The hypermethylation was not an artifact because a control region was unmethylated in the two tumors. Although not all tumors were hypermethylated, these observations are consistent with an important role for epigenetic mechanisms in human cancer.", 
"The p73 gene is located on 1p36.2-3, a region that is frequently deleted in human cancer. Because p73 encodes for a protein that is both structurally and functionally homologous to the p53 protein, p73 has been postulated to be a candidate tumor suppressor gene. To date, however, mutations of p73 have not been found. To study methylation of the p73 5'CpG island, a human bacterial artificial chromosome clone containing exon 1 and the 5' region of p73 was isolated. There was no evidence for p73 exon 1 methylation in normal tissues. In contrast, p73 was aberrantly methylated in approximately 30% of primary acute lymphoblastic leukemias (ALLs) and Burkitt's lymphomas. There was no evidence for methylation in any other types of hematological malignancies or solid tumors examined. In both leukemia cell lines and primary ALLs, methylation was associated with transcriptional loss of p73 by reverse transcription-PCR, mie used single-strand conformational polymorphisms to screen fur paint mutations in a series of primary ALLs and found no mutations leading to a change in protein structure. Our results show that methylation of p73 is a frequent event in specific types of hematological malignancies and suggest that epigenetic silencing of p73 could have important consequences for cell-cycle regulation.", 
" ", 
"Loss of imprinting (LOI) is an epigenetic alteration of some cancers involving loss of parental origin-specific expression of imprinted genes. We observed LOI of the insulin-like growth factor-ii gene in twelve of twenty-seven informative colorectal cancer patients (44%), as well as in the matched normal colonic mucosa of the patients with LOI in their cancers, and in peripheral blood samples of four patients. Ten of eleven cancers (91%) with microsatellite instability showed LOI, compared with only two of sixteen tumors (12%) without microsatellite instability (P< 0.001). Control patients without cancer showed LOI in colonic mucosa of only two of sixteen cases (12%, P < 0.001) and two of fifteen blood samples (13%, P < 0.001). These data suggest that LOI in tumor and normal tissue identifies most colorectal cancer patients with microsatellite instability in their tumors, and that LOI may identify an important subset of the population with cancer or at risk of developing cancer.", 
"Purpose: Patients treated with identical radiotherapy schedules show a substantial variation in the degree of acute and late normal tissue reactions. To identify any possible contributing factors to this phenomenon, we have analyzed the treatments of 402 breast cancer patients. Methods and Materials: The patients received adjuvant postoperative radiotherapy between 1972 and 1985 and have been followed up since then. Multivariate analyses were performed with peak reflectance erythema and peak acute reaction score as endpoints for the acute reactions, and with progression rate of telangiectasia as well as telangiectasia score as endpoints for the late reactions. Twenty patient- and treatment-related factors were tested such as age, menopausal status, hemoglobin level, serum calcium, smoking habits, hypothyroidism, diabetes, hypertension, blood pressure, cardiovascular and autoimmune disease, the influence of hormone therapy and chemotherapy, pretreatment reflectance value, acute skin reactions, radiation quality, individual dose, bilateral fields, and the total effect (TE) for the dose schedule applied. Results: The TE was a strong prognostic factor for all endpoints. In addition to TE, blood pressure was prognostic for the peak erythema measured by reflectance spectrophotometry, and the pretreatment reflectance value was prognostic for the acute score. The only independent prognostic factors found for the progression of skin telangiectasia and telangiectasia score except for TE were the individual dose and the acute skin reactions. Conclusions: These factors explained at most about 30% of the variance describing the total patient-to-patient variability for each endpoint. The remaining variability is still unexplained but may be related to individual differences in cellular radiosensitivity, partly determined by genetic variations and partly by unknown epigenetic factors. Copyright (C) 1996 Elsevier Science Inc.", 
"The early epidemiological data indicated different carcinogenic risks from inhalation of different nickel compounds, but it was not clear what characteristics governed the intrinsic carcinogenic hazard of the various nickel compounds. Based on the earlier results, all soluble and insoluble nickel compounds were assumed to have the same carcinogenic mechanism albeit different potencies, Recent in vivo and in vitro studies challenged this assumption. In this paper an attempt is made to integrate the most relevant human, animal, and in vitro data into a general model that can help understand the different carcinogenic potentials of the various nickel compounds. In this perspective, it is recognized that there are two main components that could contribute to the development of lung cancer via exposure to certain nickel compounds, The first component corresponds to the heritable changes (genetic or epigenetic) derived from the direct or indirect actions of nickel compounds, The second component may be the promotion of cell proliferation elicited by certain nickel compounds. The different contributions of three nickel compounds to these two components are presented. This paper emphasizes the importance of recognizing the individuality of the different nickel species in reaching regulatory decisions and the fact that different risk assessment considerations may apply for compounds that appear to produce immortality and cancer by genetic/epigenetic mechanisms (like nickel subsulfide), compounds that may present a threshold for the induction of tumors in rats (like high-temperature nickel oxide), or compounds that may only have an enhancing effect on carcinogenicity (like nickel sulfate). (C) 1997 Academic Press.", 
"Chronic exposure to high doses of phenobarbital (PB) causes hepatocellular adenomas in both mice and rats and hepatocellular carcinomas in some strains of mice, Long term PB therapy has not been found to cause human tumors, PB is not DNA reactive, and most genotoxicity tests have yielded negative results, PB has been extensively studied as an epigenetic, rodent liver tumor promoter, At exposures causing rodent liver tumors, PB has measurable effects on hepatocytes: PB inhibits cell-to-cell communication; PB induces enzymes, including P450 cytochromes; PB stimulates proliferation and inhibits apoptosis of hepatocytes in neoplastic foci, Threshold exposures for some of these endpoints coincide with the threshold exposure for tumorigenesis.", 
"T helper (Th) lymphocytes, when reactivated, recall expression of those cytokines they had been instructed to express in earlier activations, even in the absence of specific cytokine-inducing factors. In cells that memorize their expression, the cytokine genes are modified by chromatin rearrangement and demethylation, suggesting that they have been somatically imprinted. Here we show, by using inhibitors blocking the cell cycle in various stages, that for the instruction of a Th cell to express interleukin (IL)-4 or IL-10 upon restimulation, entry of the cell into the S phase of the first cell cycle after initial activation is required. Separation of the IL-4 receptor (IL-4R) and T cell antigen receptor (TCR) signals in time, demonstrates that this instruction is dependent on concomitant signaling from both receptors. In Th cells, inhibited to progress into the first S phase after activation, the IL-4R and TCR signals can be memorized for at least 1 d, priming the T cell, to become instructed for expression of IL-4 upon restimulation, when entering the S phase after release of the cell cycle block. The requirement of the initial S phase of T cell activation, for instruction of Th cells to express IL-4 or IL-10 upon restimulation points to the decisive role of epigenetic modification of cytokine genes as a molecular correlate of the memory to express particular cytokines.", 
"We have developed a quantitative method for methylation analysis of the p16 gene based on real-time methylation-specific PCR (MSP). Realtime MSP is sensitive enough to detect down to 10 genome equivalents of the methylated p16 sequence. Application of real-time MSP to DNA from tumor-derived cell Lines revealed complete concordance with conventional MSP analysis. Quantitative data generated by real-time MSP were expressed as the methylation index, which was defined as the percentage of bisulfite-converted DNA that consisted of methylated target sequences. The methylation index was shown to be inversely correlated with p16 gene transcription during demethylation treatment of cell lines with 5-aza-2'-deoxycytidine. The application of real-time MSP to bone marrow aspirates from patients with multiple myeloma revealed complete concordance with conventional MSP analysis. Real-time quantitative MSP may have applications in elucidating diverse biological processes involving DNA methylation and mag become a valuable diagnostic tool for detecting tumor-associated epigenetic changes in cancer patients.", 
"Hypermethylation of tumor suppressor genes and other genes functionally important in the neoplastic process is a recently recognized process. This epigenetic process is characterized by loss of function of these genes associated with transcriptional loss in the absence of structural alterations. The growing list of genes inactivated by promoter region hypermethylation provide an opportunity to examine the patterns of inactivation of such genes and provide clues as to the roles these events play in the neoplastic process.", 
"To determine whether neuroblastomas acquire a sustained drug-resistant phenotype from exposure to chemotherapeutic agents given to patients in vivo, we studied neuroblastoma cell lines established at different points of therapy: six at diagnosis before therapy (DX), six at progressive disease during induction therapy (PD-Ind), and five at relapse after intensive chemoradiotherapy and bone marrow transplantation (PD-BMT). Cells were maintained in the absence of drug selective pressure. Dose-response curves of melphalan, cisplatin, carboplatin, doxorubicin, and etoposide for the cell line panel were determined by measuring cytotoxicity with a 96-well-plate digital imaging microscopy (DIMSCAN) microassay. Drug resistance of cell lines progressively increased with the intensity of therapy delivered in vivo. The greatest resistance was seen in PD-BMT cell lines: IC90 values in PD-BMT cell lines were higher than clinically achievable drug levels by 1-37 times for melphalan, 1-9 times for carboplatin, 25-78 times for cisplatin, 6-719 times for doxorubicin, and 3-52 times for etoposide. Genomic amplification of MYCN did not correlate with resistance. Cross-resistance by Pearson correlation (r greater than or equal to 0.6) was observed between: (a) cisplatin + doxorubicin; (b) carboplatin + cisplatin, etoposide, or melphalan; (c) etoposide + cisplatin, melphalan, or doxorubicin. These data indicate that during therapy, neuroblastomas can acquire resistance to cytotoxic drugs because of the population expansion of tumor cells possessing stable genetic or epigenetic alterations that confer resistance.", 
"The mouse skin model of chemical carcinogenesis has been very well characterized with respect to epigenetic changes, which occur during tumour cell initiation, promotion and progression. The use of transgenic and gene knock-out mice has contributed greatly to knowledge in this area. The H-ras genetic locus has been shown to undergo multiple genetic changes, including mutagenic activation, amplification of the mutant gene, and loss of the normal allele, These different genetic events lead to thresholds of ras activity which contribute to different stages along the pathway to neoplasia. The genetic and epigenetic events which lead to tumour invasion and metastasis hale been less well characterized than studies on tumour initiation and promotion, despite the fact that it is metastases which ultimately kill the animal/patient. In the mouse skin model, loss of p53 contributes to malignant conversion. Gene deletion of the INK4 locus is associated with transformation to a highly invasive spindle cell tumor phenotype, This spindle cell transformation can also be induced in vitro or in vivo by TGF beta 1, possible by synergizing with mutant H-ras. TGF beta can have both positive and negative effects on tumourigenesis, acting early as a tumour suppresser, but later as a stimulator of tumour invasion. It is this latter effect which may be clinically more significant, since many human tumours overexpress TGF beta, yet the majority still retain the intracellular signaling systems necessary for the cell to respond to this growth factor. Copyright (C) 1999 John Wiley & Sons, Ltd.", 
"Aberrant 5' CpG island methylation is an alternative mechanism of gene inactivation during the development of cancer as demonstrated for several tumor-suppressor genes. Also, marked relationship of microsatellite instability (MSI) and DNA methylation has been reported in sporadic colorectal cancer, which is a result of epigenetic inactivation of hMLHI in association of promoter hypermethylation, In the present study, we investigated the 5' CpG island hypermethylation of hMLHI, E-cadherin and p16 in 61 primary gastric cancers (GCs) by using combined bisulfite restriction analysis (COBRA) and methylation-specific FCR (MSP), and their MSI status. Of 61 GCs investigated, 5 (8.1%) tumors presented hMLHI methylation, 16 (26.2%) and 25 (40.9%) showed E-cadherin and p16 methylation respectively, and 8 (13.1%) presented high-frequency MSI (MSI-H). Of the a MSI-H patients, 5 presented hMLHI methylation, whereas no low-frequency MSI (MSI-L) and microsatellite stable (MSS) cases exhibited hMLHI methylation (5/8 vs. 0/43, p<0.00001). Furthermore, these patients also presented E-cadherin and p16 hypermethylation. Our data showed a significant correlation between hMLHI methylation and MSI in GC, and suggested that a common mechanism of aberrant de novo methylation can be postulated in these cancers. (C) 1999 Wiley-Liss, Inc.", 
"The cyclin-dependent kinase inhibitor p16 (p16(INK4A)/CDKN2/MTS1) is a potent inhibitor of the cyclin D-dependent phosphorylation of the retinoblastoma gene (Rb) product, the inactivation of which induces loss of Rb-dependent G1 arrest through inappropriate phosphorylation of the Rb protein. To analyse the role of po16(INK4A) as a tumor suppressor in the genesis of non small cell lung cancers (NSCLC) and correlate loss of p16(INK4A) protein expression to genetic or epigenetic mechanisms, we have performed a comprehensive study of p16 status in a series of 43 NSCLC. To this end, we have investigated p16(INK4A) protein expression with immunohistochemistry, deletions of the gene by FISH, and determined the methylation status of exon 1 alpha using a PCR-based methylation assay. Finally, possible mutations were studied by SSCP and subsequent sequencing, Twenty one of the 43 (49%) NSCLC studied exhibited an absence of p16(INK4A) nuclear staining. Of these, three (14%) had frameshift or missense mutations, seven (33%) displayed methylation of exon1 alpha and 10 (48%) displayed homozygous deletions, In total, 95% of the tumors with p16(INK4A) negative staining carried one of these three alternative genetic or epigenetic alterations. Furthermore, a high degree of chromosome 9 polysomy was found (58%) in those tumors with p16(INK4A) inactivation. Taken together these results suggest that deregulation of the p16 gene locus is a frequently occurring event in NSCLC through distinct mechanisms including rare point mutations, promotor methylation and frequent homozygous deletions. Furthermore, our data show that immunohistochemistry is a rapid and an accurate technique for screening of p16(INK4A) gene inactivation events that result in loss of protein expression.", 
"Inactivation of p16(INK4) tumor suppressor gene function is frequently observed in breast cancer. We examined p16(INK4) expression in human mammary epithelial cell (HMEC) cultures established from four normal donors. Normal HMECs divide a limited number of times before proliferation ceases in a state referred to as selection (or M-o). The cell subpopulation that emerges spontaneously from selection undergoes a further limited period of proliferation before senescence. By immunofluorescence and Western blot analysis of four independent cultures, me have shown Loss of p16(INK4) expression in postselection HMECs. In contrast, p16(INK4) was present in both early and late passage fibroblasts from the same individuals. Bisulfite genomic sequencing revealed extensive methylation of the p16(INK4) CPG island in post- but not preselection cells. Thus, the extended period of growth observed in postselection HMECs is associated with hypermethylation of the p16(INK4) CpG island and Loss of p16(INK4) expression. Although postselection HMECs are widely considered to be normal, these data indicate that they have sustained an epigenetic alteration.", 
"Chromatin ir the natural substrate for the control of gene expression. Chromatin contains DNA, the transcriptional machinery and structural proteins such as histones. Recent advances demonstrate that transcriotional activity of a gene Is Inr largely controlled by the packaging of the template within chromatin. The covalent modification of chromatin provides an attractive mechanism for establishing and maintaining stable states of gene activity. DNA methylation and histone acetylation alter the nucleosomal infrastructure to repress or activate transcription. These covalent modifications have causal roles in both promoter-specific events and the global control of chromosomal activity. DNA methylation and histone acetylation have a major impact in both oncogenic transformation and normal development.", 
"Transforming Growth Factor-beta (TGF beta) is the most potent known inhibitor of the progression of normal mammary epithelial cells through the cell cycle. During the early stages of breast cancer development, the transformed epithelial cells appear to still be sensitive to TGF beta-mediated growth arrest, and TGF beta can act as an anti-tumor promoter. In contrast, advanced breast cancers are mostly refractory to TGF beta-mediated growth inhibition and produce large amounts of TGF beta, which may enhance tumor cell invasion and metastasis by its effects on extracellular matrix. We postulate that this seemingly paradoxical switch in the responsiveness of tumor cells to TGF beta during progression is the consequence of the activation of the latent TGF beta that is produced and deposited into the tumor microenvironment, thereby driving the clonal expansion of TGF beta-resistant tumor cells. While tumor cells themselves may activate TGF beta, recent observations suggest that environmental tumor promoters or carcinogens, such as ionizing radiation, can cause stromal fibroblasts to activate TGF beta by epigenetic mechanisms. As the biological effects of the anti-estrogen tamoxifen may well be mediated by TGF beta, this model has a number of important implications for the clinical uses of tamoxifen in the prevention and treatment of breast cancer. In addition, it suggests a number of novel approaches to the treatment of advanced breast cancer.", 
"The major cause of death from cancer is metastases that are resistant to conventional therapies. Several reasons account for treatment failure in patients with metastases. First, neoplasms are biologically heterogeneous and contain subpopulations of cells with different angiogenic, invasive, and metastatic properties. Second, the process of metastasis selects a small subpopulation of cells that preexist within a parental neoplasm. Although metastases can have a clonal origin, genetic instability results in rapid biological diversification and the regeneration of heterogeneous subpopulations of cells. Third, and perhaps the most important principle for the design of new cancer therapies, is that the outcome of metastasis depends on multiple interactions (cross-talk) of metastatic cells with homeostatic mechanisms which the tumor cells usurp. The organ microenvironment can influence the biology of cancer growth, angiogenesis, and metastasis in several different ways. For example, the survival and growth of tumor cells are dependent on angiogenesis, which is mediated by an imbalance between positive and negative regulating molecules released by tumor cells, normal cells surrounding a tumor, and infiltrating lymphoid cells. Many cytokines that stimulate or inhibit angiogenesis are present in different tissues, and thus the organ environment profoundly influences this process. Moreover, the organ microenvironment can also influence the response of metastases to chemotherapy by regulating the expression of different drug resistance genes, such as mdr-1. The finding that the resistance of metastases to some chemotherapeutic agents can be mediated by epigenetic mechanisms has obvious implications for therapy. The identification of organ-specific cytokines that can upregulate expression of mdr-1 (or other resistant mechanisms) may suggest an approach to overcome the resistance of some metastases to particular chemotherapeutic agents. Therefore therapy of metastasis should be targeted not only against metastatic tumor cells, but also the homeostatic factors that are favorable to metastasis, growth, and survival of the metastatic cells.", 
"Our laboratory has found genomic imprinting of a large genomic domain of human 11p15.5, identifying six imprinted genes within this domain: (a) insulin-like growth factor II (IGF-II), an important autocrine growth factor in a wide variety of malignancies; (b) H19, an untranslated RNA that is a putative growth suppressor gene regulating IGF-II; (c) p57(KIP2), cyclin-dependent kinase inhibitor that causes G(1)-S arrest; (d) K(V)LQT1, a voltage-gated potassium channel; (e) TSSC3, a gene that is homologous to mouse TDAG51, which is implicated in Fas-mediated apoptosis; and (f) TSSC5, a putative transmembrane protein-encoding gene, We hypothesize that 11p15 harbors a large domain of imprinted growth- regulatory genes that are important in cancer, Several lines of evidence support this hypothesis: (a) we have discovered a novel genetic alteration in cancer, loss of imprinting, which affects several of these genes, and is one of the most common genetic changes in human cancer; (b) we have found that the hereditary disorder Beckwith-Wiedemann syndrome, which predisposes to cancer and causes prenatal overgrowth, involves alterations in p57(KIP2), IGF-II, H19, and K(V)LQT1; (c) we have found both genetic (somatic mutation in Wilms' tumor) and epigenetic alterations (DNA methylation) in cancer; and (d) we can partially reverse abnormal imprinting using an inhibitor of DNA methylation, We propose a model of genomic imprinting as a dynamic developmental process involving a chromosomal domain. According to this model, cancer involves both genetic and epigenetic mechanisms affecting this imprinted domain and the genes within it.", 
"Keratin 8 (K8) and keratin 18 (K18) are the most common and characteristic members of the large intermediate filament gene family expressed in 'simple' or single layer epithelial tissues of the body. Their persistent expression in tumor cells derived from these epithelia has led to the wide spread use of keratin monoclonal antibodies as aids in the detection and identification of carcinomas. Oncogenes which activate ras signal transduction pathways stimulate expression of the K18 gene through transcription factors including members of the AP-1 (jun and fos) and ETS families. The persistent expression of K8 and K18 may reflect the integrated transcriptional activation of such transcription factors and, in the cases of ectopic expression, an escape from the suppressive epigenetic mechanisms of DNA methylation and chromatin condensation. Comparison of the mechanisms of transcriptional control of K18 expression with expression patterns documented in both normal and pathological conditions leads to the proposal that persistent K8 and K18 expression is a reflection of the action of multiple different oncogenes converging on the nucleus through a limited number of transcription factors to then influence the expression of a large number of genes including these keratins. Furthermore, correlation of various tumor cell characteristics including invasive behavior and drug sensitivity with K8 and K18 expression has stimulated consideration of the possible functions of these proteins in both normal development and in tumorigenesis. Recent developments in the analysis of the functions of these intermediate filament proteins provide new insights into diverse functions influenced by Kg and K18.", 
"The cell cycle regulatory genes p16/CDKN2 RE are frequently deleted in prostate cancers. In this study, We examined the role of alterations in p16 and pRb during growth, senescence, and immortalization in vitro of human prostate epithelial cells (HPECs), HPECs are established from normal prostate tissues and cultured on collagen-coated dishes, Our results show that p16 is reproducibly elevated at senescence in HPECs, HPECs are immortalized using human papilloma virus 16 E6 and/or E7 as molecular tools to inactivate p53 and/or pRb, respectively. Immortalization occurs infrequently in this system and only after a latent period during which additional genetic/epigenetic changes are thought to occur. Notably;, all of the E6-immortalized HPEC lines but none of the E7 lines show, inactivation oof p16/CDKN2 by deletion, methylation, or mutation in association with immortalization. In contrast, E7 lines, in which pRb function is abrogated bg E7 binding, retain the high levels of p16 observed at senescence. Thus, all lines show either a p16 or pRb inactivation. Analysis of six independent lines from metastatic prostate cancers reveals a similar loss of either p16 or pRb, Comparative genomic hybridization of HPECs shows that gains of chromosomes 5q, so, and 20 are nonrandomly associated with; bSpassing senescence (probability = 0.95). These results suggest that high levels of the cyclin-dependent kinase inhibitor p16 mediate senescence G(1), arrest in HPECs and that bypassing this block by a p16/pRb pathway alteration is required for immortalization in vitro and possibly tumorigenesis in vivo. Our results further indicate that inactivation of the p16/pRb pathway alone is not sufficient to immortalize HPECs and that additional genetic alterations are required for this process.", 
"Expression of the invasion/metastasis suppressor, E-cadherin, is diminished or lost in thyroid carcinomas. Yet, mutational inactivation of E-cadherin is rare. Herein, we show that this loss is associated with hypermethylation of the E-cadherin 5' CpG island in a panel of human thyroid cancer cell lines. This aberrant methylation is evident in 83% of papillary thyroid carcinoma, 11% of follicular thyroid carcinoma, 40% of Hurthle's cell carcinoma, and 21% of poorly differentiated thyroid carcinomas. Contrary to previous reports, the majority of these poorly differentiated thyroid carcinomas express E-cadherin, but often within the cytoplasm rather than at the cell surface. Together, our data indicate that the invasion/metastasis suppressor function of E-cadherin is frequently compromised in human papillary, Hurthle's cell, and pearly differentiated thyroid carcinoma by epigenetic and biochemical events.", 
"The development of a malignant tumor involves the progressive acquisition of mutations and epigenetic abnormalities in multiple genes that have highly diverse functions, Some of these genes code for pathways of signal transduction that mediate the action of growth factors, The enzyme protein kinase C plays an important role in these events and in the process of tumor promotion, Therefore, we examined the effects of three inhibitors of protein kinase C, CGP 41251, RO 31-8220, and calphostin C, on human glioblastoma cells, These compounds inhibited growth and induced apoptosis; these activities were associated with a decrease in the level of CDC2 and cyclin B1/CDC2-associated kinase activity, This may explain why the treated cells accumulated in G(2)-M. In a separate series of studies, we examined abnormalities in cell cycle control genes in human cancer, We have found that cyclin D1 is frequently overexpressed in a variety of human cancers, Mechanistic studies indicate that cyclin D1 can play a critical role in carcinogenesis because: overexpression enhances cell transformation and tumorigenesis; introduction of an antisense cyclin D1 cDNA into either human esophageal or colon cancer cells reverts their malignant phenotype; and overexpression of cyclin D1 can enhance the amplification of other genes, The latter finding suggests that cyclin D1 can enhance genomic instability and, thereby, the process of tumor progression, Therefore, inhibitors of the function of cyclin D1 may be useful in both cancer chemoprevention and therapy, We obtained evidence for the existence of homeostatic feedback loops between cyclins D1 or E and the cell cycle inhibitory protein p27(Kip1). On the basis of these and other findings, we hypothesize that, because of their disordered circuitry, cancer cells suffer from gene addiction and gene hypersensitivity, disorders that might be exploited in both cancer prevention and therapy.", 
"Spontaneous regression of cancer is reported in virtually all types of human cancer, although the greatest number of cases are reported in patients with neuroblastoma, renal cell carcinoma malignant melanoma and lymphomas/leukemias. Study of patients with these diseases has provided most of the data regarding mechanisms of spontaneous regression. Mechanisms proposed for spontaneous regression of human cancer include: immune mediation, tumor inhibition by growth factors and/or cytokines, induction of differentiation, hormonal mediation, elimination of a carcinogen, tumor necrosis and/or angiogenesis inhibition psychologic factors, apoptosis and epigenetic mechanisms. Clinical observations and laboratory studies support these concepts to a variable extent. The induction of spontaneous regression may involve multiple mechanisms in some cases although the end result is likely to be either differentiation or cell death. Elucidation of the process of spontaneous regression offers the possibility of improved methods of treating and preventing cancer.", 
"Recent evidence suggests that resistance to antineoplastic therapy may result from mutations in genes mediating the apoptotic response to DNA damage, To determine the effects of epigenetic changes on tumor responsiveness to cytotoxic agents inducing DNA damage, we examined the chemosensitivity of neuroblastoma (NB) after differentiation by retinoic acid (RA). Differentiation of the cell lines SH-SY5Y and SMS-KCNR by RA abolished the cytotoxic effects of adriamycin (Adr) and cisplatin. Chemoresistance was not the result of decreased proliferation induced by RA because: (a) growth arrest by nutrient deprivation did not affect sensitivity; (b) growth arrested NE cell lines, which did not differentiate, remained chemosensitive; and (c) RA concentrations which promoted differentiation without affecting growth, induced resistance, Apoptosis characterized NE cells responding to Adr, although differentiated SH-SY5Y did not apoptose and were resistant to Adr and cisplatin. Marked induction of bcl-2 in NE cells followed RA-induced differentiation, whereas in cell lines failing to differentiate, bcl-2 was not detected, Our data indicate that NE differentiation induces drug resistance after a loss of the apoptotic response to antineoplastic drugs and suggest that bcl-2 overexpression is an important mechanism of resistance in differentiated tumor cells.", 
"The few imprinted genes characterized so far include the insulin-like growth factor-2 gene (IGF2) coding for a foetal growth factor and the H19 gene whose normal function is unknown but which is likely to act as an RNA with an antitumour effect, IGF2 is expressed by the paternal allele and H19 by the maternal allele. This reciprocal expression is quite interesting because both H19 and IGF2 genes are located close to each other on chromosome 11p15.5 in a region subject to loss of heterozygosity (LOH), Moreover, loss of imprinting (LOI) or biallelic expression has been proposed as an epigenetic mechanism for tumorigenesis in a variety of human cancers including Wilms' tumour, In this study we report the LOH, LOI and methylation status of H19 and IGF2 genes in 29 invasive cervical carcinomas of different clinical stages, Fourteen (48%) and 13 (45%) tumours were heterozygous for H19 and IGF2 respectively, LOH for H19 and IGF2 genes were found in 2 of 14 (14%) and 3 of 13 (23%) informative tumours, respectively, LOI of H19 and IGF2 was detected in 2 of 12 (17%) and 5 of 10 (50%) tumours with no LOH, respectively, More interestingly, monoallelic expression of the otherwise silent H19 allele (allele switch) was observed in 2 of 12 (17%) tumours and biallelic expression of IGF2 was detected in one specimen of normal cervix adjacent to the tumour, The expressing H19 allele, and to a lower degree also the silent allele, were hypomethylated in tumours suggesting that demethylation of both H19 alleles may be associated with an early step of imprinting alteration. In cervical cancer H19 and IGF2 expressions could be independently regulated, In conclusion, our data suggest that H19 and IGF2 genes, via deletions and/or abnormal imprinting, could play a crucial role in a large proportion (58%) of cervical cancers where they may be associated with disease progression.", 
"Background: The identification in 1994 of the CDKN2 gene as a target for mutations in a wide range of human cancers, including malignant mesothelioma, has been controversial because subsequent studies have detected a lower frequency of CDKN2 gene mutations in primary tumors than in cultured cell lines, These reports raised the hypothesis that another gene, distinct from CDKN2, might be the target of the chromosome 9p21 deletions frequently observed in these tumors, Purpose: To address whether inactivation of CDKN2 function is an essential event in the etiology of malignant mesothelioma, we examined p16(INK4) protein expression in primary thoracic mesotheliomas, in nonmalignant pleural tissues, and in independent mesothelioma cell lines, We also studied the growth rate of tumor cell lines following stable transfection of CDKN2 gene, Methods: Retinoblastoma (Rb) and p16(INK4) protein expression was determined by immunohistochemical analysis from archival paraffin specimens of 12 primary thoracic mesotheliomas and a nonmalignant pleural biopsy specimen, In addition, protein immunoblot analysis for Rb and p16(INK4) expression was conducted on 15 independent mesothelioma cell lines, and the ability of a transfected CDKN2 gene to suppress the growth of the mesothelioma cell lines H2373 and H2461 in vitro was examined, Results: We demonstrated abnormal p16(INK4) expression in 12 of 12 primary mesothelioma specimens and in 15 of 15 mesothelioma cell lines, All tumor specimens and the tumor cell lines showed expression of wild-type Rb protein, In addition, we have confirmed the ability of a transfected CDKN2 gene to suppress growth of two independent mesothelioma cell lines, Conclusions: Immunohistochemical analysis of the p16(INK4) gene product is feasible in archival biopsy samples, With this analysis, CDKN2 gene inactivation can be determined in tumors that are contaminated with nonmalignant cells, Furthermore, since loss of p16(INK4) protein expression can result from both genetic (gene mutations) and epigenetic (abnormal DNA hypermethylation) mechanisms, as we and others have shown recently, examination of protein expression is a highly sensitive method for analyzing the CDKN2 status in large numbers of tumor samples, Implications: This study suggests that inactivation of the CDKN2 gene is an essential step in the etiology of malignant mesotheliomas, Defining the role of the p16(INK4):Rb tumor suppressor pathway and its immediate downstream substrates will be an important goal in designing future therapeutic strategies.", 
"The yeast Saccharomyces cerevisiae has a limited life span that can be measured by the number of times individual cells divide. Several genetic manipulations have been shown to prolong the yeast life span. However, environmental effects that extend longevity have been largely ignored. We have found that mild, nonlethal heat stress extended yeast life span when it was administered transiently early in life. The increased longevity was due to a reduction in the mortality rate that persisted over many cell divisions (generations) but was not permanent. The genes RAS1 and RAS2 were necessary to observe this effect of heat stress. The RAS2 gene is consistently required for maintenance of life span when heat stress is chronic or in its extension when heat stress is transient or absent altogether. RAS1, on the other hand, appears to have a role in signaling life extension induced by transient, mild heat stress, which is distinct from its life-span-curtailing effect in the absence of stress and its lack of involvement in the response to chronic heat stress. This distinction between the RAS genes may be partially related to their different effects on growth-promoting genes and stress-responsive genes. The ras2 mutation clearly hindered resumption of growth and recovery from stress, while the ras1 mutation did not. The HSP104 gene, which is largely responsible for induced thermotolerance in yeast, was necessary for life extension induced by transient heat stress. An interaction between mitochondrial petite mutations and heat stress was found, suggesting that mitochondria may be necessary for life extension by transient heat stress. The results raise the possibility that the RAS genes and mitochondria may play a role in the epigenetic inheritance of reduced mortality rate afforded by transient, mild heat stress. (C) 1998 Academic Press.", 
"We previously reported 9 unilateral, sporadic retinoblastomas with hypermethylation in the 5' region of the RE gene, and we found that CpG methylation in the RE promoter inhibits the binding of the retinoblastoma binding factor I (RBF-1) and the activating transcription factor (ATF)-like factors, thereby resulting in a considerable reduction in RE promoter activity. In this study, we screened for hypermethylation in 121 additional cases of retinoblastoma, and found 5 tumors with hypermethylation, including 4 unilateral, sporadic tumors, and one hereditary tumor. The hereditary tumor had a germline deletion of one allele, and the hypermethylation was an acquired, epigenetic change in the other allele. Another tumor had hypermethylation restricted to approximately 800 base pairs in the RS promoter region including the essential RBF-1 and A TF sites. The frequency of hypermethylation in unilateral, sporadic tumors was 9.3% combining our previous and present examinations (13 among 140), whereas the frequency was 1.0% in bilateral hereditary tumors (one among 101). The statistical analyses using the chi-square test indicated significant correlation between hypermethylation and unilateral, sporadic tumors (p < 0.05). These results suggest that hypermethylation in the RE gene is always an acquired, epigenetic change and causes about 9% of unilateral, sporadic tumors. (C) Elsevier Science Inc., 1997.", 
"To investigate the relationship between aberrant crypt foci (ACF) and colon neoplasia in colorectal carcinogenesis, we evaluated 433 ACF, which were collected from the grossly normal mucosa of surgical specimens from 57 patients with colorectal cancer. The ACF ranged in size from 3 to 412 aberrant crypts/focus. Large ACF (greater than or equal to 50 crypts/focus) comprised 25% of the total ACF studied. Histopathologically, 65% (67 of 103) of large ACF were diagnosed as hyperplasia, 10% (10 of 103) as adenoma, and 1% (1 of 103) as within normal colorectal mucosa. The remaining 24% (25 of 103) were diagnosed as ''stage I abnormality crypts,'' which were characterized by their extension of the proliferative compartment to the surface of crypts but with no changes in the major site of proliferation, as designated by E. E. Deschner [Pathol, Annu., 18 (Part 1): 205-219, 1983]. Of the 25 stage I abnormality ACF, 7 ACF coexisted with hyperplasia. Of 10 adenomatous ACF, two coexisted with stage I abnormality crypts, A K-ras codon 12 mutation was identified in 85% (93 of 109) of large ACF. The proliferative activity of stage I crypts was significantly higher than that of hyperplastic crypts in the same ACF. These observations suggest that some hyperplastic ACF may develop into adenomatous ACF by way of stage I abnormality ACF with concomitant acquisition of higher proliferative activity through some genetic and/or epigenetic changes.", 
"Several lines of evidence from our laboratory and others indicate that epigenetic alterations in protein kinase C (PKC) are involved in colonic carcinogenesis in both man and experimental animals. Furthermore, bile salts, known activators of PKC, have also been implicated in colonic tumor development. Recently, however, our laboratory has demonstrated that, whereas dietary cholic acid increased the occurrence of azoxymethane (AOM)-induced rat colonic tumors, ursodeoxycholic acid was associated with a significant protective effect. In the present studies, we therefore examined changes in PKC isoforms that accompanied AOM-induced tumor formation and investigated whether the chemopromotional and/or chemopreventional actions of these supplemental dietary bile salts involved changes in specific isoforms of PKC. Rats treated with vehicle (saline) or AOM and maintained on bile salt unsupplemented or supplemented diets were used to isolate control colonocytes and carcinogen-induced tumors, which were then subjected to subcellular fractionation. The homogenates and subcellular fractions mere then probed for individual PKC isoforms by quantitative Western blotting using isoform-specific antibodies. Normal rat colonocytes expressed PKC-alpha, -beta(II), -delta, -epsilon, and -zeta. AOM, in unsupplemented or cholate-supplemented groups, caused significant down-regulation of PKC-alpha, -delta and -zeta and up-regulation of PKC-beta(II) while increasing particulate PKC-alpha, -beta(II), and -zeta in carcinogen-induced tumors compared to normal colonocytes. Dietary supplementation with ursodeoxychohc acid, in marked contrast to these groups, prevented the changes in the subcellular distributions of PKC-alpha, -beta(II), and -zeta, and preserved the expression of PKC-zeta in AOM-induced tumors. These studies suggest that changes in specific isoforms of PKC (particularly, PCK-alpha, -beta(II), -delta, and/or -zeta) are involved in colonic malignant transformation in the AOM model but do not account for the chemopromotional actions of cholic acid in this model. furthermore, the ability of ursodeoxycholic acid to block AOM-induced increases in particulate PKC-alpha, -beta(II), and -zeta, and/or inhibit down-regulation of PKC-zeta, may contribute to the chemopreventive effects of this bile acid.", 
"The developmentally regulated H19 gene displays several remarkable properties: expression of an apparently non-translated mRNA, genomic imprinting (maternal allele only expressed), relaxation of the imprinting and/or epigenetic lesions demonstrated in some tumors. Despite several observations after relaxation of imprinting status of the gene, data on tr ans and cis-acting factors required for the human H19 gene expression are still missing. As a first approach to address identification of factors involved in the regulation of the gene, we found that cells from a p53 antisense-transfected HeLa clone displayed increased amounts of H19 transcripts when compared to the non-transfected cells. Moreover, a HeLa clone stably transfected with a temperature sensitive (ts) 143 Ala p53 mutant exhibited temperature-dependent regulation of H19 expression. This preliminary indication of the repressing effect of the p53 protein on H19 expression has been confirmed by transient cotransfection experiments in HeLa cells, using luciferase surrogate constructs under the control of the 823 bp sequence immediately upstream of the transcription start point of the H19 gene, and different constructs containing sense, antisense or a ts 143 Ala mutant p53 cDNA. We observed an increase of H19 promoter-driven activity in transient cotransfections with the antisense p53 cDNA and the temperature sensitive mutant p53 at the non-permissive temperature, but a decrease with sense wild-type p53 cDNA. Furthermore, the cotransfection experiments were repeated in a cell line lacking endogenous p53, (Calu 6 cells) and the results provided additional evidence for a down regulation of the expression of the H19 gene by the p53 protein.", 
"Gelsolin is a multifunctional, actin-binding protein that is greatly decreased in many transformed cell lines and tumor tissues, including breast cancers, Downregulation of gelsolin RNA occurs in most breast cancers of rats, mice, and humans, but gross mutations of the gelsolin gene have not been found. Here we demonstrate by PCR and RT-PCR analysis that there are no point mutations in putative regulatory regions or the entire coding region of the cytoplasmic isoform of the gelsolin gene in human breast cancer cells (BCC). To determine if epigenetic modification is involved in downregulating gelsolin expression in MDA-MB-231 (MDA231), MCF7, and T47D BCC, we have used Southern blot analysis, 5-azacytidine (5aza) treatment, and trichostatin A (TSA) treatment. Southern blot analysis performed on genomic DNA demonstrated altered CpG methylation within intron 1 in DNA from all BCC compared to normal, mortal human mammary epithelial cells (HMEC). Treatment of the BCC with 5aza converted the DNA restriction pattern to that seen in untreated HMEC genomic DNA and caused modest increases in gelsolin RNA and protein. Incubation with TSA, an inhibitor of histone deacetylase, induced a dramatic upregulation of gelsolin RNA and protein levels which preceded apoptotic death of all BCC within 48-60 h. Our data support a role for epigenetic changes in chromatin structure leading to downregulation of gelsolin expression in human breast cancer, To our knowledge, this is the first example of a tumor suppressor gene downregulated in human breast cancer by changes in histone acetylation. (C) 1999 Academic Press.", 
"Genomic DNA methylation patterns influence the development and maintenance of function during cellular differentiation. Methylation of regulatory sequences can have long-lasting effects on gene expression if inherited in an epigenetic manner. Recent work suggests that DNA methylation has a regulatory role in differential cytokine gene expression in primary T lymphocytes, Here we show, by clonal Lineage analysis, that methylation patterns in the IFN-gamma promoter exhibit long term faithful inheritance in CD44(high)CD8(+) T cells and their progeny, through 16 cell divisions and a clonal expansion of 5 orders of magnitude. Moreover, the demethylated IFN-gamma promoter is faithfully inherited following the withdrawal of T cell stimulation and the loss of detectable IFN-gamma mRNA, consistent with passive rather than active maintenance mechanisms. This represents a form of stable cellular memory, of defined epigenetic characteristics, that may contribute to the maintenance of T cell cytokine expression patterns and T cell memory.", 
"The presence of ETV6 deletions wats investigated in 215 children with acute lymphoblastic leukemia (ALL) using the loss of heterozygosity (LOH) approach. We used four intragenic or juxtagenic microsatellite markers to detect allelic deletions. In this series of unselected patients, LOH of ETV6 markers was found in 23% of cases (6% of T-ALL and 26% of B lineage ALL) confirming that chromosome 12p12-13 deletions represent a major genetic alteration in childhood ALL, frequently missed by cytogenetic analysis. The presence of a t(12;21)(p13;q22) was studied by RT-PCR and/or FISH in a total of 134 patients (125 B lineage ALL, nine T-ALL) including 42 cases with LOH. Thirty-four out of 44 patients (77%) for whom a t(12;21) was observed displayed LOH of the ETV6 markers. When associated with a t(12;21), ETV6 is very likely to be the target of deletions as indicated by the detection of intragenic deletions in three patients. Although deletion of ETV6 and t(12;21) were associated in most patients, in eight cases (six B lineage and two I-ALL) LOH was detected at the ETV6 locus without ETV6-AML1 hybrid RNA. FISH studies conducted in five of these eight patients confirmed the absence of translocation involving ETV6. In such patients, the other allele of ETV6 could be disrupted by either a small deletion, a point mutation, or an epigenetic modification and it will be of interest to study the! structure and expression of the remaining allele of ETV6 in these cases. Alternatively, a tumor suppressor gene located close to ETV6 and CDKN1B could be the target of deletions.", 
"The tumor suppressor genep53 encodes a nuclear phosphoprotein which is critical for cell cycle control and prevention of uncontrolled cell proliferation that can lead to cancer. Previous studies have shown that cells respond to DNA damage by increasing their levels of p53, which then acts to prevent replication of damaged DNA, This study examined the effects on p53 protein levels of several different categories of chemical carcinogens. N-Methyl-N' -nitro-nitrosoguanidine and N-ethyl-N-nitrosourea, two direct-acting genotoxic (DNA-reactive) carcinogens, caused p53 induction as early as 2 h following treatment, with peak increases within 4-12 h, Aflatoxin B-1 and 2-acetylaminofluorene, indirect-acting genotoxic carcinogens, caused a later induction of p53, with the peak increase appearing between 16 and 24 h following treatment. These observations demonstrate a correlation between p53 induction pattern and DNA damaging mechanism of genotoxins. Phenol, diethylstilbestrol and ethylacrylate also induced increases in cellular p53, The half-life of p53 protein was increased in cells treated with genotoxic agents, On the other hand, the epigenetic (non-DNA-reactive) carcinogens azathioprine and saccharin, as well as two substances generally considered to be non-carcinogens, dimetfiylsulfoxide and benzethonium chloride, had no effect on p53 protein levels of treated cells. Measurement of the cytotoxic effects of each of these chemicals led to the conclusion that p53 protein induction is not a general, non-specific consequence of the cytotoxic effect of these genotoxins, These results suggest that measurement of p53 protein induction may be an effective tool to identify environmental genotoxins.", 
"The two gene products of the CDKN2A gene, p16 and p19(ARF), have recently been linked to each of two major tumour suppressor pathways in human carcinogenesis, the RB1 pathway and the p53 pathway. p16 inhibits the phosphorylation of the retinoblastoma gene product by cyclin D-dependent kinases, whereas p19(ARF) targets MDM2, a p53 inhibitory protein, for degradation. A deletion of CDKN2A would therefore disturb both pathways. To explore the p53 pathway genes as a functional unit in diffuse large B cell non-Hodgkin's lymphomas (DLCL), we wanted to see whether there exists mutually exclusiveness of aberrations of CDKN2A, MDM2 and p53, since this has not been analysed previously. We investigated 37 DLCL for aberrations of p15, p16, p19(ARF), MDM2, and p53 at the epigenetic, genetic and/or protein levers. Homozygous deletion of CDKN2A was detected in seven (19%) of 37 tumours, and another three cases were hypermethylated at the 5' CpG island of p16. No point mutations were found in CDKN2B or CDKN2A. immunohistochemical staining of formalin-fixed, paraffin-embedded tissue for p16 confirmed these results, as all tumours with alterations of CDKN2A were pie immunonegative. We found p53 mutations in eight (22%) cases and MDM2 overexpression in 16 (43%) tumours. Twenty-three (62%) tumours had alterations of one or more p53 pathway components (p53, p19(ARF) and MDM2). Furthermore, 7/9 (78%) p16-immunonegative tumours showed co-aberration of p53 and/or MDM2. The lack of correlation between these aberrations suggests that DLCL acquire additional growth advantage by inactivating both of these critical regulatory pathways.", 
"Thromboxane A(2) is a biologically potent arachidonate metabolite through the cyclooxygenase pathway. It induces platelet aggregation and smooth muscle contraction and may promote mitogenesis and apoptosis of other cells. Its roles in physiological and pathological conditions have been widely documented. The enzyme that catalyzes its synthesis, thromboxane A(2) synthase, and the receptors that mediate its actions, thromboxane A(2) receptors, are the two key components critical for the functioning of this potent autacoid. Recent molecular biological studies have revealed the structure-function relationship and gene organizations of these proteins as well as genetic and epigenetic factors modulating their gene expression. Future investigation should shed light on detailed molecular signaling events specifying thromboxane A(2) actions, and the genetic underpinning of the enzyme and the receptors in health and disease.", 
"Carcinogenesis proceeds in discrete steps involving initiation and promotion, There is ample evidence that the underlying cause of initiation is mutation, whereas for tumor promotion different hypotheses exist postulating the involvement of both epigenetic and genetic changes, DNA repair protects against tumor formation, but it has not been proven whether protection occurs at the level of tumor initiation or promotion, Since the most advanced experimental system for studying multistep carcinogenesis is the mouse skin, we generated transgenic mice that overexpress the human DNA repair protein O-6-methylguanine-DNA methyltransferase (MGMT) in their epidermal cells by virtue of cytokeratin (Ck) promoters, Total cellular methyltransferase activity was found to he significantly higher in skin protein extracts of transgenic as compared to nontransgenic mice, CkMGMT transgenic mice along with nontransgenic controls were treated according to the multistage skin carcinogenesis protocol. For initiation, a single subthreshold dose of N-nitroso-N-methylurea (MNU) or 7,12-dimethylbenz(a)anthracene (DMBA) was topically applied to the dorsal skin of the mice, Tumor promotion was carried out by repeated 12-O-tetradecanoylphorbol-13-acetate application, Our results clearly show that CkMGMT transgenic mice are strongly protected against MNU- but not DMBA-initiated skin tumor formation, As compared to nontransgenic controls, transgenic mice exhibited an approximately 6-fold reduction of skin tumor incidence after treatment with 20 mu mol or 50 mu mol MNU followed by 12-O-tetradecanoylphorbol-13-acetate. These results provide direct and the most compelling evidence to date that the DNA lesion O-6-methylguanine is of decisive importance in tumor initiation, and that the protective effect of the repair protein MGMT in carcinogenesis is due to prevention of initiation without affecting tumor promotion.", 
"Relaxation of IGF2 imprinting occurs in Wilms tumours and many other cancers, but the mechanism of loss of imprinting (LOI) remains unknown. To investigate the role of altered DNA methylation in LOI, we examined the pattern of methylation of the human insulin-IGF2 region in Wilms tumours and the normal kidney. The analysis included regions homologous to three 'differentially methylated regions' of the mouse Igf2 gene (dmrs 0, 1 and 2), In tumours displaying normal IGF2 imprinting, and in the normal kidney, maternal allele-specific DNA methylation was identified spanning exons 2 and 3, This region is homologous to dmr 0, a site of maternal-specific differential methylation in the mouse. In Wilms tumours with relaxed imprinting or 11p15.5 LOH this region was unmethylated. No other differential methylation was identified, In particular, two sites of paternal methylation in the mouse (dmrs 1 and 2), and all three imprinted IGF2 promoters were not methylated in the kidney or in Wilms tumours, We postulate that LOI in Wilms tumours is associated with loss of maternal allele-specific methylation from a region located upstream of the imprinted IGF2 promoters, This region may contain cis acting sequences that coordinately influence imprinting.", 
"Disseminated disease, especially to the liver, constitutes the major risk of recurrence for colorectal cancer patients. However, successful resection can still be achieved in 25-35% of colorectal cancer patients with isolated metastases, To evaluate the clinical value of occult micrometastatic disease detection in lymph nodes, we tested genetic (K-ras and p53 gene mutations) and epigenetic (p16 promoter hypermethylation) molecular markers in the perihepatic lymph nodes from colorectal cancer patients with isolated liver metastases, DNA was extracted from 21 paraffin-embedded liver metastases and 80 lymph nodes from 21 colorectal cancer patients. K-ras and p53 gene mutations were identified in DNA from liver metastases by PCR amplification followed by cycle sequencing. A sensitive oligonucleotide-mediated mismatch ligation assay was used to search for the presence of K-ras and p53 mutations to detect occult disease in 68 lymph nodes from tumors positive for these gene mutations. Promoter hypermethylation at the pld tumor suppressor gene was examined in both liver lesions and lymph nodes by methylation-specific PCR, Sixteen of the 21 (76%) liver metastases harbored either gene point mutations or p16 promoter hypermethylation, Twelve of the 68 lymph nodes were positive for tumor cells by molecular evaluation and negative for tumor cells by histopathology and cytokeratin immunohistochemistry, whereas none were positive for tumor cells by histopathology or negative for tumor cells by molecular analysis (P = 0.0005, McNemar's test). Moreover, in three patients with lymph nodes that were histologically negative at all sites, molecular screening detected tumor DNA at one or more lymph nodes. Survival analysis shelved a median survival of 1056 days for patients without evidence of lymph node involvement by molecular analysis and 165 days for patients with positive lymph nodes by this approach (P = 0.0005), These results indicate that lymph node metastasis screening in colorectal cancer patients by molecular-based techniques increases the sensitivity of tumor cell detection and can be a good predictor of recurrence in colorectal cancer patients with resectable liver metastases.", 
"Epigenetic mechanisms may be the main driving force for critical changes in gene expression that are responsible for progression of prostate cancers. The three most extensively characterized mechanisms for epigenetic gene-regulation are (i) changing patterns of DNA methylation, (ii) histone acetylations/deacetylations, and (iii) alterations in regulatory feedback loops for growth factors. Several studies have indicated that DNA hypermethylation is an important mechanism in prostate cancer for inactivation of key regulatory genes such as E-cadherin, pi-class glutathione S-transferase, the tumor suppressors CDKN2 and PTEN, and IGF-II. Similarly, histone acetylations and deacetylations are frequently associated respectively with transcriptional activation (e.g. IGFBP-2 and p21) and repression (e.g. Mad:Max dimers) of genes linked to prostate cancer progression. Recently, histone acetyltransferase and deacetylase activities have been shown to be intrinsic with transcriptional coregulator proteins that bind to steroid receptors (e.g. SRC-1 and PCAF). Changes in regulatory feedback loops for growth factors with prostate cancer progression tend toward shifts from paracrine to autocrine control where the receptor and ligand are produced by the same cell. While there are several examples of this progression pattern in prostate tumors such as with IGF, FGF, TGF-alpha and their respective receptors, the precise mechanism (i.e. epigenetic or mutational) is less certain. In the context of treatment options, the contribution of mutational versus epigenetic events to prostate cancer progression is an improtant consideration. Irreversible genetic changes are likely to be less amenable to therapeutic control than are epigenetic ones.", 
"This review highlights current knowledge of environmental factors in carcinogenesis and their cellular targets. The hypothesis that environmental factors influence carcinogenesis is widely supported by both epidemiological and experimental studies. The fact that only a small fraction of cancers can be attributed to germline mutations in cancer-related genes further buttresses the importance of environmental factors in carcinogenesis, Furthermore, penetrance of germline mutations may be modified by either environmental or other genetic factors. Examples of environmental factors that have been associated with increased cancer risk in the human population include chemical and physical mutagens (e.g. cigarette smoke, heterocyclic amines, asbestos and UV irradiation), infection by certain viral or bacterial pathogens, and dietary nongenotoxic constituents (e.g. macro- and micronutrients), Among molecular targets of environmental influences on carcinogenesis are somatic mutation (genetic change) and aberrant DNA methylation (epigenetic change) at the genomic level and post-translational modifications at the protein level. At both levels, changes elicited affect either the stability or the activity of key regulatory proteins, including oncoproteins and tumor suppressor proteins. Together, via multiple genetic and epigenetic lesions, environmental factors modulate important changes in the pathway of cellular carcinogenesis.", 
"DNA methylation is now recognized as an important mechanism regulating different functions of the genome; gene expression, replication, and cancer, Different factors control the formation and maintenance of DNA methylation patterns. The level of activity of DNA methyltransferase (MeTase) is one factor. Recent data suggest that some oncogenic pathways can induce DNA MeTase expression, that DNA MeTase activity is elevated in cancer, and that inhibition of DNA MeTase can reverse the transformed state. What are the pharmacological consequences of our current understanding of DNA methylation patterns formation! This review will discuss the possibility that DNA MeTase inhibitors can serve as important pharmacological and therapeutic tools in cancer and other genetic diseases.", 
"The mouse skin model of multistage carcinogenesis has for many years provided a conceptual framework for studying carcinogenesis mechanisms and potential means for inhibiting specific stages of carcinogenesis. The process of skin carcinogenesis involves the stepwise accumulation of genetic change ultimately leading to malignancy. Initiation, the first step in multistage skin carcinogenesis involves carcinogen-induced genetic changes. A target-gene identified for some skin tumor initiators is c-Ha-ras. The second step, the promotion stage, involves processes whereby initiated cells undergo selective clonal expansion to form visible premalignant lesions termed papillomas. The process of tumor promotion involves the production and maintenance of a specific and chronic hyperplasia characterized by a sustained cellular proliferation of epidermal cells. These changes are believed to result from epigenetic mechanisms such as activation of the cellular receptor, protein kinase C, by some classes of tumor promoters. The progression stage involves the conversion of papillomas to malignant tumors, squamous cell carcinomas. The accumulation of additional genetic changes in cells comprising papillomas has been correlated with tumor progression, including trisomies of chromosomes 6 and 7 and loss of heterozygosity. The current review focuses on the mechanisms involved in multistage skin carcinogenesis, a summary of known inhibitors of specific stages and their proposed mechanisms of action, and the relevance of this model system to human cancer.", 
"Carcinogenesis is a multistage process driven by carcinogen-induced genetic and epigenetic damage in susceptible cells that gain a selective growth advantage and undergo clonal expansion as the result of activation of protooncogenes and/or inactivation of tumor suppressor genes. Therefore, the mutational spectra of chemical and physical carcinogens in these critical genes are of interest to define endogenous and exogenous mutational mechanisms. The p53 tumor suppressor gene is ideally suited for analysis of the mutational spectrum. Such an analysis has revealed evidence for both exogenous and endogenous molecular mechanisms of carcinogenesis. For example, an informative p53 mutational spectrum of frequent G --> T transversions in codon 249 is found in hepatocellular carcinomas from either Qidong, People's Republic of China, or southern Africa. This observation links exposure to aflatoxin B1, a known cancer risk factor in these geographic regions, with a specific mutation in a cancer-related gene. Other studies indicate that abnormalities in genes controlling the cell cycle may cause genomic instability and increase the probability of neoplastic transformation. Finally, mechanistic understanding of carcinogenesis is leading to improved cancer risk assessment and to the identification of individuals at high cancer risk.", 
"Background: Estrogens are potent mammary tumor promoters influencing post-initiational events via epigenetic mechanisms. The upregulation (i.e., induction) of the C16-alpha-hydroxylation pathway during 17-beta-estradiol (E2) biotransformation has been associated with mammary cell transformation. The action of E2 metabolites on tumorigenic transformation, however, is poorly understood. Purpose: The newly established mammary epithelial cell line C57/MG, derived from the C57BL mouse strain, was used to examine whether E2 or its metabolites, 16-hydroxyestrone (16-alpha-OHE1) and estriol (E3), function as initiators of mammary cell transformation. Methods: DNA repair (hydroxyurea-insensitive thymidine uptake), estrogen metabolism (H-3 exchange to form (H2O)-H-3), hyperproliferation (increased cell number), and acquisition of anchorage-independent growth (soft-agar colonies) were used as quantitative end points to measure the relative extent of transformation. Results: Treatment of cells with 200 ng/mL 16-alpha-OHE1 resulted in a 55.2% increase in DNA repair synthesis, a 23.09% increase in proliferative activity, and a 18-fold increase in the number of soft-agar colonies, relative to the solvent controls (P < .0001). The extent of upregulation of the three end points was similar to that induced by the genotoxic mammary carcinogen 7, 12-dimethylbenz[a]anthracene (DMBA, positive control). DMBA treatment also upregulated the ratio of 16-alpha/C2 hydroxylation of E2 leading to increased formation of 16-alpha-OHE1. E2 and E3 were not effective in upregulating these markers for transformation. Conclusion: These results demonstrate that in nontransformed C57/MG cells, 16-alpha-OHE1 may function as an initiator, perturbing the intermediate biomarkers for preneoplastic transformation.", 
"Environmental, occupational, and recreational exposures to carcinogens contribute to cancer risk in humans. Cancer formation is a multistage process involving tumor initiation, promotion, conversion, and progression. Carcinogens can affect any of these stages through genetic and epigenetic mechanisms. The association of a suspected carcinogenic exposure and cancer risk can be studied in populations with classic epidemiologic techniques. However, these techniques are not applicable to the assessment of risk in individuals. Molecular epidemiology, in contrast, is a field that integrates molecular biology, in vitro and in vivo laboratory models, biochemistry, and epidemiology to infer individual cancer risk. Carcinogen-macromolecular adduct levels, and somatic cell mutations can be measured to determine the biologically effective dose of a carcinogen. Molecular epidemiology also explores host cancer susceptibilities, such as carcinogen metabolic activation, DNA repair, endogenous mutation rates, and inheritance of mutated tumor suppressor genes. Substantial interindividual variation for each of these biologic end points has been shown and, therefore, highlights the need for assessing cancer risk on an individual basis. Given the pace of the last decade, it is feasible that the next 10 years will allow molecular epidemiologists to develop a cancer-risk profile for an individual that includes assessment of a number of factors. This will help focus preventive strategies and strengthen quantitative risk assessments.", 
"Background. Global DNA hypomethylation has been observed in some human neoplasms and has been implicated as an important factor in carcinogenesis. The current study was designed to assess whether DNA hypomethylation occurs in cervical dysplasia and cancer, and to determine the relationship between the degree of DNA hypomethylation and the grade of neoplasia. Methods. Cervical biopsy specimens were obtained from colposcopically identifiable lesions in 41 patients with abnormal Pap smear results. The extent of global DNA methylation was assessed by incubating the extracted DNA with [H-3]-S-adenosylmethionine and Sss1 methyltransferase, an enzyme that specifically catalyzes the transfer of methyl groups to cytosine residues in the cytosine-guanine doubler. The degree of exogenous H-3-methyl group incorporation into the DNA therefore is related reciprocally to the extent of endogenous DNA methylation. These data were compared with the histopathologic classification of the lesions. Results. The extent of H-3-methyl group incorporation was increased threefold and sevenfold in the DNA from cervical dysplasia and cancer, respectively, compared with the DNA from normal cervical tissue (P = 0.006, analysis of variance). Significant incremental increases in DNA hypomethylation were observed in the progression from normal and low grade squamous intraepithelial lesions (SIL) to high grade SIL and to cancer (P < 0.0001,trend). Conclusions. These data show that global DNA hypomethylation is a significant epigenetic event in cervical carcinogenesis and that the degree of DNA hypomethylation increases with the grade of cervical neoplasia. These data suggest that global DNA methylation may serve as a biochemical marker of cervical neoplasia.", 
"Intestinal metaplasia in the stomach increases the risk of gastric cancer, and the increased risk is proportional to the extent of the metaplasia. This risk could be generated by one or more mechanisms: (1) the metaplastic tissue is an early step in a multistep induction process; (2) the metaplastic tissue is an epigenetic change that raises the pH of gastric juice by replacing oxyntic mucosa, favoring the growth of a bacteria capable of generating endogenous mutagens; and/or (3) the metaplasia is only a marker for chronic gastritis due to H. pylori infection or pernicious anemia. With the last mechanism, the inflammatory response favors intramural mutagenesis that might result in metaplasia or neoplasia as independent events. Finding gene rearrangements common to both metaplastic and neoplastic tissue may establish a direct link between them, but too few have been identified to account for the large number of stomach cancers that develop in high risk populations. Histochemical and immunochemical stains that identify enzymes or mucosubstances may suggest that metaplastic epithelial cells resemble small or large intestinal cells, but they are distinctly different from both. Moreover, these stains do not indicate whether a given cytologic change is genetic or epigenetic; therefore, they cannot be used to define the relationship between metaplasia and neoplasia. It is unnecessary for practicing physicians to await resolution of this question. It can be assumed that any person with extensive metaplasia is at high risk for gastric cancer and should be subject to periodic screening. The extent of the metaplastic process is probably more important than the metaplastic subtype.", 
"Immunocytochemistry (ICC) has been used routinely to stain for p53 overexpression in a range of human tumours. The underlying assumption has been that positive staining indicates a mutation in the p53 coding sequence. Recently, however, discordancy has been observed and the accuracy of ICC as a marker of p53 gene mutation has been questioned. In this study of 109 colorectal adenocarcinomas, we compared ICC staining with p53 gene mutations detected by single-strand conformation polymorphism (SSCP) analysis. Concordancy between the two techniques was found in 69% of tumours. ICC-positive/SSCP-negative cases accounted for 20% of rumours and ICC-negative/SSCP-positive cases for the remaining 11%. These results caution against the assumption that p53 protein overexpression is always associated with a gene mutation. Epigenetic phenomena may account for a significant proportion of ICC-positive tumours.", 
"The outcome of cancer metastasis depends on the interaction of metastatic cells with various host factors. The implantation of human cancer cells into anatomically correct (orthotopic) sites in nude mice can be used to ascertain their metastatic potential. While it is clear that vascularity and local immunity can retard or facilitate tumor growth, we have found that the organ environment also influences tumor cell functions such as production of degradative enzymes. The organ microenvironment can also influence the response of metastases to chemotherapy. It is not uncommon to observe the regression of cancer metastases in one organ and their continued growth in other sites after systemic chemotherapy. We demonstrated this effect in a series of experiments using a murine fibrosarcoma, a murine colon carcinoma, and a human colon carcinoma. The tumor cells were implanted subcutaneously or into different visceral organs. Subcutaneous tumors were sensitive to doxorubicin (DXR), whereas lung or liver metastases were not. In contrast, sensitivity to 5-FU did not differ between these sites of growth. The differences in response to DXR between s.c. tumors (sensitive) and lung or liver tumors (resistant) were not due to variations in DXR potency or DXR distribution. The expression of the multidrug resistance-associated P-glycoprotein as determined by flow cytometric analysis of tumor cells harvested from lesions in different organs correlated inversely with their sensitivity to DXR: increased P-glycoprotein was associated with overexpression of mdr1 mRNA. However, the organ-specific mechanism for upregulating mdr1 and P-glycoprotein has yet to be elucidated.", 
"The genetic and epigenetic effects of nutritional folate deficiency were studied in two Chinese hamster ovary (CHO) cell Lines. The CHO-ABS cell line (hemizygous at the aprt locus) and CHO-UV5 (DNA repair-deficient mutant of AA8) were cultured in Ham's F-12 medium or in custom-prepared Ham's F-12 medium lacking foIic acid, thymidine, and hypoxanthine. Cells cultured acutely in the folate deficient medium exhibited initial growth arrest, followed by massive cell death and DNA fragmentation into nucleosomal multimers characteristic of apoptosis. Although prolonged culture in the folate deficient medium was cytostatic and lethal to the majority cells, minor subpopulations in both cell lines failed to initiate cell death, exhibited phenotypic abnormalities, and adapted a selective growth advantage under marginal folate conditions. These ''resistant'' clones exhibited major alterations in deoxynucleotide pools associated with an increase in mutant frequency at the aprt locus as detected by resistance to cytotoxicity in 8-azaadenosine. The mutation frequency in the DNA repair-deficient CHO-UV5 cells was similar to 100-fold greater than that in the parental AA8 clones, underscoring the importance of DNA repair under conditions of folate deficiency and nucleotide pool imbalance. The enhanced mutation frequency in the DNA repair-competent folate-deficient CHO-BAS cells suggests that DNA repair activity is less effective under folate-deficient conditions. These results add to the accumulating clinical and experimental evidence relating chronic folate deficiency to genomic instability and carcinogenesis.", 
"The demand for taxol, a promising cancer chemotherapeutic agent, far exceeds supply. Presently, taxol is derived from the bark of the Pacific yew, Taxus brevifolia, a small, slow-growing evergreen tree native to the northwestern United States. Knowledge of the distribution and magnitude of genetic and non-genetic sources of variation in taxol content in the genus Taxus is necessary if supply issues are to be met through plant harvesting. Analytical determinations of taxol, cephalomannine, and baccatin III in more than 200 trees representing several populations of T. brevifolia and other yew taxa indicate that (1) significant variation in taxane content exists among and within populations and species, (2) taxol levels exceeding those reported for T. brevifolia bark were found in shoots of individual trees from most taxa studied, and (3) the season in which samples are collected and handling procedures can influence taxane content.", 
"During the evolution of single-celled organisms to multi-cellular metazoans, a family of highly conserved genes coding for proteins (connexins), which as hexameric units (connexons), has evolved to form intercellular channels (gap junctions). These gap junctions allow ions and small molecular weight molecules to flow between coupled cells, thereby facilitating synchronization of electrotonic or metabolic cooperation. Control of cell proliferation, cell differentiation and adaptive responses of differentiated cells have been speculated to be biological roles of gap junctions. The regulation of these gap junctions can occur at the transcriptional, translational and posttranslational levels. Transient downregulation by endogenous or exogenous chemicals can bring about adaptive or maladaptive consequences depending on circumstances. Stable abnormal regulation of gap junction function has been associated with the activation of several oncogenes. Several tumor suppressor genes have also been associated with the up-regulation of gap junction function. Since gap junctions exist in all organs of the multi-cellular organisms, the dysfunction of these gap junctions by various toxic chemicals which have cell type/tissue/organ specificity could bring about very distinct clinical consequences, such as embryo lethality or teratogenesis, reproductive dysfunction in the gonads, neurotoxicity of the CNS system, hyperplasia of the skin, and tumor promotion of initiated tissue. understanding how many non-mutagenic chemicals might alter normal gap junction function should form the basis of ''epigenetic'' toxicology. On the other hand, restoring normal gap junction function to cells which have dysfunctional intercellular communication could be the basis for a new approach for therapeutic pharmaceuticals.", 
"Carcinogenesis in mouse skin can be divided into three distinct stages: initiation, promotion, and progression (malignant conversion). Initiation, induced by a single exposure to a genotoxic carcinogen, can result from a mutation in a single critical gene (e.g., ras(Ha)), apparently in only a few epidermal cells. The change is irreversible. Promotion, resulting in the development of numerous benign tumors (papillomas), is accomplished by the repeated application of a nonmutagenic tumor promoter. The effects of single applications of tumor promoters are reversible since papillomas do not develop after insufficient exposure of initiated skin to promoters or when the interval between individual promoter applications is increased sufficiently. The reversibility of promotion suggests an epigenetic mechanism. Promoter treatment provides an environment that allows the selective clonal expansion of foci of initiated cells. The conversion of squamous papillomas to carcinomas (termed progression or malignant conversion) occurs spontaneously at a low frequency. The rate of progression to malignancy can be significantly increased by treatment of papilloma-bearing mice with certain genotoxic agents. These progressor agents or converting agents are likely to act via a second genetic change in papillomas already bearing the initiating mutation. Progression in the skin is characterized by genetic changes that result in several distinct changes in the levels or activity of structural proteins, growth factors, and proteases. The mechanisms involved in progression are being studied in epidermal cell culture. In order to determine the in vivo phenotype of cultured cells, a grafting system was developed in which the cells were transferred from culture to a prepared skin bed in athymic mice. Introduction of an activated v-fos oncogene into initiated cells bearing an activated ras(Ha) gene produced cells with a carcinoma phenotype, i.e., carcinomas formed when the cells were grafted as part of reconstituted skin. Grafted keratinocytes containing the ras(Ha) gene alone produced papillomas; with v-fos alone, normal skin formed when grafted. The ras(Ha)/fos carcinomas showed changes in differentiation markers characteristic of chemically induced carcinomas. A cell culture assay utilizing cells initiated by the introduction of an activated ras(Ha) oncogene was developed to study progression. After exposure of initiated cells to progressor agents under conditions in which the proliferation of the ras(Ha)-initiated cells was suppressed, proliferating foci developed, with a good correlation of activity in the assay with activity in the progression stage in vivo. The cell culture assay provides a quantitative model to study chemically induced neoplastic progression and may be useful to identify potential progressor agents.", 
"Over-expression and abnormal intracellular location of the product of the oncogene c-myc in colonic dysplasia and neoplasia may be related to alterations in epigenetic mechanisms controlling the functioning of this gene. We have investigated the methylation patterns of the c-myc oncogene in human colorectal tissue representing various stages of dysplasia and neoplasia, including metastasis to liver, omentum and lymph node. Comparison of normal and neoplastic tissues from the same patient showed a decrease in methylation in a specific CCGG site in the third exon of c-myc through the progression from normal via dysplastic to neoplastic and metastatic tissue. Quantitative analysis revealed that in colonic adenocarcinomas an average of 66.1% and in metastatic deposits 83.1% of the myc gene DNA was hypomethylated at this site, as compared to a value of 9.2% in normal colonic mucosa. Adenomatous polyps showed an average value of 50.5% and hyperplastic polyps, 24.8%. The results suggest that partial hypomethylation of the c-myc gene third exon is associated with cell proliferation, and that deregulation of proliferation may be linked to the high levels of hypomethylation, presumably involving both copies of the gene in some cells, which occur at a relatively early stage in neoplastic progression.", 
"Despite the plethora of ''oncogenes'' and ''tumor suppressor genes,'' the hypothesis that cancer is usually the result of genomic mutations may be wrong. We should at least examine the alternative hypothesis, for which there is considerable evidence, that mutations do not commonly beget cancer, but rather that cancer phenotypes result from confused or aberrant patterns of normal-gene expression; the abnormal patterns are postulated to result from epigenetic mechanisms rather than from mutations. The epigenetic hypothesis that I am proposing suggests that cancers may exhibit mutations primarily because replicative errors at inactive sites in the cancer genome may he repaired slowly or not at all, but the mutations so produced, occurring at already inactivated sites in the genome, may have limited biological significance. Thus, it may be more correct to say that cancers beget mutations than it is to say that mutations beget cancers.", 
"Cells possess within their repertoire of epigenetic programs the ability to undergo a process of cellular suicide, termed programmed cell death. This programmed cell death process involves an epigenetic reprogramming of the cell that results in an energy-dependent cascade of biochemical and morphologic changes within the cell (also termed apoptosis), resulting in its death and elimination. Although the final steps (i. e. DNA and cellular fragmentation) are common to cells undergoing programmed death, the activation of this death process is initiated either by sufficient injury to the cell induced by various exogenous damaging agents (e.g. radiation, chemicals, viruses etc) or by changes in the levels of a series of endogenous signals (e.g. hormones and growth/survival factors). As an illustrative example, the role of androgen as a cell type specific endogenous regulator of the programmed death of normal and neoplastic prostatic cells will be presented.", 
"Fibroblasts are important contributors to both benign and malignant growth of prostate epithelial cells in vivo. In the human prostate cancer model that we have established, we can grow human LNCaP tumors reproducibly in athymic mice by coinoculating the animals with human LNCaP epithelial cells plus fibroblasts derived from either the prostate or bone; human lung, normal rat kidney, and embryonic mouse fibroblasts were inactive. We have delivered conditioned medium isolated from competent fibroblasts directly to sites where the tumor cells were injected and found that the conditioned medium alone confers tumorigenicity. Further studies of the mechanism of fibroblast-epithelial interaction have indicated that close metabolic cooperation between fibroblast and epithelial cells, involving the production of growth factors by the epithelial cells and the production of extracellular matrices and growth factors by the fibroblasts (assayed in vitro), is important in promoting prostate tumor growth in vivo. We have also investigated the possible in vivo interaction between extracellular matrix proteins such as laminin, collagens, heparan sulfate proteoglycans and Matrigel and prostate epithelial cells. Selective extracellular-matrix components were found to confer tumorigenicity to the prostate epithelial cells. Moreover, extracellular-matrix components were observed to induce cancer cell differentiation and alter permanently the morphology, gene expression and tumorigenic potential of the cancer epithelial cells.", 
" ", 
"Butylated hydroxyanisole (BHA) is a synthetic food antioxidant used to prevent oils, fats and shortenings from oxidative deterioration and rancidity. This review depicts the current knowledge on BHA. The physical and chemical characteristics of BHA are summarized and its function as a food antioxidant is made clear. The toxicological characteristics of BHA and its metabolic fate in man and animal are briefly reviewed. Special emphasis is laid on the carcinogenicity of BHA in the forestomach of rodents and to related events in the forestomach and other tissues in experimental animals. At present there is sufficient evidence for carcinogenicity of BHA, but there is hardly any indication that BHA is genotoxic. Therefore risk assessment for this epigenetic carcinogen is based on non-stochastic principles. However, the mechanism underlying the tumorigenicity of BHA is not known. In the last part of this review an attempt is made to unravel the unknown mechanism of carcinogenicity. It is hypothesized that BHA gives rise to tumor formation in rodent forestomach by inducing heritable changes in DNA. Evidence is being provided that reactive oxygen species, in particular hydroxylradicals, may play a crucial role. The key question with respect to risk assessment for BHA is whether or not the underlying mechanism is thresholded, which is important for the choice of the appropriate model to assess the risk, if any, for man and to manage any potential risk.", 
"Critical phenotypic changes that occur during the progression of breast cancer include the loss of hormone-dependence, acquired resistance to systemic therapies, and increased metastatic potential. We have isolated a series of MCF-7 human breast cancer variants which exhibit hormone-independent growth, antiestrogen resistance, and increased metastatic potential. Analysis of the phenotypes of these variants strongly suggests that changes in the expression of specific genes may be critical to the generation of phenotypic diversity in the process of malignant progression in breast cancer. Epigenetic changes may contribute significantly to the generation of these phenotypic changes observed during breast cancer progression. Many of the characteristics of the progressed phenotypes appear to have arisen in response to appropriate selective pressures (growth in ovariectomized nude mice: growth in the presence of antiestrogens). These observations are consistent with the concept of clonal selection and expansion in the process of malignant progression.", 
"In the present study, the possible role of the polyunsaturated fatty acids linoleic and arachidonic acid in the chemical induction of carcinogenesis has been investigated. Analysis of 7,8-dihydro-8-oxo-2'-deoxyguanosine (8-oxodG) levels in 2'-deoxyguanosine (dG) and isolated DNA has demonstrated that linoleic and arachidonic acid are capable of inducing this specific genotoxic damage. This effect appears to be related to the degree of fatty acid unsaturation, since it was not induced by monounsaturated oleic acid. Enzymatic peroxidation of linoleic and arachidonic acid resulted in a significant increase in oxidative DNA damage. Studies on the interference of radical scavengers with the induction of 8-oxodG in combination with electron spin resonance spectroscopy demonstrated that the superoxide anion was generated during peroxidation of these fatty acids and that singlet oxygen is most likely involved in the formation of oxidative DNA damage. The level of oxidative damage in dG and single-stranded DNA was higher as compared to that in native DNA after equimolar treatment. Exposure of human lymphocytes to linoleic or arachidonic acid did not result in a significant increase in levels of 8-oxodG. This may indicate that the rate of intracellular peroxidation is relatively low and/or that nuclear DNA in intact cells is effectively protected against genetic damage induced by reactive oxygen species. It is therefore concluded that relatively short periods of linoleic or arachidonic acid administration are not likely to impose a direct genotoxic risk. It can, however, not be excluded that chronic exposure to polyunsaturated fatty acids induces oxidative DNA damage or is related to cancer risk by epigenetic mechanisms, as is also indicated by the observed cytotoxic effects of linoleic and arachidonic acid.", 
"Hemophilia A is caused by the lack of functional blood-clotting factor VIII. We have used retrovirus-mediated gene transfer to generate various cell lines, rodent as well as human, that secrete the human factor VIII protein. To study whether transplantation of genetically modified fibroblasts is a feasible approach for gene therapy of hemophilia A, we implanted the factor VIII-secreting cells into immune-deficient mice. Implantation of factor VIII-secreting primary human skin fibroblasts resulted in long-term persistence of the transplanted cells; cells recovered from the implants up to 2 months post-implantation still had the capacity to secrete factor VIII when regrown in tissue culture. However, we were unable to detect any human factor VIII in plasma samples of the recipient mice. The absence of human factor VIII in the recipients' plasma is shown to be due neither to (epigenetic) inactivation of the retroviral vector in vivo, nor to inability of the stationary cells to secrete factor VIII protein. However, we did note a rapid clearing of the human factor VIII : CAg from plasma upon intravenous injection of plasma-derived human factor VIII in mice (t1/2 < 60 min vs. 10 hr in humans). This phenomenon can fully explain the apparent absence of human factor VIII in the recipients' plasma.", 
"The liver tumor-prone B6C3F1 mouse (C57BL/6female x C3H/Hemale), in conjunction with the more susceptible C3H/He paternal strain and the resistant C57BL/6 maternal strain, is an excellent model for studying the mechanisms involved in carcinogenesis. The study reported here indicated that the B6C3F1 mouse inherited a maternal raf allele containing a methylated site not present in the paternal allele. Seven days after partial hepatectomy or after administration of a promoting dose of phenobarbital (PB) for 14 d; raf in B6C3F1 mouse liver was hypomethylated. The additional methylated site in the allele inherited from C57BL/6 was not maintained. The methylation status of raf in the liver of the C57BL/6 mouse was not affected by PB treatment. This indicates that the B6C3F1 mouse is less capable of maintaining methylation of raf than the C57BL/6 strain is. In both PB-induced and spontaneous B6C3F1 liver tumors, raf was hypomethylated in a nonrandom fashion. The level of raf MRNA increased in seven of 10 PB-induced tumors but in only one of five spontaneous tumors, whereas the level of Ha-ras mRNA increased in nine of 10 PB-induced tumors and in four of five spontaneous tumors. The results of our investigation (a) support the hypothesis that hypomethylation of DNA is a nongenotoxic mechanism involved in tumorigenesis, (b) support the notion that PB promotes liver tumors that develop along a pathway different from that leading to spontaneous tumors, and (c) indicate that differences in DNA methylation between C57BL/6 and B6C3F1 mice could, in part, account for the unusually high tendency of the latter strain to develop liver tumors. (C) 1994 Wiley-Liss, Inc.", 
"Objective.-To evaluate the role of candidate genes in the susceptibility to multiple sclerosis (MS) and describe the role of T-cell receptor (TCR) gene rearrangements in the MS brain lesion in identifying a major target of the immune response in this disease. Data Sources.-MEDLINE, bibliography review of published data, and unpublished studies. Study Selection.-Published studies using novel molecular approaches to analyze the role of the major histocompatibility complex (MHC) and TCR gene complexes, as well as other candidate genes, in susceptibility to MS. We analyze epigenetic events involving TCR genes in individuals with MS and describe recent clinical trials in which immunotherapy has been attempted. Data Synthesis.-Consistent with a polygenic model for disease predisposition, MHC and TCR gene associations with MS are relatively weak. Despite intensive research, no other putative ''MS genes'' have been firmly established. The analysis of TCR rearrangements in the brain lesion has helped to identify a major target of the immune response in MS. Conclusion.-Understanding the genetic basis for autoimmune demyelination will offer new possibilities for the treatment of this illness.", 
"By analyzing a culture system of human prostatic epithelial cells (HPEC) and human prostatic fibroblasts (HPF) for expression of several determinants by immunocytochemistry, we have shown that long-term cultures are able to preserve the phenotypic characteristics of the normal tissue from which they are derived. The cytoskeletal elements, prostate-specific proteins, and steroid receptor profiles were compared to those of prostatic epithelium and stroma in situ. When cultured in low serum and low calcium medium, the adult HPEC grew as two layers of cells, the upper one of which retained the differentiation characteristics observed in the luminal fraction of normal prostatic epithelium. This cell type is the likely origin of prostatic neoplasia, with expression of CK8, 18, and 19 but not CK14. Androgen receptors, prostatic-specific antigen, and prostatic acid phosphatase are also expressed in vitro but at lower level than in situ. The lower cell layer expressed most of the same determinants but at a much lower level, suggestive of a stem-cell type. The HPF cultured in RPMI serum supplemented media retained the stromal pattern of the cells observed in situ. Culture systems which conserve the characteristics of their normal counterparts in vivo should provide useful models for studying in vitro genetic and epigenetic factors associated with differentiation and proliferation, but also with tumorigenic progression in the prostatic gland. (C) 1994 Academic Press, Inc.", 
"This study measured the effect of precise doses of 2-acetylaminofluorene (AAF) in inducing DNA damage, functional changes and neoplastic conversion in rat liver. Groups of male F344 rats at 9 weeks of age were exposed to cumulative doses of 0.5 or 2.0 mmol AAF per kg body weight given by gavage daily 5 days per week over an 8-week period and maintained with no further exposure for up to 8 weeks. Administration of AAF resulted in the formation of N-deoxyguanosin-(8-yl)-2-aminofluorene in liver DNA in relationship to dose. In centrilobular hepatocytes the zone of glutamine synthetase-expressing cells was reduced by exposure. By 8 weeks, but not at 4 weeks, the higher of the two doses of AAF provoked an increase in cell proliferation measured by immunohistochemical incorporation of bromode-oxyuridine. Altered hepatocellular foci expressing the placental form of glutathione transferase were induced by the high dose of AAF at 4 weeks, but not at the low dose. At 8 weeks the incidence of foci at the high dose was 79 times that induced by the low dose. These foci were highly proliferative. In animals exposed to AAF for 8 weeks and maintained for 4 weeks with no exposure, DNA adducts decreased by 80% and cell proliferation subsided by 80%, although the glutamine synthetase zone remained diminished. After discontinuation of AAF, the number of foci diminished by 50% and their proliferation subsided by 80% at 4 weeks, indicating a phenotypic reversion of many foci. With this protocol of administration of precise doses of AAF, we have established non-linearity of effects and a lack of correlation between DNA adduct formation and induction of cellular lesions. We suggest that doses in the range of those reported can be used to study the contribution of epigenetic and genotoxic effects in carcinogenesis and to study threshold events.", 
"The hepatocarcinogenicity of peroxisome proliferators (PPs) in rodents has been attributed both to oxidative DNA damage resulting from excessive leakage of peroxisomal H2O2 and to increased hepatocellular replication that may be independent of peroxisome proliferation. Because of the growing association between tumor promotion and alterations in growth-regulatory signal transduction pathways, we investigated whether PPs can modulate these pathways in a mouse liver epithelial cell line, BNL-CL.2. We tested two PPs that differ markedly in rodent tumorigenicity for their ability to activate immediate-early proto-oncogene expression. 4-Chloro-6-(2,3-xylidino)-2-pyrimidinylthioacetic acid (Wy-14643), a highly tumorigenic PP, was an exceptionally strong inducer of c-fos expression. In contrast, diethylhexyl phthalate (DEHP), a weakly tumorigenic PP, was a very poor inducer of c-fos expression. Wy-14643 was also stronger than DEHP in stimulating c-jun expression, whereas both PPs were fairly strong inducers of jun-B and jun-D. The induction of fos and jun expression by Wy-14643 was specifically inhibited by the protein kinase C inhibitor 1-(5-isoquinolinesulfonyl)-2-methylpiperizine dihydrochloride (H-7). DEHP-induced gene expression was strongly inhibited by H-7, but was also partially inhibited by an inhibitor of protein kinase A. The activation of fos and jun gene expression by PPs was independent of peroxisome proliferation since it was an immediately-early response not requiring protein synthesis and since the cell lines used in this study do not undergo peroxisome proliferation. Our results raise the possibility that the carcinogenicity of PPs may be due, in part, to epigenetic modulation of growth-regulatory signal transduction pathways. (C) 1993 Wiley-Liss, Inc.", 
"Supine arterial blood pressure measurements of 89 patients with homozygous sickle cell disease and normal renal function were compared with those of an age-, race-, and sex-matched normal population and with those of individuals who had similar levels of anemia due to beta thalassemia. Consistent with previous reports, sickle cell patients had significantly lower blood pressure than the normal population. However, within most age groups, sickle cell patients tended to have higher than expected blood pressure than individuals with similar or less severe degrees of anemia. Furthermore, the authors have found an association between cerebrovascular accident and elevated blood pressure in men, even in a range of systolic and diastolic pressures that would be considered normal by conventional standards. These results reiterate the intricate relationship that exists between factors governing red cell rheology and microvascular tone. They also raise the possibility that ''relative'' hypertension may be associated with other vaso-occlusive manifestations of the sickle cell syndromes.", 
"5-Azacytidine (5-aza-CR) and 5-aza-2'-deoxycytidine (5-aza-CdR), analogs of cytidine modified in position 5, were originally developed as antitumor agents, and have been useful in the treatment of both childhood and adult leukemias. These agents are cytotoxic per se, but also induce differentiation in several experimental systems, most notably the induction of muscle, adipocytes, and chondrocytes in cultures of drug-treated mouse embryo fibroblasts. The mechanisms underlying this drug-induced differentiation have been difficult to unravel, a fact which limits the rational design of differentiation therapy as a modulator of cancer using these agents. Induction of new developmental pathways in cultured cells involves stable, heritable changes, presumably of an epigenetic nature. Our early studies demonstrated that changes in methylation of cytosine in DNA occurred concurrently with changes in developmental potential, and that the presence of 5-azacytosine in DNA interfered with the action of DNA methyltransferase. Since DNA methylation is believed to be involved at some level in the regulation of gene expression, the hypothesis was developed that changes in methylation allowed the expression of new genes whose activity initiated new pathways of differentiation. The characterization of this drug-induced system of differentiation has therefore opened the way to identifying genes directly involved in the initiation or modification of pathways of differentiation. The first of these was MyoD, a member of a family of myogenic determination genes. Expression of MyoD in myogenic cell lines has been correlated with loss of methylation at specific sites in the genome, but the critical events leading to expression of MyoD and muscle differentiation are poorly understood. Recant developments in understanding this mechanism are discussed.", 
"The dose and time dependence of the cellular phenotype in preneoplastic and neoplastic liver lesions was evaluated quantitatively in groups of male Sprague-Dawley rats exposed for 7 weeks to 0, 12 and 24 mg/kg body wt of N-nitrosomorpholine (NNM) and studied at different time points up to 80 weeks after withdrawal of NNM (stop model). NNM-treated rats showed a dose- and time-dependent increase in the total number and volume of preneoplastic foci of altered hepatocytes (FAH) and in the incidence of hepatocellular adenomas (HCA) and carcinomas (HCC) at both dose levels, compared with the untreated controls. After stopping treatment with 12 mg/kg body wt, the well-known sequence of cellular changes leading from glycogenotic clear and clear/acidophilic cell foci to mixed and diffusely basophilic cell populations poor in glycogen was found. In contrast, at the higher NNM dose level (24 mg/kg) predominantly mixed and diffusely basophilic cell foci appeared immediately after cessation of treatment, but their number rapidly declined up to 13 weeks after withdrawal. At the same time, there was a reciprocal increase in the number of the less altered clear/acidophilic cell foci, indicating an early reversion-linked phenotypic instability of FAH. However, in spite of this reversion higher numbers of mixed and diffusely basophilic cell foci were retained after treatment with 24 compared to 12 mg/kg of NNM at all time points studied, and there was even a slow additional increase in the number of these types of FAH 20 weeks after withdrawal of NNM. At both dose levels, the volume fraction of the persistent mixed cell foci correlated positively with the incidence of HCA and HCC, suggesting that this phenotype of FAH represents a direct precursor of the neoplastic lesions. Tigroid cell foci, which appeared most frequently after treatment with the lo,ver dose of NNM, were not integrated into the predominant sequence of cellular changes leading to HCC, but they may represent an intermediate stage in a side lineage of this sequence, endowed with the potential to progress at least to HCA. Our results show that reversion-linked phenotypic instability of FAH occurs mainly after high dose treatment, possibly resulting from rapid adaptive cellular responses to the primary carcinogenic lesion(s) which may be fixed by genetic or epigenetic mechanisms. In contrast, progression-linked phenotypic instability of FAH is a slow process developing in a dose- and time-dependent manner at all dose levels leading to hepatic neoplasia.", 
"Carcinogenesis is a complex process characterized by the cumulative activation of various oncogenes and the inactivation of suppressor genes. Epigenetic mechanisms are also involved. Mutational activation of ras family genes occurs in most spontaneous or carcinogen-induced liver tumors, in susceptible mice, and less frequently in preneoplastic lesions. This suggests a pathogenetic role of these changes in hepatic carcinogenesis, in the mouse. Overexpression of various growth-related genes occurs in preneoplastic tissue during rat liver carcinogenesis, but mutational activation of protooncogenes, notably of ras family genes, seems to be a late and rare event, while c-myc amplification is a late but frequent event in both rodent and human carcinogenesis. However, mutation of the suppressor p53 gene has been found in relatively early preneoplastic lesions in rat liver, and it may be frequently seen in human hepatocellular carcinomas. The possibility that this mutation is involved in the initiation stage of liver carcinogenesis is an attractive hypothesis which needs further evaluation. DNA hypomethylation is involved in carcinogenesis, but the mechanisms underlying this effect are still elusive. Hypomethylation of growth-related genes is associated with their overexpression and this could favor overgrowth of preneoplastic liver tissue. Decrease in S-adenosyl methionine/S-adenosylhomocysteine (SAM/SAH) ratio occurs in the liver of rats fed a methyl deficient diet, which is a carcinogenic treatment, and in preneoplastic liver tissue, developing in initiated/promoted rats fed an adequate diet. The role of low SAM/SAH ratio in carcinogenesis is substantiated by the tumor chemopreventive effect of lipotropic compounds. Treatment with exogenous SAM prevents the development of preneoplastic and neoplastic lesions in rat liver. This is associated with recovery of SAM/SAH ratio, DNA methylation and inhibition of growth-related gene expression. SAM effect on prenoplastic cell growth is abolished by 5-azacytidine, a hypomethylating agent, indicating the involvement of DNA methylation. The possibility that in SAM-treated rats, methylation and inhibition of the expression of growth-related genes is implicated in growth restraint is attractive and should be further evaluated. Modulation of rat liver carcinogenesis by influencing gene expression through DNA methylation or other epigenetic mechanisms could be a new approach to chemoprevention of these tumors.", 
"Isolated limb perfusion with melphalan is a long-standing treatment for melanoma but the clinical conditions have not been subjected to a systematic evaluation. In order to establish optimal conditions for perfusion, three human melanoma cell lines were cultured with melphalan in vitro under conditions comparable to in vivo therapy. The most important findings were that: (a) 41.5-degrees-C was synergistic for melphalan killing of three human melanoma cell lines; (b) prolonging the treatment time beyond 1 h had little additional toxicity; and (c) varying the initial pH of the culture medium had no effect. After 1 h of treatment, cells accumulated more melphalan at 41.5-degrees-C than at 37-degrees-C, relative to the extracellular concentration. A cell line (MM418) derived from a primary tumour was the most resistant of the three lines; pigmented or non-pigmented sublines were equally resistant. The A2058 line showed the lowest level of synergism with hyperthermia, and displayed a marked plateau at 10% of controls in the dose-response for survival, yet no melphalan-resistant subpopulation could be isolated. The implications of this work are that (a) enhanced cellular uptake of melphalan may account for hyperthermic synergism of melphalan; (b) varying conditions other than treatment time will be necessary to deal with the variation in resistance between tumours; and (c) repeated cycles of treatment may be needed for phenotypes such as A2058 where melphalan resistance appears to be based on an epigenetic mechanism.", 
"Regulation of the expression of the DNA repair enzyme O6-methylguanine-DNA methyltransferase (MGMT) has been investigated in a number of human lymphoblastoid cell lines. In a number of Mex- cell lines that do not express methyltransferase activity, CpG sequences in the mgmt gene were hypomethylated with respect to methyltransferase-expressing Mex+ lines. In the cell line GM1953(S), in which the mgmt gene is coregulated with the thymidine kinase and galactokinase genes, reexpression of all three activities was experimentally induced. In this case, the mgmt gene in the nonexpressing cells was found to be hypermethylated and underwent a demethylation at CpG sequences that was coincident with the reappearence of the mgmt mRNA and the three enzyme activities. The simultaneous silencing of three activities in these cells was correlated with an increase in DNA 5-methylcytosine that was widespread thoughout the genome. The data indicate that MGMT expression can be controlled epigenetically in human lymphoid cell lines, although the relationship between cytosine methylation and MGMT expression is complex. Furthermore, the rapid alterations in methylation in GM1953(S) cells indicate the existence of signals that can induce widespread and abrupt alterations in cytosine methylation in human cells in culture.", 
"Long-term culturing of viral Kirsten-ras-oncogene (v-Ki-ras)-transformed BALB/c 3T3 cells (KiMSV) selectively enriches for cells which have deleted the viral oncogene. In contrast, v-Ki-ras in vivo is amplified and expression increased in all late-stage tumors and lung metastases relative to late-passage KiMSV cells being injected. The nature and significance of these selection processes, in terms of the v-Ki-ras gene, have been explored using genetically-tagged cells, as have the properties of v-Ki-ras- revertant subclones. Inoculation of KiMSV late-passage cells (containing < 5% v-Ki-ras+ cells) into nude mice, generated primary and lung metastatic tumors with the v-Ki-ras gene at increased dosage in all tumors and their single-cell clones, isolated at both early and late stages of tumor developments; this demonstrates early and specific in vivo selection for v-Ki-ras+ cells in both induction and progression of tumors in this system. v-Ki-ras- revertant subclones, isolated from the late-passage KiMSV cells and inoculated individually into athymic nude mice, yielded tumors for only 1 of the 4 revertants, with no evidence for v-Ki-ras sequences in these tumor cells, thereby revealing a v-Ki-ras-independent mechanism for tumor formation in a small subset of revertant cells. Mixtures of the 4 subclones yielded tumors in all animals, although at a much longer latent period than observed with v-Ki-ras+ cells. Experiments with mixtures of v-Ki-ras- revertant cells and pSV2neotagged/v-Ki-ras+ cells (both complex Neo(R) cell mixtures and individual Neo(R) clones tested) at various cell ratios revealed clonal variability among v-Ki-ras+ cells for dominance during tumor formation. Moreover, the complex Neo(R) cell mixtures yielded both primary and metastatic tumors with simplified patterns of pSV2neo integration sites, suggesting that secondary genetic or epigenetic events, in addition to v-Ki-ras, contribute to the tumor-progressing phenotype. These experiments taken together demonstrate (a) clone-specific early selection of distinct v-Ki-ras+ cells amongst themselves and over v-Ki-ras- cells in both induction and progression of tumors, (b) reduced tumorigenic competence of v-Ki-ras- revertant cells, with a small subset displaying a v-Ki-ras-independent mechanism for tumor formation in this BALB/c 3T3 system, and (c) the significance of additional genetic or epigenetic events for tumor-progressing competence in unique subsets of v-Ki-ras+ cells.", 
"Emerging evidence suggests that breast cancer may originate during early life. In particular, offspring of mothers who during pregnancy exhibited behaviors that are associated with increased incidence of breast cancer, may be at risk. These behaviors include intake of high fat diet or alcohol, or stressful life style. We have found that neonatal exposure to handling that leads to improved ability to cope with stress, reduces 7,12-dimethylbenz(a)anthracene (DMBA)-induced mammary tumors in rats. Further, our results indicate that maternal exposure to high fat diet increases the incidence of DMBA-induced mammary tumors in female offspring. High fat diet also increases serum 17 beta-estradiol (E2) levels in pregnant animals. These results support the hypothesis that in utero concentrations of estrogens play a critical role in the vulnerability to develop boast cancer. The mechanism of estrogen action might be related to its effect on the induction of epithelial hyperplasia and altered breast differentiation. These events then increase the rate of genetic/epigenetic changes that increase the possibility of neoplastic transformation. Increased pregnancy estrogens may also lead to behavioral alterations in the offspring. This could explain the proposed association between certain behavioral patterns and increased tumorigenity. Our results in transgenic mice overexpressing transforming growth factor alpha (TGF alpha) are in accordance with this interpretation. The male TGF alpha mice exhibit elevated serum E2 levels, impaired ability to cope with stress, increased voluntary alcohol intake and high incidence of spontaneous hepatocellular tumors. These findings indicate that animal models offer a unique opportunity to investigate the role of timing of risk behaviors on breast cancer. They are also useful in the attempts to understand the mechanism of early estrogen action on mammary tumorigenesis.", 
"Metastasis is a complex non-stochastic process that is most likely the result of genetic and epigenetic interactions of a wide variety of genes. The search for a single gene which can encompass such a pleiotropic response as to account for the observed phenotypic characteristics of metastatic tumour populations has been unsuccessful. Particular studies involving gene transfection, subtractive hybridisation and cell fusion are beginning to identify specific genes which contribute to metastasis in some cell types. However, such analyses are complicated by the inherent genetic instability and phenotypic heterogeneity present in tumour populations. A more detailed understanding of the metastatic process may require an abandoning of current generalised approaches to metastasis in favour of concentrating on key components of the metastatic cascade such as adhesion and invasion.", 
"Models of animal melanoma are still indispensable tools in oncological research because they can be studied in syngeneic hosts while human melanomas cannot. One such model is a family of melanomas which originated in 1959 from a spontaneous melanotic melanoma of the skin in Syrian hamster, the Bomirski melanomas. Currently, it consists of 5 transplantable in vivo-variants (Ma, Ab, MI, MI-B and Ab455) and several cell lines derived from Ab amelanotic and Ma melanotic tumors. Here we present the Bomirski family of melanomas as an excellent model to study regulatory mechanisms governing melanogenesis, differentiation and intermediary metabolism in melanocytes, and as a good animal model for developing and testing potential melanoma vaccines.", 
"Genomic imprinting results in a functional non-equivalence of parental chromosomes, presumably by epigenetic modification of the genome, and is required for normal mammalian development. In general, reciprocal phenotypes are observed in embryos containing alterations in the dosage of parental chromosomes, for example where both copies of chromosomes or chromosomal regions are derived from one parent. These phenotypes indicate that duplications of maternal chromosomes inhibit embryonic growth and proliferation whereas duplications of the paternal genome result in enhanced cell growth and proliferation. Alterations in the dosage of parental chromosomes have recently been observed in some forms of recessive tumour in man. Here we discuss the role and possible mechanisms of genomic imprinting during embryogenesis and attempt to draw parallels between the parental origin of the loss of heterozygosity observed in some human tumours and the developmental phenotypes that arise in mice with similar distortions of parental origin. These observations strongly implicate genomic imprinting in the genesis of some forms of tumour and, more generally, in the genetic predisposition to cancer.", 
"The prevention of cancer is, at our present state of knowledge, the most effective and inexpensive mode of controlling this disease. Passive cancer prevention is practiced voluntarily or, in part, through governmental regulation in many parts of the world. However, active cancer prevention by the administration of vaccines, dietary factors, antihormones, or other agents is likely to be the most effective mechanism of cancer prevention in humans. Carcinogenesis develops in three defined stages-initiation, promotion, and progression-the first and third resulting from irreversible genetic changes in the cell, whereas the intermediate stage of promotion involves an epigenetic alteration of the expression of the genome and cell division. Thus, the stage of promotion is the most effective site to target for both active and passive cancer prevention. A review of the modalities presently used in cancer prevention, both active and passive, demonstrates that all such methods involve an inhibition and/or reversal of this intermediate stage, thus preventing the development of the malignant process. Future advances in cancer prevention will depend on better knowledge of the mechanisms of the stages of carcinogenesis in order that rational preventive agents and formats may be used in altering or modifying the appropriate stage(s) of neoplastic development.", 
"Tumor progression is a multistep process involving genetic and epigenetic changes in a transformed clone. Some of these changes may be induced by host factors which may also select for transformed cellular variants with a high ability to survive and propagate. In this article we review studies showing that receptors for the Fc portion of IgG may be expressed on cells from human or animal tumors of nonlymphoid origin. We also review data demonstrating that at least with respect to cells transformed in vitro with Polyoma virus, transformation per se is not sufficient for the induction of Fc receptor expression. We also summarize preliminary data showing that Fc receptor expression is causally involved in conferring a high malignancy phenotype upon transformed cells. Possible mechanisms to explain these observations are discussed.", 
"Epigenetic changes may play a role in genetic alterations in cancer cells, but little is known about this phenomenon. In this study we examined the correlation between rearrangement and methylation status of the T-cell receptor (TCR) beta-chain gene in 23 patients with B precursor acute lymphoblastic leukemia (ALL). In B precursor ALL, all patients had a CC(me) GG sequence in the C-beta-2 region, a pattern similar to that observed in normal mature B-cells. Approximately 55% of patients with B precursor ALL exhibiting a hypomethylated CCGG sequence at the J-beta-1 region showed rearrangement of this region. Furthermore, the same allele of rearranged J-beta-1 always contained an unmethylated sequence in the region, although another allele without rearrangement contained methylated J-beta-1. By contrast, none of the patients had a rearrangement in the J-beta-1 region without hypomethylation. Therefore, rearrangement of the J-beta-1 region may link to the hypomethylation status of this region. A close association between hypomethylation and rearrangement of the TCR-beta-chain gene indicates that hypomethylation of the J-beta-1 region may promote accessibility to a putative common recombinase to produce TCR J-beta-1 rearrangement. In contrast, about 45% of patients with a hypomethylated J-beta-1 did not show rearrangement in this region, thus allowing categorization of B precursor ALL patients into subtypes, according to the combination of TCR-beta-chain gene rearrangement and hypomethylation status, especially in the J-beta-1 region.", 
"The thiopurines 6-thioguanine (6TG) and 6-mercaptopurine (6MP) are cytotoxic to proliferating cells by a mechanism involving incorporation into DNA via the purine salvage pathway, and resistance to these agents can be conferred by lack of the salvage pathway enzyme hypoxanthine-guanine phosphoribosyltransferase. However, human and murine hypoxanthine-guanine phosphoribosyltransferase-deficient leukemia cell Lines have been shown to respond to 6TG by growth arrest and differentiation by a mechanism apparently not involving incorporation of 6TG into DNA. If so, leukemia cells resistant to 6MP should still respond to 6TG by growth arrest via an undescribed epigenetic mechanism. To test this, polyclonal 6MP-resistant variants were produced from three human leukemia cell lines, HL-60, U937, and CCRF-CEM. Treatment of both sensitive and resistant cells with 6TG induced growth arrest. The effect of 6TG in the 6MP-sensitive HL-60 and U937 cells was associated with significant loss of viability and DNA fragmentation. In contrast, the 6TG-treated 6MP-resistant cells exhibited a slower decline in viability and no DNA fragmentation. To identify the mechanism by which 6TG may induce growth arrest, tRNA was isolated from 6MP-resistant cells cultured for 48 h with 6TG. 6TG was found to be incorporated into tRNAs normally containing queuine in the anticodon wobble position. These studies may provide a basis for the development of new therapeutic regimens for the treatment of leukemia.", 
"Rapid progress in understanding the biology of colorectal carcinoma has occurred in the last decade. During this time, the importance of tumor suppressor genes has been delineated. Mutations in proto-oncogenes, and some of the genetic and epigenetic defects that occur during neoplastic transformation, have been characterized. The very recent identification of a technology that detects some mutations in genes by analyzing fecal specimens offers the real prospect of effective, low-cost screening of large segments of the population at risk for the development of large bowel cancer. A better understanding of the mechanisms involved in metastasis is slowly leading to better methods of prognostication for patients with carcinoma of the colon or rectum. This is important because a better biologic identification of patients at high risk for subsequent metastasis will help determine which patients should receive adjuvant therapy. Further, the development of new approaches to inhibit various aspects of the cascade of events necessary to produce metastasis, notably die inhibition of neovascularization, suggests the real possibility that established metastases may be treated by relatively nontoxic medical therapies. As a result, further knowledge of the biology of colorectal cancer will translate into strategies that will continue to improve the control of this prevalent carcinoma.", 
"The epigenetic phenomenon could play a role in the interaction between chromatin and DNA-binding enzymes, allowing us to consider an association between the phenomenon and gene rearrangement. The correlation between methylation status and rearrangement of the T-cell receptor (TCR) beta chain gene in leukemia cells obtained from patients with acute myeloid leukemia (AML) was examined. All of the AML patients with a TCR-beta rearrangement had hypomethylated CCGG sequences within the Jbeta1 region on the rearranged allele, while the germ-line allele had completely methylated CC(me)GG sequence in this region, indicating a strong association between hypomethylation status and rearrangement of the TCR beta chain gene. In the DNA from AML patients with or without a TCR-beta rearrangement, the Cbeta2 region contained completely methylated CC(me)GG sequences, even though they express T-cell-associated antigens, including CD7; this pattern is quite different from that observed in T-cell neoplasias. Moreover, some AML patients showed a TCR-beta rearrangement without the presence of immunoglobulin heavy-chain gene rearrangement, suggesting that TCR beta chain gene involvement in AML is required for unknown factors other than common recombinase activity.", 
"Analyses to detect loss of heterozygosity (LOH) were performed at 11 polymorphic loci on chromosome 11 and, using a polymorphic CA repeat sequence in the WT1 gene, on a series of 39 tumours from 28 unilateral and 10 tumours from 6 bilateral Wilms' tumour (WT) patients. LOH was seen in 13 out of 35 patients including 12 out of 29 unilateral tumours, but only one of 10 bilateral tumours. This suggests that bilateral WT represents a subgroup of WT in which tumour initiating events less frequently involve LOH on chromosome 11 and that either epigenetic events, point mutations or another non-chromosome 11p locus are important in bilateral tumours. The observation of LOH in one WT but not another WT in a bilateral WT patient provides evidence that these tumours arising in the same patient are not monoclonal proliferations and most likely arise via different molecular pathways.", 
"Loss of cell-substratum adhesion is an important factor during tumour progression. We have previously described reduced focal contact components and poorly organized cytoskeletal actin in renal cell carcinomas. In this study, we have used the potent tumour promoter TPA on cultured human renal epithelium to mimic neoplastic transformation. The morphological changes induced by TPA were examined by phase contrast and fluorescence microscopy. TPA treatment caused rounding up of cells and loss of adhesion to either fibronectin or laminin substrata. Cytoskeletal actin was redistributed from orientated stress fibre bundles to a perinuclear circumferential arrangement. This was accompanied by a progressive reduction in the number of vinculin-containing contacts with accumulation of vinculin in punctate spots in the perinuclear region. These altered membrane-cytoskeletal interactions induced by TPA are entirely reversible and mimic epigenetic changes which occur during tumour progression.", 
" ", 
"The Philadelphia (Ph) translocation [t(9;22)(q34;q11)] is the most common genetic abnormality in human leukemia; a transposition of the ABL gene to the major-breakpoint cluster region (M-BCR) is associated with the pathogenesis in Ph+ chronic myelogenous leukemia (Ph+ CML) and in some cases of Ph+ acute leukemia (Ph+ AL). Our current understanding of the methylation of human genomes allows us to consider the association between the epigenetic phenomenon and the control of differentiation and proliferation in mammalian cells. In order to determine whether the methylation status of the M-BCR is associated with breakpoint-localization in this region and with the lineage of hematopoietic cells, we have examined 28 patients with Ph+ leukemias, including nine with Ph+ AL, six patients with acute myeloblastic leukemia without Ph (Ph- AML), and five patients with Ph- acute lymphoblastic leukemia (Ph- ALL); using the restriction endonuclease isochizomers, MspI and HpaII. In CML patients in the chronic phase, the hypomethylated status within the normal M-BCR allele is heterogeneous. In contrast, patients with Ph- CML in the lymphoid blast crisis phase exhibited a 2.5/2.7 kb band with a complete disappearance of the germline M-BCR fragment (type L). This pattern is consistently noted in Ph- ALL cells, and the pattern is quite different from that found in myeloid blast crisis or Ph- AML (type M). In patients with M-BCR-nonrearranged Ph+ ALL, it is suggested that the M-BCR methylation patterns are cell-lineage specific but some Ph+ ALL cells had a hypomethylation pattern thal was identical to that observed in Ph- AML, suggesting a distinction of genetic diversity of leukemia cells with the Ph chromosome, especially Ph+ AL.", 
"We karyotyped two histologically distinct components with different metastatic behavior of a testicular nonseminomatous germ cell tumor. The two components showed an almost identical chromosomal pattern. These almost identical karyotypes of the two components with different metastatic potential suggest that the difference in biologic behavior might result from subtle differences (on microscopic or submicroscopic level) in chromosomal pattern or that these differences are predominantly epigenetically determined and depend primarily on the lineage of differentiation of the tumor component. Trophoblastic differentiation results in an aggressive, angioinvasive tumor but in development of teratoma in a tumor with low malignant potential.", 
" ", 
" ", 
]


In [8]:
###import pandas as pd

###df = pd.read_excel (r'C:/Users/brady/Desktop/TM/Evo-Bio All Time/Test1.xlsx')
###print (df)

In [9]:
print(df)

["MicroRNAs (mi-RNAs) are critical regulators of gene expression. Amplification and overexpression of individual 'oncomiRs' or genetic loss of tumour suppressor mi-RNAs are associated with human cancer and are sufficient to drive tumorigenesis in mouse models. Furthermore, global miRNA depletion caused by genetic and epigenetic alterations in components of the miRNA biogenesis machinery is oncogenic. This, together with the recent identification of novel miRNA regulatory factors and pathways, highlights the importance of miRNA dysregulation in cancer.", 'PD-L1 and PD-1 (PD) pathway blockade is a highly promising therapy and has elicited durable antitumor responses and long-term remissions in a subset of patients with a broad spectrum of cancers. How to improve, widen, and predict the clinical response to anti-PD therapy is a central theme in the field of cancer immunology and immunotherapy. Oncologic, immunologic, genetic, and biological studies focused on the human cancer microenviron

In [10]:
###CLEAN DATA###

In [11]:
# Convert to list
data = [simple_preprocess(doc) for doc in df]

In [12]:
print(data)

[['micrornas', 'mi', 'rnas', 'are', 'critical', 'regulators', 'of', 'gene', 'expression', 'amplification', 'and', 'overexpression', 'of', 'individual', 'oncomirs', 'or', 'genetic', 'loss', 'of', 'tumour', 'suppressor', 'mi', 'rnas', 'are', 'associated', 'with', 'human', 'cancer', 'and', 'are', 'sufficient', 'to', 'drive', 'tumorigenesis', 'in', 'mouse', 'models', 'furthermore', 'global', 'mirna', 'depletion', 'caused', 'by', 'genetic', 'and', 'epigenetic', 'alterations', 'in', 'components', 'of', 'the', 'mirna', 'biogenesis', 'machinery', 'is', 'oncogenic', 'this', 'together', 'with', 'the', 'recent', 'identification', 'of', 'novel', 'mirna', 'regulatory', 'factors', 'and', 'pathways', 'highlights', 'the', 'importance', 'of', 'mirna', 'dysregulation', 'in', 'cancer'], ['pd', 'and', 'pd', 'pd', 'pathway', 'blockade', 'is', 'highly', 'promising', 'therapy', 'and', 'has', 'elicited', 'durable', 'antitumor', 'responses', 'and', 'long', 'term', 'remissions', 'in', 'subset', 'of', 'patients'

In [13]:
###MAY NEED CLEANING STEP, MAY NOT BECAUSE ABSTRACTS ARE SO CLEAN###

In [14]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['micrornas', 'mi', 'rnas', 'are', 'critical', 'regulators', 'of', 'gene', 'expression', 'amplification', 'and', 'overexpression', 'of', 'individual', 'oncomirs', 'or', 'genetic', 'loss', 'of', 'tumour', 'suppressor', 'mi', 'rnas', 'are', 'associated', 'with', 'human', 'cancer', 'and', 'are', 'sufficient', 'to', 'drive', 'tumorigenesis', 'in', 'mouse', 'models', 'furthermore', 'global', 'mirna', 'depletion', 'caused', 'by', 'genetic', 'and', 'epigenetic', 'alterations', 'in', 'components', 'of', 'the', 'mirna', 'biogenesis', 'machinery', 'is', 'oncogenic', 'this', 'together', 'with', 'the', 'recent', 'identification', 'of', 'novel', 'mirna', 'regulatory', 'factors', 'and', 'pathways', 'highlights', 'the', 'importance', 'of', 'mirna', 'dysregulation', 'in', 'cancer']]


In [15]:
###THIS STEP NEEDS TO BE CAREFULLY MONITORED TO MAKE SURE IT'S WORKING AS INTENDED###

In [16]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['micrornas', 'mi', 'rnas', 'are', 'critical', 'regulators', 'of', 'gene', 'expression', 'amplification', 'and', 'overexpression', 'of', 'individual', 'oncomirs', 'or', 'genetic', 'loss', 'of', 'tumour', 'suppressor', 'mi', 'rnas', 'are', 'associated', 'with', 'human', 'cancer', 'and', 'are', 'sufficient', 'to', 'drive', 'tumorigenesis', 'in', 'mouse', 'models', 'furthermore', 'global', 'mirna', 'depletion', 'caused', 'by', 'genetic', 'and', 'epigenetic', 'alterations', 'in', 'components', 'of', 'the', 'mirna', 'biogenesis', 'machinery', 'is', 'oncogenic', 'this', 'together', 'with', 'the', 'recent', 'identification', 'of', 'novel', 'mirna', 'regulatory', 'factors', 'and', 'pathways', 'highlights', 'the', 'importance', 'of', 'mirna', 'dysregulation', 'in', 'cancer']


In [17]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [18]:
###en_core_web_sm needs to be explicitly loaded and I believe you need to use "python -m spacy download en" in cmd.exe to properly load everything.###

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['microrna', 'rna', 'critical', 'regulator', 'gene', 'expression', 'amplification', 'overexpression', 'individual', 'oncomir', 'genetic', 'loss', 'associate', 'human', 'cancer', 'sufficient', 'drive', 'tumorigenesis', 'mouse', 'model', 'furthermore', 'global', 'mirna', 'depletion', 'cause', 'genetic', 'epigenetic', 'alteration', 'component', 'biogenesis', 'machinery', 'oncogenic', 'together', 'recent', 'identification', 'novel', 'mirna', 'regulatory', 'factor', 'pathway', 'highlight', 'importance', 'mirna', 'dysregulation', 'cancer']]


In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 3), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]]


In [21]:
#to call an individual dictionary term switch value as appropriate
id2word[0]

'alteration'

In [22]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('alteration', 1),
  ('amplification', 1),
  ('associate', 1),
  ('biogenesis', 1),
  ('cancer', 2),
  ('cause', 1),
  ('component', 1),
  ('critical', 1),
  ('depletion', 1),
  ('drive', 1),
  ('dysregulation', 1),
  ('epigenetic', 1),
  ('expression', 1),
  ('factor', 1),
  ('furthermore', 1),
  ('gene', 1),
  ('genetic', 2),
  ('global', 1),
  ('highlight', 1),
  ('human', 1),
  ('identification', 1),
  ('importance', 1),
  ('individual', 1),
  ('loss', 1),
  ('machinery', 1),
  ('microrna', 1),
  ('mirna', 3),
  ('model', 1),
  ('mouse', 1),
  ('novel', 1),
  ('oncogenic', 1),
  ('oncomir', 1),
  ('overexpression', 1),
  ('pathway', 1),
  ('recent', 1),
  ('regulator', 1),
  ('regulatory', 1),
  ('rna', 1),
  ('sufficient', 1),
  ('together', 1),
  ('tumorigenesis', 1)]]

In [23]:
###Loads of room for customization at this step. Read carefully and adjust accordingly.###

In [24]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=14, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [25]:
# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.028*"maternal" + 0.027*"cervical" + 0.021*"parental" + 0.018*"myc" + '
  '0.018*"environmental" + 0.015*"paternal" + 0.011*"gap" + 0.010*"offspring" '
  '+ 0.009*"determinant" + 0.009*"health"'),
 (1,
  '0.050*"tumor" + 0.019*"experimental" + 0.019*"spontaneous" + 0.014*"nfatc" '
  '+ 0.012*"heterogeneity" + 0.012*"sensitivity" + 0.010*"yet" + 0.010*"cell" '
  '+ 0.010*"fat" + 0.009*"differentiation"'),
 (2,
  '0.018*"gene" + 0.015*"epigenetic" + 0.014*"lineage" + '
  '0.012*"differentiation" + 0.011*"leukemia" + 0.011*"modification" + '
  '0.008*"methylate" + 0.008*"protein" + 0.008*"factor" + 0.007*"cell"'),
 (3,
  '0.079*"cell" + 0.030*"line" + 0.015*"expression" + 0.013*"tissue" + '
  '0.011*"tumor" + 0.011*"cancer" + 0.010*"liver" + 0.009*"gene" + '
  '0.009*"tumour" + 0.008*"vivo"'),
 (4,
  '0.042*"immune" + 0.022*"endogenous" + 0.020*"response" + 0.016*"function" + '
  '0.015*"regression" + 0.015*"tumor" + 0.013*"macrophage" + 0.013*"antigen" + '
  '0.012*"host" + 0.01

In [26]:
###Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. 
###In my experience, topic coherence score, in particular, has been more helpful.

###Need to learn more about these values.

In [27]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.802441860970765

Coherence Score:  0.35301100000333624


In [28]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\brady\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13     0.244006 -0.036341       1        1  30.222928
10     0.272133  0.006025       2        1  23.555986
9      0.176611  0.113435       3        1  13.412392
3      0.155091  0.145937       4        1  11.924628
5      0.115364 -0.185986       5        1   7.716033
2      0.039234 -0.176548       6        1   5.300573
1     -0.041971  0.057304       7        1   2.261112
6     -0.075751  0.075295       8        1   2.143079
4     -0.122574  0.036875       9        1   1.700558
0     -0.158221 -0.015516      10        1   1.017089
7     -0.166958 -0.005828      11        1   0.433891
12    -0.151943 -0.005889      12        1   0.195141
8     -0.143801 -0.004412      13        1   0.080439
11    -0.141221 -0.004351      14        1   0.036151, topic_info=            Term        Freq       Total Category  logprob  loglift
48          cell  856.000000  856.000000  Default  30.0000  30.0000
411  methylation  327.000000  327.000000  Default  29.0000  29.0000
89         tumor  359.000000  359.000000  Default  28.0000  28.0000
394          dna  355.000000  355.000000  Default  27.0000  27.0000
15          gene  523.000000  523.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
891     adaptive    0.002631   11.708162  Topic14  -8.3468  -0.4754
331      however    0.002631   71.445195  Topic14  -8.3468  -2.2841
590   occurrence    0.002631    5.946470  Topic14  -8.3468   0.2020
151       target    0.002631   96.012353  Topic14  -8.3468  -2.5796
12    expression    0.002631  294.834531  Topic14  -8.3468  -3.7016

[771 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
658       1  0.892383     ability
658       7  0.077599     ability
890       4  0.546049     absence
890       6  0.397127     absence
556       6  0.728848      accord
...     ...       ...         ...
1870      3  0.108355  widespread
1870      5  0.758482  widespread
2609      8  0.921629         wnt
976       5  0.110645         yet
976       7  0.774516         yet

[1123 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 11, 10, 4, 6, 3, 2, 7, 5, 1, 8, 13, 9, 12])

In [ ]:
###rename file.html before running command. Search computer and it should be in the user directory.
p = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(p, 'file.html')